In [3]:
import typing
import pandas as pd
import re
import functools
import numpy as np
import math

In [4]:
class Token:
    def __init__(self, type: str, value: str):
        self.type = type
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, '{self.value}')"

In [5]:
class ParseTreeNode:
    
    def __init__(self, type: str, value: typing.Optional[str] = None, children: typing.Optional[typing.List["ParseTreeNode"]] = None):
        self.type = type
        self.value = value
        self.children = children or []
        self.start_index = None
        self.end_index = None
        self.parse_input = None

    def __repr__(self):
        if self.parse_input is not None:
            return f"ParseTreeNode({self.type}, value={self.value}, children={self.children}, start_index={self.start_index}, end_index={self.end_index})"
        return self.parse_input
    
    def reify(self, function_factory):
        if self.type == "factor" and len(self.children) == 1 and self.children[0].type == "identifier":
            identifier_node = self.children[0]
            name = identifier_node.value

            # we need to use late binding
            f = function_factory.get("Dereference")
            pkeys = [a for a in f.parameters.keys()]
            params = {}
            params[pkeys[0]] = name
            return f.create_function(**params)            

        
        if (self.type == "term") and len(self.children) == 4 and self.children[0].type == "factor" and self.children[1].value == "[" and self.children[2].type == "expression" and self.children[3].value == "]":
            array = self.children[0].reify(function_factory)
            index = self.children[2].reify(function_factory)
            f = function_factory.get("Lookup")
            params = {}
            pkeys = [a for a in f.parameters.keys()]
            params[pkeys[0]] = array
            params[pkeys[1]] = index
            return f.create_function(**params)
            
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "+" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 + arg1
            else:
                # we need to use late binding
                f = function_factory.get("Add")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if we want to do away with the requirement for parentheses
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "-" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 - arg1
            else:
                # we need to use late binding
                f = function_factory.get("Sub")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if we want to do away with the requirement for parentheses
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "&&" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 and arg1
            else:
                # we need to use late binding
                f = function_factory.get("And")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if we want to do away with the requirement for parentheses
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "||" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 or arg1
            else:
                # we need to use late binding
                f = function_factory.get("or")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if(self.type == "expression") and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "*" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 * arg1
            else:
                # we need to use late binding
                f = function_factory.get("Mul")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        # TODO: this doesn't have the associative property, so it's necessary to flatten the tree if this is actually the plan
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "/" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 / arg1
            else:
                # we need to use late binding
                f = function_factory.get("Div")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "%" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 % arg1
            else:
                # we need to use late binding
                f = function_factory.get("Mod")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "**" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 ** arg1
            else:
                # we need to use late binding
                f = function_factory.get("Pow")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "<" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 < arg1
            else:
                # we need to use late binding
                f = function_factory.get("Lt")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "<=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 <= arg1
            else:
                # we need to use late binding
                f = function_factory.get("Le")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == ">" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 > arg1
            else:
                # we need to use late binding
                f = function_factory.get("Gt")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == ">=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 >= arg1
            else:
                # we need to use late binding
                f = function_factory.get("Ge")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "==" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 == arg1
            else:
                # we need to use late binding
                f = function_factory.get("Eq")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
        if (self.type == "expression") and len(self.children) == 3 and self.children[0].type == "term" and self.children[1].value == "!=" and self.children[2].type == "term":
            arg0 = self.children[0].reify(function_factory)
            arg1 = self.children[2].reify(function_factory)
            if(isinstance(arg0, int) or isinstance(arg0, float)) and (isinstance(arg1, int) or isinstance(arg1, float)):
                return arg0 != arg1
            else:
                # we need to use late binding
                f = function_factory.get("Ne")
                params = {}
                pkeys = [a for a in f.parameters.keys()]
                params[pkeys[0]] = arg0
                params[pkeys[1]] = arg1
                return f.create_function(**params)
                
        if self.type == "term" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
            
        if self.type == "factor" and len(self.children) == 1:
            return self.children[0].reify(function_factory)
        if self.type == "factor" and len(self.children) == 3 and self.children[0].value == "(" and self.children[2].value == ")":
            return self.children[1].reify(function_factory)
            
        if self.type == "number" and (self.children is None or len(self.children) == 0):
            try:
                return int(self.value)
            except:
                return float(self.value)
        if self.type == "string" and self.children is None or len(self.children) == 0:
            return self.value
        
        if self.type == "factor" and len(self.children) == 4 and self.children[0].type == "identifier" and self.children[1].value == "(" and self.children[2].type == "arguments" and self.children[3].value == ")":

            identifier_node = self.children[0]
            name = identifier_node.value

            try:
                definition = function_factory.get(name)
            except ValueError:
                raise ValueError(f"Function '{name}' not found in the factory.")

            params = {}

            arguments_node = self.children[2]
            param_index = 0
            param_names = list(definition.parameters.keys())

            # Handle named arguments first
            named_params_processed = set()  # Keep track of named params

            def flatten_arguments(arguments_node):
                rv = []
                if arguments_node.children[0].type == "argument":
                    rv.append(arguments_node.children[0])
                    if(len(arguments_node.children) == 3 and arguments_node.children[1].value == ","):
                        rv.extend(flatten_arguments(arguments_node.children[2]))
                return rv
                
            flattened_arguments = flatten_arguments(arguments_node)

            #print(flattened_arguments)
                
            for argument_node in flattened_arguments:
                if len(argument_node.children) == 3 and argument_node.children[1].type == "operator" and argument_node.children[1].value == "=":
                    param_name = argument_node.children[0].value
                    param_value_node = argument_node.children[2]

                    if param_name in named_params_processed: # Skip already processed named parameters
                        continue

                    try:
                        param_def = definition.parameters[param_name]
                    except KeyError:
                        raise ValueError(f"Parameter '{param_name}' not found for indicator '{name}'.")

                    param_value = param_value_node.reify(function_factory)  # Evaluate the value node
                    params[param_name] = param_value
                    named_params_processed.add(param_name) # Add to the set of processed named parameters

            # Next, handle positional arguments (skip named ones)
            for argument_node in flattened_arguments:
                if len(argument_node.children) == 1:  # Positional argument
                    try:
                        param_name = param_names[param_index]
                        if param_name in named_params_processed: # Skip if already named
                            param_index += 1
                            continue

                        param_def = definition.parameters[param_name]
                    except IndexError:
                        raise ValueError(f"Incorrect number of positional parameters for '{name}'.")

                    param_value_node = argument_node.children[0]
                    param_value = param_value_node.reify(function_factory)
                    params[param_name] = param_value
                    param_index += 1



            required_params = set(definition.parameters.keys())
            provided_params = set(params.keys())
            if required_params != provided_params:
                missing = required_params - provided_params
                raise ValueError(f"Missing required parameters for {name}: {missing}")

            return definition.create_function(**params)
#        raise ValueError(f"Cannot reify node of type: {self.type} with {len(self.children)} children: {self}")
        raise ValueError(f"Cannot reify node of type: {self.type} with children: {self}")
        

In [6]:
class GrammarRule:
    def __init__(self, left: str, right: typing.List[str]):
        self.left = left
        self.right = right

    def __repr__(self):
        return f"{self.left} -> {' '.join(self.right)}"

In [7]:
class Grammar:
    def __init__(self, grammar_string: str):
        """Initializes a Grammar object by parsing the grammar string."""
        self.rules = []
        for line in grammar_string.strip().splitlines():
            if line.strip():  # Skip empty lines
                parts = line.split("->")
                if len(parts) != 2:
                    raise ValueError(f"Invalid grammar rule: {line}")
                left = parts[0].strip()
                right = [part.strip() for part in parts[1].split()]
                self.rules.append(GrammarRule(left, right))  # Store rules as attributes

    def build_parse_tree(self, tokens: typing.List["Token"], start_symbol: str = "expression") -> typing.Optional["ParseTreeNode"]:
        """Builds a parse tree from a list of tokens using the grammar rules."""

        def _parse(index: int, nonterminal: str, current_depth=0) -> typing.Optional["ParseTreeNode"]:
            applicable_rules = [rule for rule in self.rules if rule.left == nonterminal]

            if index >= len(tokens):  # End of tokens
                if any(not rule.right for rule in applicable_rules): # Check for a matching epsilon rule
                    return ParseTreeNode(nonterminal, children=[])
                return None # No matching epsilon rule

            if not applicable_rules:
                return None

            for rule in applicable_rules:
                rule_matched = True
                children = []
                current_index = index

                for symbol in rule.right:
                    if current_index >= len(tokens):
                        rule_matched = False
                        break

                    if current_index < len(tokens):
                        token = tokens[current_index]

                        if (symbol == token.type) or (symbol == f'"{token.value}"') or \
                           (symbol == "identifier" and token.type == "identifier") or \
                           (symbol == "number" and token.type == "number") or \
                           (symbol == "string" and token.type == "string") or \
                           (symbol == "operator" and token.type == "operator"):
                            child = ParseTreeNode(token.type, value=token.value)
                            child.start_index = current_index
                            child.end_index = current_index
                            children.append(child)
                            current_index += 1  # Increment for terminal

                        elif any(gr.left == symbol for gr in self.rules):
                            child_node = _parse(current_index, symbol, current_depth + 1)
                            if child_node:
                                children.append(child_node)
                                current_index = child_node.end_index + 1
                            else:
                                rule_matched = False
                                break

                        else:
                            rule_matched = False
                            break

                if rule_matched:
                    node = ParseTreeNode(nonterminal, children=children)
                    node.start_index = children[0].start_index if children else index # Handle epsilon rules where children is empty
                    node.end_index = children[-1].end_index if children else index -1 # Handle epsilon rules where children is empty

                    return node

            return None

        return _parse(0, start_symbol)  # Allow specifying the start symbol
        
    def parse(self, input_string: str, start_symbol: str = "expression"):
        """Parses an input string into a parse tree."""
        tokens = self.tokenize(input_string)  # Tokenize the input string
        return self.build_parse_tree(tokens, start_symbol)

    def tokenize(self, expression: str) -> typing.List[Token]:
        """
        Tokenizes a string expression, splitting on spaces and identifying operators.
        """
    
        # Pattern to match tokens. Note: if we wanted to be really fancy, we would specify the token types in the grammar.
        pattern = r"(\*\*|\*|/|//|%|\+|-|==|!=|<=|>=|<|>|=|!|&&|\|\||&|\||\^|~|<<|>>|\(|\)|\[|\]|\{|\}|,|:|\.|->|@|=|;|\+=|-=|\*=|/=|//=|%=|&=|\|=|\^=|\<<=|>>=)|'([^']+)'|\"([^\"]+)\"|(\d+\.?\d*)|([a-zA-Z_]\w*)"
    
        tokens = []
        for match in re.finditer(pattern, expression):
            operator_match = match.group(1)
            single_quote_match = match.group(2)
            double_quote_match = match.group(3)
            number_match = match.group(4)
            identifier_match = match.group(5)
    
            if operator_match:
                tokens.append(Token("operator", operator_match))
            elif single_quote_match:
                tokens.append(Token("string", single_quote_match))
            elif double_quote_match:
                tokens.append(Token("string", double_quote_match))
            elif number_match:
                tokens.append(Token("number", number_match))
            elif identifier_match:
                tokens.append(Token("identifier", identifier_match))
            else:
                raise ValueError(f"invalid token in {expression}")
    
        return tokens

In [8]:
class FunctionInstance:
    def __init__(self, name: str, parameters: typing.Dict[str, typing.Any], definition):
        self.name = name
        self.parameters = parameters
        self.definition = definition

    def evaluate_parameters(self, data):
        rv = {}

        for k in self.parameters:
            v = self.parameters[k]
            if isinstance(v, FunctionInstance):
                rv[k] = v.calculate(data)
            else:
                rv[k] = v
        return rv

    def calculate(self, data: pd.DataFrame): 
        """
        Screens the data using the screener's definition and parameters.

        Args:
            data: The Pandas DataFrame containing the data.

        Returns:
            A Pandas Dataframe
        """
        return self.definition.calculate(data, self.evaluate_parameters(data)) 

    def __repr__(self):
        params_str = ", ".join(f"{name}={value}" for name, value in self.parameters.items())
        return f"{self.definition.name}({params_str})"

In [9]:
class FunctionDefinition:
    def __init__(self, name: str, parameters: typing.Dict[str, "ParameterType"], calculation_function, factory=None): 
        if not isinstance(name, str):
            raise TypeError("name must be a string")

        if not isinstance(parameters, dict):
            raise TypeError("parameters must be a dictionary")

        if not all(isinstance(param, ParameterType) for param in parameters.values()):
            raise TypeError("All values in parameters must be ParameterType objects")

        if len(set(parameters.keys())) != len(parameters.keys()): # Check for duplicate keys
            raise ValueError("Parameter names must be unique.")

        if not callable(calculation_function):
            raise TypeError("calculation_function must be callable")

        self.name = name
        self.parameters = parameters
        self.calculation_function = calculation_function
        self.factory = factory

    def create_function(self, **kwargs: typing.Any) -> "FunctionInstance":
        params = {}
        for name, param_def in self.parameters.items():
            value = kwargs.get(name)

            if value is None:
                value = param_def.get_default()

            if param_def.data_type == "integer" and not isinstance(value, int):
                raise TypeError(f"Value for parameter '{name}' must be an integer")
            elif param_def.data_type == "real" and not isinstance(value, (int, float)):
                raise TypeError(f"Value for parameter '{name}' must be a number")
            elif param_def.data_type == "boolean" and not isinstance(value, bool):
                raise TypeError(f"Value for parameter '{name}' must be a boolean")
            elif param_def.data_type == "string" and not isinstance(value, str):
                raise TypeError(f"Value for parameter '{name}' must be a string")
            elif param_def.data_type in ("integer", "real"):
                if param_def.min_val is not None and value < param_def.min_val:  # Check min_val
                    raise ValueError(f"Value for parameter '{name}' must be greater than or equal to {param_def.min_val}")
                if param_def.max_val is not None and value > param_def.max_val:  # Check max_val
                    raise ValueError(f"Value for parameter '{name}' must be less than or equal to {param_def.max_val}")

            if param_def.data_type == "string" and param_def.allowed_strings is not None and value not in param_def.allowed_strings:
                raise ValueError(f"Value {value} is not in allowed strings for parameter {name}")

            params[name] = value

        return FunctionInstance(self.name, params, self)

    def calculate(self, data: pd.DataFrame, params: typing.Dict[str, typing.Any]) -> pd.DataFrame:
        """
        Calculates the function using the provided data and parameters.
        """
        kwargs = params.copy() 
        return self.calculation_function(data, **kwargs)

    def __repr__(self):
        return f"FunctionDefinition(name='{self.name}', parameters={self.parameters}, calculation_function={self.calculation_function.__name__ if hasattr(self.calculation_function, '__name__') else str(self.calculation_function)}, factory={self.factory})"

    

In [10]:
class ParameterType:
    """
    A class for specifying parameters for screeners and indicators.
    """

    def __init__(self,
#                 name: str,
                 data_type: typing.Literal["integer", "real", "boolean", "string"],
                 min_val: typing.Union[int, float, None] = None,
                 max_val: typing.Union[int, float, None] = None,
                 default: typing.Any = None,
                 timeframe_defaults: typing.Dict[typing.Literal["tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"], typing.Any] = None,
                 increment: typing.Union[int, float, None] = None,
                 allowed_strings: typing.List[str] | None = None):
#        if not isinstance(name, str):
#            raise TypeError("name must be a string")
        if data_type not in ("integer", "real", "boolean", "string", "any"):
            raise ValueError("data_type must be 'integer', 'real', 'boolean', 'string', or 'any'")

        if min_val is not None:
            if data_type == "integer" and not isinstance(min_val, int):
                raise TypeError("min_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(min_val, (int, float)):
                raise TypeError("min_val must be a number for real or integer data_type")

        if max_val is not None:
            if data_type == "integer" and not isinstance(max_val, int):
                raise TypeError("max_val must be an integer for integer data_type")
            elif data_type in ("real", "integer") and not isinstance(max_val, (int, float)):
                raise TypeError("max_val must be a number for real or integer data_type")

        if timeframe_defaults is not None:
            if not isinstance(timeframe_defaults, dict):
                raise TypeError("timeframe_defaults must be a dictionary")
            for timeframe in timeframe_defaults:
                if timeframe not in ("tick", "1s", "5s", "15s", "1m", "2m", "5m", "15m", "1d", "1w", "1M"):
                    raise ValueError(f"Invalid timeframe: {timeframe}")

        if data_type == "integer" and increment is None:
            increment = 1
        elif data_type == "real" and increment is None:
            increment = 0.01

        if data_type == "string" and allowed_strings is not None and not isinstance(allowed_strings, list):
          raise TypeError("allowed_strings must be a list of strings")

        if data_type != "string" and allowed_strings is not None:
          raise ValueError("allowed_strings can only be specified for string data type")

#        self.name = name
        self.data_type = data_type
        self.min_val = min_val
        self.max_val = max_val
        self.default = default
        self.timeframe_defaults = timeframe_defaults or {}
        self.increment = increment
        self.allowed_strings = allowed_strings

    def get_default(self) -> typing.Any:
        return self.default

    def get_possible_values(self) -> typing.Iterable[typing.Any]:
        if self.data_type == "integer":
            if self.min_val is not None and self.max_val is not None:
                return range(self.min_val, self.max_val + 1)
        elif self.data_type == "real":
            if self.min_val is not None and self.max_val is not None:
                current = self.min_val
                while current <= self.max_val:
                    yield current
                    current += 0.01
        elif self.data_type == "boolean":
            return [True, False]
        elif self.data_type == "string":
            if self.allowed_strings is not None:  # Check if allowed_strings is defined
                return self.allowed_strings  # If defined, return those values
            else:
                return []  # Return an empty list if allowed_strings is None (unrestricted)
        return []

    def __repr__(self):
#        return f"ParameterType(name='{self.name}', data_type='{self.data_type}', min_val={self.min_val}, max_val={self.max_val}, default={self.default}, allowed_strings={self.allowed_strings})"
        return f"ParameterType(data_type='{self.data_type}', min_val={self.min_val}, max_val={self.max_val}, default={self.default}, allowed_strings={self.allowed_strings})"

In [11]:
# FIXME: the grammar and reification modules do not correctly handle order of operations for operations
# that do not have the associative property. An easy workaround is to require parentheses.
# The parser does not handle epsilon correctly either.
# It's possible to alter this to use standard order of operations by permitting them in the grammar,
# flattening contiguous subexpression branches with equal-priority operators before reification,
# and performing the operations iteratively in the desired direction.

# REMINDER for LLM tools: This grammar MUST be an epsilon-free grammar.
# That means that rules may not be terminated explicitly.
# Higher rules have higher priority.
# More than one lookahead token is permitted.
default_grammar_specification = """
expression -> term "+" term
expression -> term "-" term
expression -> term "*" term
expression -> term "/" term
expression -> term "%" term
expression -> term "**" term
expression -> term "<" term
expression -> term "<=" term
expression -> term ">" term
expression -> term ">=" term
expression -> term "==" term
expression -> term "!=" term
expression -> term "&&" term
expression -> term "||" term
expression -> term "^^" term
expression -> term
term -> factor "[" expression "]"
term -> factor
factor -> "(" expression ")"
factor -> "-" factor
factor -> "!" factor
factor -> "+" factor
factor -> identifier "(" arguments ")"
factor -> identifier
factor -> optimization
factor -> string
factor -> number
optimization -> "@" identifier "(" expression "," optimization_arguments ")"
optimization -> "@" identifier "(" expression ")"
optimization_arguments -> optimization_argument "," optimization_arguments
optimization_arguments -> optimization_argument
optimization_argument -> argument
optimization_argument -> optimization_parameter
optimization_parameter -> "@" identifier "=" expression
arguments -> argument "," arguments
arguments -> argument
argument -> identifier "=" expression
argument -> expression
"""

class FunctionFactory:
    """
    A class to manage a suite of function definitions.
    """

    def __init__(self, grammar_specification=default_grammar_specification, should_register_basic_operations=True, should_register_basic_indicators=True, should_register_basic_screeners=True, should_register_basic_portfolio_calculators=True, should_register_candlestick_detectors=True, should_register_builtin_functions=True):
        self.function_definitions: typing.Dict[str, Definition] = {}
        self.grammar = Grammar(default_grammar_specification)
        if should_register_basic_operations:
            self.register_basic_operations()
        if should_register_basic_indicators:
            self.register_basic_indicators()
        if should_register_basic_screeners:
            self.register_basic_screeners()
        if should_register_basic_portfolio_calculators:
            self.register_basic_portfolio_calculators()
        if should_register_candlestick_detectors:
            self.register_candlestick_detectors()
        if should_register_builtin_functions:
            self.register_builtin_functions()

    def register(self, function_definition):
        """
        Registers a new screener definition.

        Args:
            function_definition: The Definition to register.

        Raises:
            ValueError: If a screener with the same name is already registered.
        """
#        if function_definition.name in self.function_definitions:
#            raise ValueError(f"A screener with the name '{function_definition.name}' is already registered.")
        self.function_definitions[function_definition.name] = function_definition
        function_definition.ffactory = self
        
    def register_basic_operations(self):
        lookup_array_param = ParameterType("any")
        lookup_index_param = ParameterType("integer")
        self.register(FunctionDefinition("Lookup", {"array": lookup_array_param, "index": lookup_index_param}, calculate_lookup))

        dereference_variable_param = ParameterType("string")
        self.register(FunctionDefinition("Dereference", {"variable": dereference_variable_param}, calculate_dereference))
        
        add_a0_param = ParameterType("any")
        add_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Add", {"a0": add_a0_param, "a1": add_a1_param}, calculate_add))

        sub_a0_param = ParameterType("any")
        sub_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Sub", {"a0": sub_a0_param, "a1": sub_a1_param}, calculate_sub))

        mul_a0_param = ParameterType("any")
        mul_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Mul", {"a0": mul_a0_param, "a1": mul_a1_param}, calculate_mul))

        div_a0_param = ParameterType("any")
        div_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Div", {"a0": div_a0_param, "a1": div_a1_param}, calculate_div))

        mod_a0_param = ParameterType("any")
        mod_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Mod", {"a0": mod_a0_param, "a1": mod_a1_param}, calculate_mod))

        pow_a0_param = ParameterType("any")
        pow_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Pow", {"a0": pow_a0_param, "a1": pow_a1_param}, calculate_pow))

        lt_a0_param = ParameterType("any")
        lt_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Lt", {"a0": lt_a0_param, "a1": lt_a1_param}, calculate_lt))

        le_a0_param = ParameterType("any")
        le_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Le", {"a0": le_a0_param, "a1": le_a1_param}, calculate_le))

        gt_a0_param = ParameterType("any")
        gt_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Gt", {"a0": gt_a0_param, "a1": gt_a1_param}, calculate_gt))

        ge_a0_param = ParameterType("any")
        ge_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Ge", {"a0": ge_a0_param, "a1": ge_a1_param}, calculate_ge))

        eq_a0_param = ParameterType("any")
        eq_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Eq", {"a0": eq_a0_param, "a1": eq_a1_param}, calculate_eq))

        ne_a0_param = ParameterType("any")
        ne_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Ne", {"a0": ge_a0_param, "a1": ne_a1_param}, calculate_ne))

        and_a0_param = ParameterType("any")
        and_a1_param = ParameterType("any")
        self.register(FunctionDefinition("And", {"a0": and_a0_param, "a1": and_a1_param}, calculate_and))

        or_a0_param = ParameterType("any")
        or_a1_param = ParameterType("any")
        self.register(FunctionDefinition("Or", {"a0": or_a0_param, "a1": or_a1_param}, calculate_or))

    def register_builtin_functions(self):
        tod_a0_param = ParameterType("any")
        self.register(FunctionDefinition("TimeOfDay", {"a0": or_a0_param}, calculate_time_of_day))
        
        # add more builtin functions here

    def register_candlestick_detectors(factory):
        """Registers candlestick detection functions with the given factory."""
    
        #Helper function to reduce redundancy.  Handles single and multiple parameters
        def register_candlestick(name, params, func):
            calculate_func = functools.partial(calculate_indicator_by, field="symbol", indicator_function=func)
            factory.register(FunctionDefinition(name, params, calculate_func))
    
        # Define parameters (some may need adjustment based on function signatures)
        bodyRatioThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.5)
        wickBodyRatioParam = ParameterType("real", min_val=0.0, default=2.0)
        upperWickThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.7)
        lowerWickThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.7)
        shadowThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.0)
        gapThresholdParam = ParameterType("real", min_val=0.0, default=0.01)
        bodyThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        closeDistanceThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.1)
        openAboveHighThresholdParam = ParameterType("real", min_val=0.0, default=1.0)
        closeBelowMidpointThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.5)
        nParam = ParameterType("integer", min_val=1, default=3) #For patterns needing 'n' candles
        highDiffThresholdParam = ParameterType("real", default=0.0) #For patterns needing high diff
        dojiThresholdParam = ParameterType("real", min_val=0.0, default=0.005)
        insideDayThresholdParam = ParameterType("real", min_val=0.0, default=0.01)
        confirmationThresholdParam = ParameterType("integer", min_val=1, default=3)
        bodyMiddleThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        wickLengthThresholdParam = ParameterType("real", min_val=0.0, default=2.0)
        openHighRatioThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.9)
        closeLowRatioThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.9)
        lowerBodyThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        upperShadowThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        tallCandleThresholdParam = ParameterType("real", min_val=0.0, default=0.03)
        closePriceDiffThresholdParam = ParameterType("real", min_val=0.0, default=0.01)
        bodyRangeRatioThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.6)
        openingPriceSimilarityThresholdParam = ParameterType("real", min_val=0.0, default=0.02)
        tallCandleBodyFactorParam = ParameterType("real", default=2.0)
        bodySizeRatioThresholdParam = ParameterType("real", min_val=0.0, default=0.5)
        openingPriceDiffThresholdParam = ParameterType("real", min_val=0.0, default=0.01)
        minBodySizeRatioParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.7)
        maxUpperShadowRatioParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.0)
        minLowerShadowRatioParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.001)
        bodyOverlapThresholdParam = ParameterType("real", min_val=0.0, default=0.0)
        marubozuBodyThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.98)
        shadowGrowthThresholdParam = ParameterType("real", min_val=0.0, default=0.1)
        openWithinBodyThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.1)
        hammerBodyFactorParam = ParameterType("real", default=0.2)
        hammerWickFactorParam = ParameterType("real", default=2.0)
        highDiffThresholdParam = ParameterType("real", default=0.0)
        downtrendLengthParam = ParameterType("integer", min_val=1, default=3)
        recentCandleCountParam = ParameterType("integer", min_val=1, default=10)
        bodyHeightFactorParam = ParameterType("real", default=3.0)
        lookbackPeriodParam = ParameterType("integer", min_val=1, default=2)
        tallCandleFactorParam = ParameterType("real", default=2.0)
        smallCandleFactorParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        lowDiffThresholdParam = ParameterType("real", min_val=0.0, default=0.001)
        engulfingThresholdParam = ParameterType("real", min_val=0.0, default=0.0)
        upperShadowRatioThresholdParam = ParameterType("real", default=2.0)
        lowerShadowRatioThresholdParam = ParameterType("real", default=0.2)
        bodyMinRatioParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.1)
        shadowMaxRatioParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.5)
        closeToHighThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.01)
        lowerShadowThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.01)
        bodySizeThresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.2)
        wick_ratio_thresholdParam = ParameterType("real", default=2.0)
        close_match_thresholdParam = ParameterType("real", min_val=0.0, max_val=1.0, default=0.01)
        range_thresholdParam = ParameterType("real", min_val=0.0, default=0.01)
    
    
        # Register all candlestick detection functions
        candlestick_functions = [
            ("DetectAboveTheStomach", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_above_the_stomach),
            ("DetectAdvanceBlock", {"openWithinBodyThreshold": openWithinBodyThresholdParam, "shadowGrowthThreshold": shadowGrowthThresholdParam}, do_detect_advance_block),
            ("DetectBearishAbandonedBaby", {"gapThreshold": gapThresholdParam}, do_detect_bearish_abandoned_baby),
            ("DetectBearishBeltHold", {"openHighRatioThreshold": openHighRatioThresholdParam, "closeLowRatioThreshold": closeLowRatioThresholdParam}, do_detect_bearish_belt_hold),
            ("DetectBearishBreakaway", {"gapThreshold": gapThresholdParam, "tallCandleThreshold": tallCandleThresholdParam}, do_detect_bearish_breakaway),
            ("DetectBearishDojiStar", {"bodyRatioThreshold": bodyRatioThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_bearish_doji_star),
            ("DetectBearishEngulfing", {"bodyOverlapThreshold": bodyOverlapThresholdParam}, do_detect_bearish_engulfing),
            ("DetectBearishHarami", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_bearish_harami),
            ("DetectBearishHaramiCross", {"dojiThreshold": dojiThresholdParam}, do_detect_bearish_harami_cross),
            ("DetectBearishHikkake", {"insideDayThreshold": insideDayThresholdParam, "confirmationThreshold": confirmationThresholdParam}, do_detect_bearish_hikkake),
            ("DetectBearishKicking", {"gapThreshold": gapThresholdParam}, do_detect_bearish_kicking),
            ("DetectBearishMeetingLines", {"closePriceDiffThreshold": closePriceDiffThresholdParam, "bodyRangeRatioThreshold": bodyRangeRatioThresholdParam}, do_detect_bearish_meeting_lines),
            ("DetectBearishSeparatingLines", {"openingPriceSimilarityThreshold": openingPriceSimilarityThresholdParam, "tallCandleBodyFactor": tallCandleBodyFactorParam}, do_detect_bearish_separating_lines),
            ("DetectBearishSideBySideWhiteLines", {"bodySizeRatioThreshold": bodySizeRatioThresholdParam, "openingPriceDiffThreshold": openingPriceDiffThresholdParam}, do_detect_bearish_side_by_side_white_lines),
            ("DetectBearishThreeLineStrike", {"bodyThreshold": bodyThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_bearish_three_line_strike),
            ("DetectBearishTriStar", {"dojiTolerance": dojiThresholdParam}, do_detect_bearish_tri_star),
            ("DetectBelowTheStomach", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_below_the_stomach),
            ("DetectBlackCandle", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_black_candle),
            ("DetectBlackMarubozu", {"shadowThreshold": shadowThresholdParam}, do_detect_black_marubozu),
            ("DetectBlackSpinningTop", {"bodyRatioThreshold": bodyRatioThresholdParam, "shadowBodyRatio": wickBodyRatioParam}, do_detect_black_spinning_top),
            ("DetectBullishAbandonedBaby", {"gapThreshold": gapThresholdParam, "dojiThreshold": dojiThresholdParam}, do_detect_bullish_abandoned_baby),
            ("DetectBullishBeltHold", {"lowerShadowThreshold": lowerShadowThresholdParam, "closeToHighThreshold": closeToHighThresholdParam}, do_detect_bullish_belt_hold),
            ("DetectBullishBreakaway", {"gapThreshold": gapThresholdParam, "bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_bullish_breakaway),
            ("DetectBullishDojiStar", {"dojiBodyThreshold": bodyRatioThresholdParam, "shadowThreshold": upperWickThresholdParam}, do_detect_bullish_doji_star),
            ("DetectBullishEngulfing", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_bullish_engulfing),
            ("DetectBullishHarami", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_bullish_harami),
            ("DetectBullishHaramiCross", {"dojiThreshold": dojiThresholdParam}, do_detect_bullish_harami_cross),
            ("DetectBullishHikkake", {"insideDayThreshold": insideDayThresholdParam, "confirmationThreshold": confirmationThresholdParam}, do_detect_bullish_hikkake),
            ("DetectBullishKicking", {"marubozuBodyThreshold": marubozuBodyThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_bullish_kicking),
            ("DetectBullishMeetingLines", {"closeDiffThreshold": closePriceDiffThresholdParam, "bodySizeThreshold": bodyThresholdParam}, do_detect_bullish_meeting_lines),
            ("DetectBullishSeparatingLines", {"blackCandleThreshold": bodyThresholdParam, "whiteCandleThreshold": bodyThresholdParam, "openingPriceDifferenceThreshold": openingPriceDiffThresholdParam}, do_detect_bullish_separating_lines),
            ("DetectBullishSideBySideWhiteLines", {"bodySimilarityThreshold": bodySizeRatioThresholdParam, "openPriceDiffThreshold": openingPriceDiffThresholdParam}, do_detect_bullish_side_by_side_white_lines),
            ("DetectBullishThreeLineStrike", {"bodyThreshold": bodyThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_bullish_three_line_strike),
            ("DetectBullishTriStar", {"dojiBodyThreshold": bodyRatioThresholdParam}, do_detect_bullish_tri_star),
            ("DetectClosingBlackMarubozu", {"upperShadowThreshold": upperWickThresholdParam}, do_detect_closing_black_marubozu),
            ("DetectClosingWhiteMarubozu", {"minBodySizeRatio": minBodySizeRatioParam, "maxUpperShadowRatio": maxUpperShadowRatioParam, "minLowerShadowRatio": minLowerShadowRatioParam}, do_detect_closing_white_marubozu),
            ("DetectCollapsingDojiStar", {"dojiThreshold": dojiThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_collapsing_doji_star),
            ("DetectConcealingBabySwallow", {"gapThreshold": gapThresholdParam, "bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_concealing_baby_swallow),
            ("DetectDeliberation", {"tallBodyThreshold": bodyThresholdParam, "smallBodyThreshold": bodyThresholdParam, "openCloseDiffThreshold": dojiThresholdParam}, do_detect_deliberation),
            ("DetectDownsideGapThreeMethods", {"gapThreshold": gapThresholdParam, "bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_downside_gap_three_methods),
            ("DetectDownsideTasukiGap", {"gapThreshold": gapThresholdParam}, do_detect_downside_tasuki_gap),
            ("DetectDragonflyDoji", {"lowerWickThreshold": lowerWickThresholdParam}, do_detect_dragonfly_doji),
            ("DetectEveningDojiStar", {"dojiThreshold": dojiThresholdParam, "tallCandleThreshold": tallCandleThresholdParam}, do_detect_evening_doji_star),
            ("DetectEveningStar", {"bodyThreshold": bodyThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_evening_star),
            ("DetectEventPatterns", {"earningsSurpriseThreshold": bodyThresholdParam, "stockSplitThreshold": bodyThresholdParam, "ratingUpgradeThreshold": bodyThresholdParam, "ratingDowngradeThreshold": bodyThresholdParam}, do_detect_event_patterns),
            ("DetectFallingThreeMethods", {"bodyRatioThreshold": bodyThresholdParam, "rangeThreshold": range_thresholdParam}, do_detect_falling_three_methods),
            ("DetectFallingWindow", {}, do_detect_falling_window),
            ("DetectGappingDownDoji", {"dojiThreshold": dojiThresholdParam}, do_detect_gapping_down_doji),
            ("DetectGappingUpDoji", {"dojiThreshold": dojiThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_gapping_up_doji),
            ("DetectGravestoneDoji", {"upperWickThreshold": upperWickThresholdParam}, do_detect_gravestone_doji),
            ("DetectHammer", {"hammerBodyFactor": hammerBodyFactorParam, "hammerWickFactor": hammerWickFactorParam}, do_detect_hammer),
            ("DetectHangingMan", {"bodyRatioThreshold": bodyRatioThresholdParam, "wickBodyRatio": wickBodyRatioParam}, do_detect_hanging_man),
            ("DetectHighWave", {"bodyRatioThreshold": bodyRatioThresholdParam, "wickRatioThreshold": wick_ratio_thresholdParam}, do_detect_high_wave),
            ("DetectHomingPigeon", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_homing_pigeon),
            ("DetectIdenticalThreeCrows", {"bodyThreshold": bodyThresholdParam}, do_detect_identical_three_crows),
            ("DetectLadderBottom", {"lowerBodyThreshold": lowerBodyThresholdParam, "upperShadowThreshold": upperShadowThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_ladder_bottom),
            ("DetectLastEngulfingBottom", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_last_engulfing_bottom),
            ("DetectLastEngulfingTop", {"bodyThreshold": bodyThresholdParam}, do_detect_last_engulfing_top),
            ("DetectLongBlackDay", {"recentCandleCount": recentCandleCountParam, "bodyHeightFactor": bodyHeightFactorParam}, do_detect_long_black_day),
            ("DetectLongLeggedDoji", {"wickBodyRatio": wickBodyRatioParam}, do_detect_long_legged_doji),
            ("DetectLongWhiteDay", {"bodyHeightFactor": bodyHeightFactorParam, "lookbackPeriod": lookbackPeriodParam}, do_detect_long_white_day),
            ("DetectMatHold", {"tallCandleFactor": tallCandleFactorParam, "smallCandleFactor": smallCandleFactorParam}, do_detect_mat_hold),
            ("DetectMatchingLow", {"lowDiffThreshold": lowDiffThresholdParam}, do_detect_matching_low),
            ("DetectMorningDojiStar", {"dojiBodyThreshold": bodyRatioThresholdParam, "gapThreshold": gapThresholdParam, "tallCandleBodyFactor": tallCandleBodyFactorParam}, do_detect_morning_doji_star),
            ("DetectMorningStar", {"bodyThreshold": bodyThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_morning_star),
            ("DetectNNewPriceLinesFalling", {"n": nParam, "highDiffThreshold": highDiffThresholdParam}, do_detect_n_new_price_lines_falling),
            ("DetectNNewPriceLinesRising", {"n": nParam, "highThreshold": highDiffThresholdParam}, do_detect_n_new_price_lines_rising),
            ("DetectNorthernDoji", {"dojiThreshold": dojiThresholdParam, "trendLength": nParam}, do_detect_northern_doji),
            ("DetectOnNeck", {"bodyRatioThreshold": bodyRatioThresholdParam, "closeMatchThreshold": close_match_thresholdParam}, do_detect_on_neck),
            ("DetectOpeningBlackMarubozu", {"lowerShadowThreshold": lowerWickThresholdParam}, do_detect_opening_black_marubozu),
            ("DetectOpeningWhiteMarubozu", {"upperShadowThreshold": upperWickThresholdParam, "bodySizeThreshold": bodySizeThresholdParam}, do_detect_opening_white_marubozu),
            ("DetectPiercingPattern", {"midpointThreshold": bodyRatioThresholdParam}, do_detect_piercing_pattern),
            ("DetectRisingThreeMethods", {"tallCandleFactor": tallCandleFactorParam, "smallCandleFactor": smallCandleFactorParam}, do_detect_rising_three_methods),
            ("DetectRisingWindow", {}, do_detect_rising_window),
            ("DetectShootingStar", {"bodyRatioThreshold": bodyRatioThresholdParam, "upperShadowRatioThreshold": upperShadowRatioThresholdParam, "lowerShadowRatioThreshold": lowerShadowRatioThresholdParam}, do_detect_shooting_star),
            ("DetectShootingStarTwoLines", {"upperShadowThreshold": upperWickThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_shooting_star_two_lines),
            ("DetectShortBlackCandle", {"bodySizeThreshold": bodySizeThresholdParam, "shadowLengthThreshold": shadowThresholdParam}, do_detect_short_black_candle),
            ("DetectShortWhiteCandle", {"bodyThreshold": bodyThresholdParam, "shadowThreshold": shadowThresholdParam}, do_detect_short_white_candle),
            ("DetectSouthernDoji", {"dojiThreshold": dojiThresholdParam, "downtrendLength": downtrendLengthParam}, do_detect_southern_doji),
            ("DetectStickSandwich", {"bodyRatioThreshold": bodyRatioThresholdParam, "closeDiffThreshold": closePriceDiffThresholdParam}, do_detect_stick_sandwich),
            ("DetectThreeBlackCrows", {"bodyThreshold": bodyThresholdParam}, do_detect_three_black_crows),
            ("DetectThreeInsideDown", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_three_inside_down),
            ("DetectThreeInsideUp", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_three_inside_up),
            ("DetectThreeOutsideDown", {"engulfingThreshold": engulfingThresholdParam}, do_detect_three_outside_down),
            ("DetectThreeOutsideUp", {"bodyRatioThreshold": bodyRatioThresholdParam}, do_detect_three_outside_up),
            ("DetectThreeStarsInTheSouth", {"bodyRatioThreshold": bodyRatioThresholdParam, "lowerWickThreshold": lowerWickThresholdParam, "gapThreshold": gapThresholdParam}, do_detect_three_stars_in_the_south),
            ("DetectThreeWhiteSoldiers", {"bodyThreshold": bodyThresholdParam, "closeHighRatio": bodyRatioThresholdParam}, do_detect_three_white_soldiers),
            ("DetectThrusting", {"midpointThreshold": bodyRatioThresholdParam}, do_detect_thrusting),
            ("DetectTweezersBottom", {"lowDiffThreshold": lowDiffThresholdParam}, do_detect_tweezers_bottom),
            ("DetectTweezersTop", {"highTolerance": lowDiffThresholdParam}, do_detect_tweezers_top),
            ("DetectTwoBlackGapping", {"gapThreshold": gapThresholdParam}, do_detect_two_black_gapping),
            ("DetectTwoCrows", {"gapThreshold": gapThresholdParam}, do_detect_two_crows),
            ("DetectUniqueThreeRiverBottom", {"bodyRatioThreshold": bodyRatioThresholdParam, "lowerWickThreshold": lowerWickThresholdParam}, do_detect_unique_three_river_bottom),
            ("DetectUpsideGapThreeMethods", {"gapThreshold": gapThresholdParam}, do_detect_upside_gap_three_methods),
            ("DetectUpsideGapTwoCrows", {"gapThreshold": gapThresholdParam, "bodyEngulfmentThreshold": bodyRatioThresholdParam}, do_detect_upside_gap_two_crows),
            ("DetectUpsideTasukiGap", {"gapThreshold": gapThresholdParam}, do_detect_upside_tasuki_gap),
            ("DetectVolumePatterns", {"volumeIncreaseThreshold": bodyRatioThresholdParam, "priceIncreaseThreshold": bodyRatioThresholdParam}, do_detect_volume_patterns),
            ("DetectWhiteCandle", {"bodyMinRatio": bodyMinRatioParam, "shadowMaxRatio": shadowMaxRatioParam}, do_detect_white_candle),
            ("DetectWhiteMarubozu", {"bodyThreshold": bodyThresholdParam}, do_detect_white_marubozu),
            ("DetectWhiteSpinningTop", {"bodyRatioThreshold": bodyRatioThresholdParam, "wickBodyRatio": wickBodyRatioParam}, do_detect_white_spinning_top)
        ]
    
        for name, params, func in candlestick_functions:
            register_candlestick(name, params, func)

    
    def register_basic_screeners(factory):
        # Top N Screener FunctionDefinition and Registration
        top_n_field_param = ParameterType("any", default="return")  # Example allowed strings
        top_n_n_param = ParameterType("integer", min_val=1, default=5)
        factory.register(FunctionDefinition("IsTopN", {"field": top_n_field_param, "top_n": top_n_n_param}, top_n_screener_function))
        
        # Percentile Screener Definition and Registration
        percentile_field_param = ParameterType("any", default="return")
        percentile_percentile_param = ParameterType("real", min_val=0.0, max_val=1.0, default=.1)
        factory.register(FunctionDefinition("IsAbovePercentile", {"field": percentile_field_param, "percentile": percentile_percentile_param}, percentile_screener_function))

    def register_basic_portfolio_calculators(factory):
        calculate_normalize = functools.partial(apply_daily_function, f=do_normalize)
        normalize_weights = ParameterType("any", default="weights")
        factory.register(FunctionDefinition("Normalize", {"weights": normalize_weights}, calculate_normalize))
        
        calculate_abs_threshold = functools.partial(apply_daily_function, f=do_abs_threshold)
        abs_threshold_weights = ParameterType("any", default="weights")
        abs_threshold_threshold = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsThreshold", {"weights": abs_threshold_weights, "threshold": abs_threshold_threshold}, calculate_abs_threshold))
        
        calculate_abs_min_cutoff = functools.partial(apply_daily_function, f=do_abs_min_cutoff)
        abs_min_cutoff_weights = ParameterType("any", default="weights")
        abs_min_cutoff_cutoff = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsMinCutoff", {"weights": abs_min_cutoff_weights, "minimum": abs_min_cutoff_cutoff}, calculate_abs_min_cutoff))
        
        calculate_abs_max_cutoff = functools.partial(apply_daily_function, f=do_abs_max_cutoff)
        abs_max_cutoff_weights = ParameterType("any", default="weights")
        abs_max_cutoff_cutoff = ParameterType("real", default=1e-6)
        factory.register(FunctionDefinition("AbsMaxCutoff", {"weights": abs_max_cutoff_weights, "maximum": abs_max_cutoff_cutoff}, calculate_abs_max_cutoff)) 
        
    def register_basic_indicators(factory): 
        """Registers basic technical indicator functions with this factory instance."""

        # SMA
        sma_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_sma = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_sma) # Referencing global do_calculate_sma
        factory.register(FunctionDefinition("SMA", {"length": sma_length_param}, calculate_sma)) # Use factory.register

        # EMA
        ema_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_ema = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_ema) # Referencing global do_calculate_ema
        factory.register(FunctionDefinition("EMA", {"length": ema_length_param}, calculate_ema)) # Use factory.register


        # RSI
        rsi_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_rsi = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_rsi) # Referencing global do_calculate_rsi
        factory.register(FunctionDefinition("RSI", {"length": rsi_length_param}, calculate_rsi)) # Use factory.register


        # MACD
        fast_length_param = ParameterType("integer", min_val=1, max_val=100, default=12)
        slow_length_param = ParameterType("integer", min_val=1, max_val=200, default=26)
        signal_length_param = ParameterType("integer", min_val=5, max_val=50, default=9) # Corrected min_val
        calculate_macd = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_macd) # Referencing global do_calculate_macd
        factory.register(FunctionDefinition("MACD", {"fast_length": fast_length_param, "slow_length": slow_length_param, "signal_length": signal_length_param}, calculate_macd)) # Use factory.register


        # Stoch
        k_length_param = ParameterType("integer", min_val=1, max_val=100, default=14)
        d_length_param = ParameterType("integer", min_val=1, max_val=50, default=3)
        smooth_k_param = ParameterType("integer", min_val=1, max_val=20, default=1)
        calculate_stoch = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_stoch)
        
        # Register the function with the factory
        factory.register(FunctionDefinition("Stoch",{"k_length": k_length_param, "d_length": d_length_param, "smooth_k": smooth_k_param}, calculate_stoch))        

        # Bollinger Bands
        bb_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        std_dev_param = ParameterType("real", min_val=0.1, max_val=5.0, default=2.0, increment=0.1)
        calculate_bollinger_bands = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_bollinger_bands) # Referencing global do_calculate_bollinger_bands
        factory.register(FunctionDefinition("BB", {"length": bb_length_param, "std_dev": std_dev_param}, calculate_bollinger_bands)) # Use factory.register

        # Donchian Channels
        donchian_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_donchian = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_donchian)
        factory.register(FunctionDefinition("Donchian", {"length": donchian_length_param}, calculate_donchian))

        # Parabolic SAR
        sar_acceleration_param = ParameterType("real", min_val=0.01, max_val=0.1, default=0.02, increment=0.01)
        sar_maximum_param = ParameterType("real", min_val=0.1, max_val=0.3, default=0.2, increment=0.01)
        calculate_sar = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_parabolic_sar)
        factory.register(FunctionDefinition("SAR", {"acceleration": sar_acceleration_param, "maximum": sar_maximum_param}, calculate_sar))        

        # Ichimoku Cloud
        tenkan_param = ParameterType("integer", min_val=5, max_val=30, default=9)
        kijun_param = ParameterType("integer", min_val=20, max_val=60, default=26)
        senkou_param = ParameterType("integer", min_val=40, max_val=100, default=52)
        calculate_ichimoku = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_ichimoku)
        factory.register(FunctionDefinition("Ichimoku", {
            "tenkan": tenkan_param,
            "kijun": kijun_param,
            "senkou": senkou_param
        }, calculate_ichimoku))


        # Williams %R
        williams_r_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_williams_r = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_williams_r)
        factory.register(FunctionDefinition("WilliamsR", {"length": williams_r_length_param}, calculate_williams_r))


        # Rolling VWAP
        rvwap_length_param = ParameterType("integer", min_val=1, max_val=200, default=20)
        calculate_rvwap = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_rvwap) # Referencing global do_calculate_rvwap
        factory.register(FunctionDefinition("RVWAP", {"length": rvwap_length_param}, calculate_rvwap)) # Use factory.register


        # Average True Range
        atr_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_atr = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_atr) # Referencing global do_calculate_atr
        factory.register(FunctionDefinition("ATR", {"length": atr_length_param}, calculate_atr)) # Use factory.register


        # ADX
        adx_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_adx = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_adx) # Referencing global do_calculate_adx
        factory.register(FunctionDefinition("ADX", {"length": adx_length_param}, calculate_adx)) # Use factory.register


        # Commodity Channel Index
        cci_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_cci = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_cci) # Referencing global do_calculate_cci
        factory.register(FunctionDefinition("CCI", {"length": cci_length_param}, calculate_cci)) # Use factory.register


        # Chaikin Money Flow
        cmf_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_cmf = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_cmf) # Referencing global do_calculate_cmf
        factory.register(FunctionDefinition("CMF", {"length": cmf_length_param}, calculate_cmf)) # Use factory.register


        # Aroon
        aroon_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_aroon = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_aroon) # Referencing global do_calculate_aroon
        factory.register(FunctionDefinition("Aroon", {"length": aroon_length_param}, calculate_aroon)) # Use factory.register


        # MFI
        mfi_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_mfi = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_mfi) # Referencing global do_calculate_mfi
        factory.register(FunctionDefinition("MFI", {"length": mfi_length_param}, calculate_mfi)) # Use factory.register


        # Percent Rank
        pct_rank_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_pct_rank = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_pct_rank) # Referencing global do_calculate_pct_rank
        factory.register(FunctionDefinition("PCT", {"length": pct_rank_length_param}, calculate_pct_rank)) # Use factory.register


        # Price Range Percentage
        prp_length_param = ParameterType("integer", min_val=1, max_val=200, default=14)
        calculate_prp = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_prp) # Referencing global do_calculate_prp
        factory.register(FunctionDefinition("PRP", {"length": prp_length_param}, calculate_prp)) # Use factory.register


        # Log Return
        lret_length_param = ParameterType("integer", min_val=1, max_val=200, default=1)
        calculate_lret = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_lret) # Referencing global do_calculate_lret
        factory.register(FunctionDefinition("LRET", {"length": lret_length_param}, calculate_lret)) # Use factory.register


        # Shift
        shift_n_param = ParameterType("integer", min_val=1, max_val=200, default=1)
        shift_series_param = ParameterType("any", default="close")
        calculate_shift = functools.partial(calculate_indicator_by, field="symbol", indicator_function=do_calculate_shift) # Referencing global do_calculate_shift
        factory.register(FunctionDefinition("Shift", {"series": shift_series_param, "n": shift_n_param}, calculate_shift)) # Use factory.register    

    
    def get(self, name: str):
        """
        Retrieves a function definition by name.

        Args:
            name: The name of the screener.

        Returns:
            The FunctionDefinition object.

        Raises:
            ValueError: If no screener with the given name is registered.
        """
        if name not in self.function_definitions:
            raise ValueError(f"No function found with the name '{name}'.")
        return self.function_definitions[name]

    def parse(self, expression):
        parse_tree = self.grammar.parse(expression)
        reified_expression = parse_tree.reify(self)
        if isinstance(reified_expression, ParseTreeNode):
            reified_expression.parse_text = expression
        return reified_expression

    def __repr__(self):
        return f"FunctionFactory(functions={self.function_definitions})"

In [12]:
# Functions to carry out basic mathematical operations

# FIXME: these all return a Series and are somewhat inefficient

def get_header_name(item):
    """
    Checks if an item is a pandas Series or a single-column DataFrame.
    If it's a Series, returns its name.
    If it's a single-column DataFrame, returns the name of the column.
    If it's not a pandas Series or a single-column DataFrame but a scalar,
    returns its value (quoted if it's a string).
    Otherwise, returns None.
    """
    if isinstance(item, pd.Series):
        return str(item.name)
    elif isinstance(item, pd.DataFrame):
        return str(item.columns[0])
    elif pd.api.types.is_scalar(item):
        if isinstance(item, str):
            return f"'{item}'"
        else:
            return str(item)
    else:
        return str(None)

def xx(a):
    if(isinstance(a, pd.DataFrame)):
        return a.iloc[:, 0]
    else:
        return a


def calculate_ge(df, a0, a1):
    rv = xx(a0) >= xx(a1)
    name = "(" + get_header_name(a0) + ">=" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

# this only allows integer field references at the moment
def calculate_lookup(df, array, index):
    return array.iloc[:, index]

def calculate_dereference(df, variable):
    return df[variable]

#expression -> term "+" term
def calculate_add(df, a0, a1):
    rv = xx(a0) + xx(a1)
    name = "(" + get_header_name(a0) + "+" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "-" term
def calculate_sub(df, a0, a1):
    rv = xx(a0) - xx(a1)
    name = "(" + get_header_name(a0) + "-" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "*" term
def calculate_mul(df, a0, a1):
    rv = xx(a0) * xx(a1)
    name = "(" + get_header_name(a0) + "*" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "/" term
def calculate_div(df, a0, a1):
    rv = xx(a0) / xx(a1)
    name = "(" + get_header_name(a0) + "/" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "%" term
def calculate_mod(df, a0, a1):
    rv = xx(a0) % xx(a1)
    name = "(" + get_header_name(a0) + "%" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "**" term
def calculate_pow(df, a0, a1):
    rv = xx(a0) ** xx(a1)
    name = "(" + get_header_name(a0) + "**" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "<" term
def calculate_lt(df, a0, a1):
    rv = xx(a0) < xx(a1)
    name = "(" + get_header_name(a0) + "<" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "<=" term
def calculate_le(df, a0, a1):
    rv = xx(a0) <= xx(a1)
    name = "(" + get_header_name(a0) + "<=" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term ">" term
def calculate_gt(df, a0, a1):
    rv = xx(a0) > xx(a1)
    name = "(" + get_header_name(a0) + ">" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term ">=" term
def calculate_ge(df, a0, a1):
    rv = xx(a0) >= xx(a1)
    name = "(" + get_header_name(a0) + ">=" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "==" term
def calculate_eq(df, a0, a1):
    rv = xx(a0) == xx(a1)
    name = "(" + get_header_name(a0) + "==" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "!=" term
def calculate_ne(df, a0, a1):
    rv = xx(a0) != xx(a1)
    name = "(" + get_header_name(a0) + "!=" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv


#expression -> term "&&" term
def calculate_and(df, a0, a1):
    rv = (xx(a0) * xx(a1)) != 0
    name = "(" + get_header_name(a0) + "&&" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv

#expression -> term "||" term
def calculate_or(df, a0, a1):
    rv = ((xx(a0) != 0) + (xx(a1) != 0)) != 0
    name = "(" + get_header_name(a0) + "||" + get_header_name(a1) + ")"
    if isinstance(rv, (pd.DataFrame)):
        rv.rename(columns={rv.columns[0]: name}, inplace=True)
        return rv.iloc[:, 0]
    elif isinstance(rv, pd.Series):
        rv.name = name
        return rv
    return rv


#expression -> term "^^" term

In [13]:
def calculate_time_of_day(df, a0):
    
    name = "TimeOfDay(" + get_header_name(a0) + ")"
    if(isinstance(a0, pd.DataFrame)):
        a0 = a0.iloc[:, 0]
    if(isinstance(a0, pd.Series)):
        rv = a0.astype("datetime").dt.strftime("%H%M%S").astype(int)
        rv.name = name
        return rv
    return pd.to_datetime(a0).dt.strftime("%H%M%S").astype(int)


In [14]:
# Functions for basic portfolio calculations

def do_normalize(context, weights):
    """Normalizes a Series of weights (including negative) to sum to 1 (absolute values)."""
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()
    signs = wt.div(absolute_weights).fillna(0)
    total_absolute_weight = absolute_weights.sum()

    if total_absolute_weight == 0:
        return pd.DataFrame({"weight": [0.0] * len(weights)}, index=weights.index)

    normalized_absolute_weights = absolute_weights / total_absolute_weight
    normalized_weights = normalized_absolute_weights * signs

    return pd.DataFrame({"weight": normalized_weights}, index=weights.index)


def do_abs_threshold(context, weights, threshold):
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply threshold
    wt[absolute_weights < threshold] = 0

    return pd.DataFrame({"weight": wt}, index=weights.index)

def do_abs_min_cutoff(context, weights, minimum):
    """Applies a minimum weight to a list of weights."""
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply minimum
    mask = (absolute_weights > 0) & (absolute_weights < minimum)
    wt[mask] = np.sign(wt[mask]) * minimum

    return pd.DataFrame({"weight": wt}, index=weights.index)

def do_abs_max_cutoff(context, weights, maximum):
    if isinstance(weights, pd.DataFrame):
        weights = weights.iloc[:, 0]
    elif not isinstance(weights, pd.Series):
        weights = pd.Series(weights)

    wt = weights.astype("float")
    absolute_weights = wt.abs()

    # Apply maximum
    mask = absolute_weights > maximum
    wt[mask] = np.sign(wt[mask]) * maximum

    return pd.DataFrame({"weight": wt}, index=weights.index)

In [15]:
# Functions to carry out screener operations
def top_n_screener_function(context, field, top_n):
    """
    Produces a boolean mask (pd.Series) for the top n readings per day.

    Args:
        df: Pandas DataFrame with columns for date and reading.
        date_col: Name of the column containing the date. Should be datetime or convertible.
        reading_col: Name of the column containing the reading.

    Returns:
        A pandas Series (boolean mask) with True for rows corresponding to the 
        top 5 readings for each day, and False otherwise. Returns
        an empty Series if the input DataFrame is empty.
    """
    if(isinstance(field, str)):
        v = context.groupby("date")[field].rank(ascending=False, method='first')
    else:
        v = field.groupby(by=context["date"]).rank(ascending=False, method="first")
    mask = v <= top_n 
    rv = mask.iloc[:, 0]
    fn = get_header_name(field)
    header = f"IsTopN({get_header_name(top_n)},{fn})"
    rv.name = header
    return rv


def percentile_screener_function(context, field, percentile):
    if(isinstance(field, str)):
        v = context.groupby(by="date")[field].rank(ascending=False, method='first', pct=True)
    else:
        v = field.groupby(by=context["date"]).rank(ascending=False, method="first", pct=True)
    mask = v >= percentile
    
    rv = mask.iloc[:, 0]
    header = f"IsAbovePercentile({get_header_name(top_n)},{fn})"
    rv.name = header
    return rv
    

In [16]:
# Utility function for applying generic indicator functions to dataframes with multiple symbols
def calculate_indicator_by(df, field, indicator_function, *args, **kwargs):
    """
    Calculates an indicator by a specified field within a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        field (str): The field to group by (e.g., 'symbol', 'date').
        indicator_function (callable): The indicator function to apply.
        *args: Positional arguments to pass to the indicator function.
        **kwargs: Keyword arguments to pass to the indicator function.

    Returns:
        pd.DataFrame: The DataFrame with the calculated indicator(s) added.
    """
    if field in df.columns:
        result_dfs = []
        for group_value, group_df in df.groupby(field):
            result_dfs.append(indicator_function(group_df.copy(), *args, **kwargs).assign(**{field: group_value}))
        rv = pd.concat(result_dfs)
        return rv.drop(field, axis=1)
    else:
        return indicator_function(df.copy(), *args, **kwargs)


# Note: I am not completely happy with how this works; ideally, this would be merged with calculate_indicator_by
def apply_daily_function(context, weights, f, *args, **kwargs):
    if(isinstance(weights, str)):
        foo = context.groupby("date")[weights].apply(lambda a: f(context, a, *args, **kwargs))
    else:
        foo = weights.groupby(by=context["date"]).apply(lambda a: f(context, a, *args, **kwargs))
    # Swap levels and drop the 'date' level
    foo.index = foo.index.droplevel("date")    
    return foo.iloc[:, 0]

In [17]:
# Functions to calculate basic technical indicators

# SMA
def do_calculate_sma(df: pd.DataFrame, length: int) -> pd.DataFrame:
    sma_values = df['close'].rolling(window=length).mean().values
    return pd.DataFrame({f"SMA({length})": sma_values}, index=df.index)

# EMA
def do_calculate_ema(df: pd.DataFrame, length: int) -> pd.DataFrame:
    ema_values = df['close'].ewm(span=length, adjust=False).mean().values
    return pd.DataFrame({f"EMA({length})": ema_values}, index=df.index)

# RSI
def do_calculate_rsi(df: pd.DataFrame, length: int) -> pd.DataFrame:
    length = int(length)
    delta = df['close'].diff()
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    avg_gains = gains.rolling(window=length).mean()
    avg_losses = losses.rolling(window=length).mean()
    rs = avg_gains / avg_losses.replace(0, float('inf'))
    rsi = 100 - (100 / (1 + rs))
    rsi_values = rsi.values
    return pd.DataFrame({f"RSI({length})": rsi_values}, index=df.index)

# MACD
def do_calculate_macd(df: pd.DataFrame, fast_length: int, slow_length: int, signal_length: int) -> pd.DataFrame:
    ema_fast = df['close'].ewm(span=fast_length, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow_length, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_length, adjust=False).mean()
    histogram = macd - signal
    return pd.DataFrame({f'MACD({fast_length},{slow_length},{signal_length})["macd"]': macd.values, f'MACD({fast_length},{slow_length},{signal_length})["signal"]': signal.values, f'MACD({fast_length},{slow_length},{signal_length})["histogram"]': histogram.values}, index=df.index)

# Stoch
def do_calculate_stoch(df: pd.DataFrame, k_length: int, d_length: int, smooth_k: int = 1) -> pd.DataFrame:
    """
    Calculate Stochastic Oscillator (Stoch)
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'high', 'low', 'close' columns
    k_length (int): Length for %K line (typically 14)
    d_length (int): Length for %D line (typically 3)
    smooth_k (int): Smoothing factor for %K line (typically 1)
    
    Returns:
    pd.DataFrame: DataFrame with %K and %D columns
    """
    k_length = int(k_length)
    d_length = int(d_length)
    smooth_k = int(smooth_k)
    
    # Calculate %K
    low_min = df['low'].rolling(window=k_length).min()
    high_max = df['high'].rolling(window=k_length).max()
    
    k = 100 * ((df['close'] - low_min) / (high_max - low_min))
    k = k.rolling(window=smooth_k).mean()  # Smoothed %K
    
    # Calculate %D (simple moving average of %K)
    d = k.rolling(window=d_length).mean()
    
    return pd.DataFrame({
        f"Stoch({k_length},{d_length},{smooth_k})[\"%K\"]": k.values,
        f"Stoch({k_length},{d_length},{smooth_k})[\"%D\"]": d.values
    }, index=df.index)

# Bollinger Bands
def do_calculate_bollinger_bands(df: pd.DataFrame, length: int, std_dev: float) -> pd.DataFrame:
    rolling_mean = df['close'].rolling(window=length).mean()
    rolling_std = df['close'].rolling(window=length).std()
    upper_band = rolling_mean + (rolling_std * std_dev)
    lower_band = rolling_mean - (rolling_std * std_dev)
    middle_values = rolling_mean.values
    upper_values = upper_band.values
    lower_values = lower_band.values
    bb_df = pd.DataFrame({f'BB({length},{std_dev})["middle"]': middle_values, f'BB({length},{std_dev})["upper"]': upper_values, f'BB({length},{std_dev})["lower"]': lower_values}, index=df.index)
    return bb_df

def do_calculate_donchian(df: pd.DataFrame, length: int) -> pd.DataFrame:
    """
    Calculate Donchian Channels
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'high', 'low', 'close' columns
    length (int): Lookback period for the channels (typically 20)
    
    Returns:
    pd.DataFrame: DataFrame with upper, middle, and lower Donchian channels
    """
    length = int(length)
    
    # Calculate upper band (highest high of last N periods)
    upper = df['high'].rolling(window=length).max()
    
    # Calculate lower band (lowest low of last N periods)
    lower = df['low'].rolling(window=length).min()
    
    # Middle band is average of upper and lower
    middle = (upper + lower) / 2
    
    return pd.DataFrame({
        f"Donchian({length})[\"upper\"]": upper.values,
        f"Donchian({length})[\"middle\"]": middle.values,
        f"Donchian({length})[\"lower\"]": lower.values
    }, index=df.index)


def do_calculate_parabolic_sar(df: pd.DataFrame, acceleration: float = 0.02, maximum: float = 0.2) -> pd.DataFrame:
    """
    Calculate Parabolic SAR (Stop and Reverse) indicator
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'high', 'low', 'close' columns
    acceleration (float): Acceleration factor (typically starts at 0.02)
    maximum (float): Maximum acceleration factor (typically 0.2)
    
    Returns:
    pd.DataFrame: DataFrame with Parabolic SAR values
    """
    high = df['high'].values
    low = df['low'].values
    
    sar = np.zeros(len(df))
    ep = np.zeros(len(df))
    af = np.zeros(len(df))
    
    # Initial values
    sar[0] = low[0] if high[0] + low[0] <= high[1] + low[1] else high[0]
    ep[0] = high[0] if sar[0] == low[0] else low[0]
    af[0] = acceleration
    
    # Determine initial trend
    trend = 1 if sar[0] == low[0] else -1
    
    for i in range(1, len(df)):
        # Update SAR
        sar[i] = sar[i-1] + af[i-1] * (ep[i-1] - sar[i-1])
        
        # Check for SAR reversal
        if trend == 1:  # Uptrend
            if low[i] < sar[i]:
                trend = -1
                sar[i] = max(high[i-1], high[i])
                ep[i] = low[i]
                af[i] = acceleration
            else:
                trend = 1
                if high[i] > ep[i-1]:
                    ep[i] = high[i]
                    af[i] = min(af[i-1] + acceleration, maximum)
                else:
                    ep[i] = ep[i-1]
                    af[i] = af[i-1]
                # Check if SAR is above previous two lows
                if i >= 2:
                    sar[i] = min(sar[i], low[i-1], low[i-2])
                elif i >= 1:
                    sar[i] = min(sar[i], low[i-1])
        else:  # Downtrend
            if high[i] > sar[i]:
                trend = 1
                sar[i] = min(low[i-1], low[i])
                ep[i] = high[i]
                af[i] = acceleration
            else:
                trend = -1
                if low[i] < ep[i-1]:
                    ep[i] = low[i]
                    af[i] = min(af[i-1] + acceleration, maximum)
                else:
                    ep[i] = ep[i-1]
                    af[i] = af[i-1]
                # Check if SAR is below previous two highs
                if i >= 2:
                    sar[i] = max(sar[i], high[i-1], high[i-2])
                elif i >= 1:
                    sar[i] = max(sar[i], high[i-1])
    
    return pd.DataFrame({
        f"SAR({acceleration},{maximum})": sar
    }, index=df.index)

def do_calculate_ichimoku(df: pd.DataFrame, tenkan: int = 9, kijun: int = 26, senkou: int = 52) -> pd.DataFrame:
    """
    Calculate Ichimoku Cloud components
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'high', 'low', 'close' columns
    tenkan (int): Conversion line period (typically 9)
    kijun (int): Base line period (typically 26)
    senkou (int): Leading span period (typically 52)
    
    Returns:
    pd.DataFrame: DataFrame with all Ichimoku components
    """
    high = df['high']
    low = df['low']
    
    # Tenkan-sen (Conversion Line)
    tenkan_high = high.rolling(window=tenkan).max()
    tenkan_low = low.rolling(window=tenkan).min()
    tenkan_sen = (tenkan_high + tenkan_low) / 2
    
    # Kijun-sen (Base Line)
    kijun_high = high.rolling(window=kijun).max()
    kijun_low = low.rolling(window=kijun).min()
    kijun_sen = (kijun_high + kijun_low) / 2
    
    # Senkou Span A (Leading Span A)
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(kijun)
    
    # Senkou Span B (Leading Span B)
    senkou_high = high.rolling(window=senkou).max()
    senkou_low = low.rolling(window=senkou).min()
    senkou_span_b = ((senkou_high + senkou_low) / 2).shift(kijun)
    
#    # Chikou Span (Lagging Span)
#    chikou_span = df['close'].shift(-kijun)
    
    return pd.DataFrame({
        f"Ichimoku({tenkan},{kijun},{senkou})[\"tenkan\"]": tenkan_sen,
        f"Ichimoku({tenkan},{kijun},{senkou})[\"kijun\"]": kijun_sen,
        f"Ichimoku({tenkan},{kijun},{senkou})[\"senkou_a\"]": senkou_span_a,
        f"Ichimoku({tenkan},{kijun},{senkou})[\"senkou_b\"]": senkou_span_b,
#        f"Ichimoku({tenkan},{kijun},{senkou})[\"chikou\"]": chikou_span
    }, index=df.index)

# Rolling VWAP
def do_calculate_rvwap(df: pd.DataFrame, length: int) -> pd.DataFrame:
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    rolling_volume = df['volume'].rolling(length).sum()
    typical_price_x_volume = df["volume"] * typical_price
    rolling_typical_price_x_volume = typical_price_x_volume.rolling(length).sum()
    vwap = rolling_typical_price_x_volume / rolling_volume
    return pd.DataFrame({f"RVWAP({length})": vwap.values}, index=df.index)

# Average True Range
def do_calculate_atr(df, length):
    """Calculates Average True Range (ATR)."""
    tr1 = df["high"] - df["low"]
    tr2 = abs(df["high"] - df["close"].shift(1))
    tr3 = abs(df["low"] - df["close"].shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window=length).mean()
    return pd.DataFrame({f"ATR({length})": atr}, index=df.index)

# ADX
def do_calculate_adx(df, length=14, adxr_period=14):
    """Calculates ADX, +DI (pdi), -DI (mdi), DX, and ADXR with proper column naming."""
    high = df["high"]
    low = df["low"]
    close = df["close"]
    
    # 1. Calculate +DM and -DM
    upmove = high - high.shift(1)
    downmove = low.shift(1) - low
    plus_dm = pd.Series(np.where((upmove > downmove) & (upmove > 0), upmove, 0.0), index=df.index)
    minus_dm = pd.Series(np.where((downmove > upmove) & (downmove > 0), downmove, 0.0), index=df.index)
    
    # 2. Calculate True Range (TR)
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # 3. Smooth values (Wilder's EMA)
    plus_dm_smoothed = plus_dm.ewm(alpha=1/length, adjust=False).mean()
    minus_dm_smoothed = minus_dm.ewm(alpha=1/length, adjust=False).mean()
    true_range_smoothed = true_range.ewm(alpha=1/length, adjust=False).mean()
    
    # 4. Calculate +DI (pdi) and -DI (mdi)
    plus_di = 100 * (plus_dm_smoothed / true_range_smoothed)
    minus_di = 100 * (minus_dm_smoothed / true_range_smoothed)
    
    # 5. Calculate DX (Directional Movement Index)
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di).replace(0, np.inf)
    
    # 6. Calculate ADX (Average DX)
    adx = dx.ewm(alpha=1/length, adjust=False).mean()
    
    # 7. Calculate ADXR (Average of ADX and ADX from `adxr_period` ago)
    adxr = (adx + adx.shift(adxr_period)) / 2
    
    # Return DataFrame with exact column naming format
    return pd.DataFrame({
        f"ADX({length})[\"adx\"]": adx,
        f"ADX({length})[\"pdi\"]": plus_di,
        f"ADX({length})[\"mdi\"]": minus_di,
        f"ADX({length})[\"dx\"]": dx,        # Optional: Include DX if needed
        f"ADX({length})[\"adxr\"]": adxr     # Optional: Include ADXR if needed
    }, index=df.index)

def do_calculate_williams_r(df: pd.DataFrame, length: int) -> pd.DataFrame:
    """
    Calculate Williams %R indicator
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'high', 'low', 'close' columns
    length (int): Lookback period for the indicator (typically 14)
    
    Returns:
    pd.DataFrame: DataFrame with Williams %R values
    """
    length = int(length)
    
    # Calculate Williams %R
    highest_high = df['high'].rolling(window=length).max()
    lowest_low = df['low'].rolling(window=length).min()
    
    williams_r = -100 * ((highest_high - df['close']) / (highest_high - lowest_low))
    
    return pd.DataFrame({
        f"WilliamsR({length})": williams_r.values
    }, index=df.index)


# Commodity Channel Index
def do_calculate_cci(df, length):
    """Calculates Commodity Channel Index (CCI)."""
    typical_price = (df["high"] + df["low"] + df["close"]) / 3
    ma_typical_price = typical_price.rolling(window=length).mean()
    mean_deviation = pd.Series(abs(typical_price - ma_typical_price)).rolling(window=length).mean()
    cci = (typical_price - ma_typical_price) / (0.015 * mean_deviation)
    return pd.DataFrame({f"CCI({length})": cci})

# Chaikin Money Flow
def do_calculate_cmf(df, length):
    """Calculates Chaikin Money Flow (CMF)."""
    money_flow = ((df["close"] - df["low"]) - (df["high"] - df["close"])) / (df["high"] - df["low"]) * df["volume"]
    money_flow_volume = money_flow.rolling(window=length).sum()
    volume_sum = df["volume"].rolling(window=length).sum()
    cmf = money_flow_volume / volume_sum
    return pd.DataFrame({f"CMF({length})": cmf}, index=df.index)

# Aroon
def do_calculate_aroon(df, length):
    """Calculates Aroon Up and Aroon Down (incremental optimization)."""
    high = df["high"].values
    low = df["low"].values
    aroon_up = np.zeros(len(df))
    aroon_down = np.zeros(len(df))

    if len(df) < length:
        return pd.DataFrame({f"AROON({length}):up": aroon_up, f"AROON({length}):down": aroon_down}, index=df.index)

    highest_index = 0
    lowest_index = 0

    for i in range(length, len(df)):
        window_start = i - length

        # Update highest index
        if highest_index < window_start:  # If the previous highest is outside the window
            highest_index = window_start
            for j in range(window_start + 1, i):
                if high[j] > high[highest_index]:
                    highest_index = j
        elif high[i - 1] >= high[highest_index]:
            highest_index = i - 1

        # Update lowest index
        if lowest_index < window_start:  # If the previous lowest is outside the window
            lowest_index = window_start
            for j in range(window_start + 1, i):
                if low[j] < low[lowest_index]:
                    lowest_index = j
        elif low[i - 1] <= low[lowest_index]:
            lowest_index = i - 1

        aroon_up[i] = (length - (i - 1 - highest_index)) * 100.0 / length
        aroon_down[i] = (length - (i - 1 - lowest_index)) * 100.0 / length

    return pd.DataFrame({f"AROON({length}):up": aroon_up, f"AROON({length}):down": aroon_down}, index=df.index)

# MFI
def do_calculate_mfi(df, length):
    """Calculates Money Flow Index (MFI)."""
    typical_price = (df["high"] + df["low"] + df["close"]) / 3
    money_flow = typical_price * df["volume"]

    positive_money_flow = money_flow[df["close"] > df["close"].shift(1)]
    negative_money_flow = money_flow[df["close"] <= df["close"].shift(1)]

    positive_money_flow = positive_money_flow.rolling(window=length).sum()
    negative_money_flow = abs(negative_money_flow.rolling(window=length).sum())

    money_ratio = positive_money_flow / negative_money_flow
    mfi = 100 - (100 / (1 + money_ratio))
    return pd.DataFrame({f"MFI({length})": mfi}, index=df.index)

# Percent Rank
def do_calculate_pct_rank(df, length):
    """Calculates percentile rank using pandas only."""
    pct_rank = df['close'].rolling(window=length).apply(lambda x: (x < x[-1]).sum() / (len(x)-1) if len(x) > 1 else 0, raw=True)
    return pd.DataFrame({f"PCT({length})": pct_rank}, index=df.index)

# Price Range Percentage
def do_calculate_prp(df, length):
    """Calculates the Price Range Percentage."""
    high_max = df["high"].rolling(window=length).max()
    low_min = df["low"].rolling(window=length).min()
    range_width = high_max - low_min
    price_percentage = (df["close"] - low_min) / range_width * 100
    return pd.DataFrame({f"PRP({length})": price_percentage}, index=df.index)

# Log Return
def do_calculate_lret(df, length):
    """Calculates Log Return"""
    v = (df["close"] / df["close"].shift(length)).apply(lambda a: np.log(a) if a != np.nan else np.nan)
    return pd.DataFrame({f"LRET({length})": v}, index=df.index)

# Shift
def do_calculate_shift(df, series="close", n=1):
    name = "!unknown"
    if isinstance(series, str):
        name = series
        series = df[series]
    elif isinstance(series, pd.DataFrame):
        if(series.shape[1] != 1):
            raise ValueError(f"Shift() currently will operate on a single column only")
        name = series.columns[0]
        series = series.iloc[:, 0]
    elif isinstance(series, pd.Series):
        name = series.name
    return pd.DataFrame({f"Shift({name},{n})":series.shift(n)}, index=series.index)

In [18]:
# Ref: https://thepatternsite.com/8NewPriceLines.html
# This function detects the 'N new price lines (rising)' pattern in a given DataFrame.
# It checks for N consecutive candles, each with a higher high than the previous one.
# The function takes the number of consecutive candles (n) and a threshold for the minimum increase in the high (high_threshold) as parameters.
# The function returns a pandas Series of booleans, indicating whether the pattern is present at each point in the DataFrame.

def do_detect_n_new_price_lines_rising(df: pd.DataFrame, n: int, high_threshold: float = 0.0) -> pd.Series:
    """
    Detects the 'N new price lines (rising)' pattern in OHLCV data.

    Args:
        df: DataFrame with 'high' column.
        n: Number of consecutive candles to check.
        high_threshold: Minimum percentage increase in high between consecutive candles.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    highs = df["high"]
    is_rising = highs.diff() > highs.shift(1) * high_threshold
    
    pattern_detected = is_rising.rolling(window=n, min_periods=n).all()
    return pattern_detected


# Ref: https://thepatternsite.com/8NewPriceLines.html
# This function detects the 'N new price lines (falling)' pattern in a DataFrame.
# The function iterates through the DataFrame, checking if N consecutive candles have successively lower highs.
# A boolean Series is returned, indicating the presence of the pattern at each point in time.


def do_detect_n_new_price_lines_falling(df: pd.DataFrame, n: int, high_diff_threshold: float = 0.0) -> pd.Series:
    """
    Detects the 'N new price lines (falling)' candlestick pattern.

    Args:
        df: DataFrame with 'high' column.
        n: The number of consecutive candles required to form the pattern.
        high_diff_threshold: Minimum percentage difference between consecutive highs for a valid pattern.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    highs = df["high"]
    is_n_new_price_lines_falling = (highs.shift(1) - highs) / highs.shift(1) > high_diff_threshold  #FIXME: needs additional logic for the last row
    is_n_new_price_lines_falling = is_n_new_price_lines_falling.rolling(window=n, min_periods=n).all()
    
    return is_n_new_price_lines_falling


# Ref: https://thepatternsite.com/AbandonBaby.html
# This function detects the Bearish Abandoned Baby candlestick pattern.
# It identifies the pattern based on the criteria described in the provided HTML documentation.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating whether a Bearish Abandoned Baby pattern is detected for each row.  True means a pattern is detected.


def do_detect_bearish_abandoned_baby(df: pd.core.frame.DataFrame, gap_threshold: float = 0.01) -> pd.core.series.Series:
    """
    Detects the Bearish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.  Must contain columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) required between candles.

    Returns:
        pd.Series: Boolean Series indicating Bearish Abandoned Baby patterns (True for a match).
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    is_doji = abs(df['close'] - df['open']) < 0.01 * (df['high'] - df['low'])  # Assuming negligible body size for a doji

    # Shift series to check previous day's values, and deal with edge cases
    previous_high = df['high'].shift(1)
    previous_low = df['low'].shift(1)
    previous_range = (previous_high - previous_low).fillna(0)

    # Check for Bearish Abandoned Baby pattern
    pattern = (is_white.shift(2) &
               is_doji.shift(1) &
               is_black &
               (df['low'] < df['close'].shift(1)) &
               (df['high'].shift(1) > df['high']) &
               (previous_range > 0) &
               ( (df['low'].shift(1) - df['high'].shift(2)) / previous_range.shift(1) >= gap_threshold ))


    return pattern

# Ref: https://thepatternsite.com/AbandonBabyBull.html
# This function detects the Bullish Abandoned Baby candlestick pattern.
# It identifies the pattern based on the configuration described in the documentation:
# 1. Three candles in a downward trend.
# 2. The first candle is black (close < open).
# 3. The second candle is a doji (open ≈ close).
# 4. The third candle is white (close > open) and its lower shadow is above the doji's high.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_abandoned_baby(df: pd.core.frame.DataFrame, doji_threshold: float = 0.01, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: Maximum difference between open and close for a candle to be considered a doji (as a fraction of the candle's range).
        gap_threshold: Minimum gap between the doji high and adjacent candles (as a fraction of the previous candle's range).

    Returns:
        Boolean Series indicating the presence of the pattern.  Returns an empty Series if the DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    high_low_range = df['high'] - df['low']
    open_close_diff = abs(df['open'] - df['close'])
    is_doji = open_close_diff <= doji_threshold * high_low_range


    bullish_abandoned_baby = pd.Series(index=df.index, data=False)

    for i in range(2, len(df)):
        if is_black.iloc[i-2] and is_doji.iloc[i-1] and is_white.iloc[i]:
            gap1 = df['low'].iloc[i-1] - df['high'].iloc[i-2]
            gap2 = df['low'].iloc[i] - df['high'].iloc[i-1]
            if gap1 >= gap_threshold * (df['high'].iloc[i-2] - df['low'].iloc[i-2]) and gap2 >= 0 and df['low'].iloc[i] > df['high'].iloc[i-1]:
              bullish_abandoned_baby.iloc[i] = True

    return bullish_abandoned_baby


# Ref: https://thepatternsite.com/AboveStomach.html
# This function detects the "Above the Stomach" candlestick pattern.
# The pattern consists of two candles: a black candle followed by a white candle.
# The body of the white candle must be at or above the midpoint of the body of the black candle.

def do_detect_above_the_stomach(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Above the Stomach candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include columns named "open", "high", "low", "close".
        body_ratio_threshold: Minimum ratio of the second candle's body to the first candle's body for the pattern to be considered valid.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    
    body_size_1 = abs(df["close"].shift(1) - df["open"].shift(1))
    midpoint_1 = df["open"].shift(1) + body_size_1 / 2
    
    body_size_2 = abs(df["close"] - df["open"])
    
    above_stomach = (is_black.shift(1) & is_white & (df["open"] >= midpoint_1.shift(1)) & (df["close"] >= midpoint_1.shift(1))) | (is_black.shift(1) & is_white & (df["open"] >= midpoint_1.shift(1)) & (df["close"] >= midpoint_1.shift(1)) & (body_size_2 >= body_ratio_threshold * body_size_1))

    return above_stomach


# Ref: https://thepatternsite.com/AdvanceBlock.html
# The Advance Block pattern consists of three white candles in an upward trend.
# Each candle opens within the body of the previous candle. The height of the shadows grows taller on the last two candles.
# This function detects the Advance Block pattern based on the provided criteria.


def do_detect_advance_block(df: pd.DataFrame, open_within_body_threshold: float = 0.1, shadow_growth_threshold: float = 0.1) -> pd.Series:
    """
    Detects the Advance Block candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        open_within_body_threshold: Maximum percentage difference between open and previous close for a candle to be considered "opening within the body".
        shadow_growth_threshold: Minimum percentage increase in shadow height from one candle to the next.


    Returns:
        A pandas Series with True where the pattern is detected and False otherwise.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_three_white = is_white.rolling(3).apply(lambda x: all(x), raw=True)

    open_within_body = (df['open'] - df['close'].shift(1)).abs() <= df['close'].shift(1) * open_within_body_threshold
    three_open_within_body = open_within_body.rolling(3).apply(lambda x: all(x), raw=True)


    upper_shadow_1 = df['high'].shift(2) - df['close'].shift(2)
    upper_shadow_2 = df['high'].shift(1) - df['close'].shift(1)
    upper_shadow_3 = df['high'] - df['close']

    upper_shadow_growth1 = (upper_shadow_2 - upper_shadow_1) >= upper_shadow_1 * shadow_growth_threshold
    upper_shadow_growth2 = (upper_shadow_3 - upper_shadow_2) >= upper_shadow_2 * shadow_growth_threshold

    upper_shadow_growth = upper_shadow_growth1 & upper_shadow_growth2

    result = is_three_white & three_open_within_body & upper_shadow_growth
    return result


# Ref: https://thepatternsite.com/AbandonBaby.html
# This function detects the Bearish Abandoned Baby candlestick pattern.
# It identifies the pattern based on the criteria described in the provided HTML documentation.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating the presence or absence of the pattern for each row (candle).

def do_detect_bearish_abandoned_baby(df: pd.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bearish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) required between candles.

    Returns:
        Pandas Series: Boolean Series indicating Bearish Abandoned Baby patterns (True for bearish abandoned baby).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_black = df["close"] < df["open"]
    is_doji = abs(df["close"] - df["open"]) / (df["high"] - df["low"]) < 0.001 #Approximating doji, can be tuned.

    # Shift series to compare with previous candles.
    prev_is_white = is_white.shift(1)
    prev_high = df["high"].shift(1)
    prev_low = df["low"].shift(1)
    prev_close = df["close"].shift(1)
    
    prev_is_black = is_black.shift(2)

    #Calculate Gap
    gap_size = df['low'] - df['high'].shift(1)
    prev_range = (df['high'].shift(1) - df['low'].shift(1))
    gap_percentage = gap_size/prev_range


    #Identify Pattern
    bearish_abandoned_baby = (prev_is_white) & (is_doji) & (is_black) & (gap_percentage > gap_threshold) & (df["low"] > prev_high) & (df["high"].shift(1) > prev_close) & (df["high"] < df["high"].shift(1) )


    return bearish_abandoned_baby


# Ref: https://thepatternsite.com/BeltHoldBear.html
# The following function detects the Bearish Belt Hold candlestick pattern.
# A Bearish Belt Hold is characterized by a single black candle in an uptrend where:
#   - The open is near the high of the day.
#   - The close is near the low of the day.
#   - There's often a small lower shadow.

def do_detect_bearish_belt_hold(df: pd.DataFrame, open_high_threshold: float = 0.02, close_low_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Bearish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        open_high_threshold: Maximum difference between open and high as a fraction of the high.
        close_low_threshold: Maximum difference between close and low as a fraction of the low.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    open_near_high = (df['high'] - df['open']) / df['high'] <= open_high_threshold
    close_near_low = (df['close'] - df['low']) / df['low'] <= close_low_threshold
    
    is_bearish_belt_hold = is_black & open_near_high & close_near_low

    return is_bearish_belt_hold


# Ref: https://thepatternsite.com/BearBreakaway.html
# The following function implements the Bearish Breakaway candlestick pattern detection algorithm 
# as described in the provided HTML documentation.  It identifies the pattern based on a sequence 
# of five candlesticks exhibiting specific characteristics in terms of their body lengths, gaps, 
# and closing prices within an upward price trend.

def do_detect_bearish_breakaway(df: pd.DataFrame, gap_threshold: float = 0.03, tall_candle_threshold: float = 0.05) -> pd.Series:
    """
    Detects the Bearish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        gap_threshold: Minimum percentage gap between consecutive candles.
        tall_candle_threshold: Minimum height of the first and last candles as a fraction of the preceding candle's range.


    Returns:
        A pandas Series with True for rows corresponding to the Bearish Breakaway pattern, False otherwise.
        Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    candle_range = df['high'] - df['low']
    body_size = abs(df['close'] - df['open'])

    #Detect tall candles
    is_first_tall = body_size.shift(4) >= tall_candle_threshold * candle_range.shift(4)
    is_last_tall = body_size >= tall_candle_threshold * candle_range.shift(1)

    #Detect gaps
    gap_up = df['open'].shift(1) > df['close']
    gap_1 = gap_up & (df['open'].shift(1) - df['close'] >= gap_threshold * candle_range.shift(1))
    gap_2 = (df['open'].shift(2) > df['close'].shift(1)) & (df['open'].shift(2) - df['close'].shift(1) >= gap_threshold * candle_range.shift(2))
    gap_3 = (df['open'].shift(3) > df['close'].shift(2)) & (df['open'].shift(3) - df['close'].shift(2) >= gap_threshold * candle_range.shift(3))

    #Check that gaps are upwards
    gap_direction_correct = ((df['open'].shift(1) > df['close']) & (df['open'].shift(2) > df['close'].shift(1)) & (df['open'].shift(3) > df['close'].shift(2)))


    #Check if candles are white, except last
    is_white_rest = is_white.shift(4) & is_white.shift(3) & is_white.shift(2) & is_white.shift(1)
    is_black_last = ~is_white

    #Ensure increasing closes
    increasing_closes = df['close'].shift(4) < df['close'].shift(3) < df['close'].shift(2) < df['close'].shift(1)

    #Combine conditions
    pattern = (is_first_tall & is_last_tall & gap_direction_correct & is_white_rest & is_black_last & increasing_closes & (df['close'] < df['open'].shift(1)) & (df['close'] > df['open'].shift(4)) )

    return pattern

# Ref: https://thepatternsite.com/DojiStarBear.html
# Detects the Bearish Doji Star candlestick pattern.
# The pattern consists of two candles: a long white candle followed by a doji
# that gaps higher. The doji's body should be relatively small compared to the
# preceding white candle, and its shadows should be short.  This function
# implements a simplified version of the pattern.  Further refinement may be
# necessary for improved accuracy and robustness.

def do_detect_bearish_doji_star(df: pd.DataFrame, body_ratio_threshold: float = 0.2, shadow_ratio_threshold: float = 0.5, gap_threshold:float = 0.001) -> pd.Series:
    """
    Detects the Bearish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must contain columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
        body_ratio_threshold: Maximum ratio of the doji's body size to the preceding candle's body size.
        shadow_ratio_threshold: Maximum ratio of the sum of the doji's shadows to the preceding candle's body size.
        gap_threshold: Minimum gap (as a fraction) between the doji's open and the previous candle's close.

    Returns:
        A pandas Series indicating Bearish Doji Star patterns (True for bearish doji star, False otherwise).
        Returns an empty Series if input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    shadow_size = df['high'] - df['low'] - body_size

    #Calculate the previous candle's body size and shadow size
    prev_body_size = body_size.shift(1)
    prev_shadow_size = shadow_size.shift(1)


    #Check for gap
    gap = df['open'] - df['close'].shift(1)


    #Detect doji (simplified condition)
    is_doji = abs(df['close'] - df['open']) / (df['high'] - df['low']) < body_ratio_threshold

    #Check if preceding candle was a long white candle
    is_long_white_prev = (prev_body_size > 0) & (prev_body_size > prev_shadow_size)

    #Check if gap is significant
    is_significant_gap = gap > gap_threshold * (df['high'].shift(1) - df['low'].shift(1))


    #Check for Bearish Doji Star conditions
    bearish_doji_star = (is_long_white_prev) & (is_doji) & (is_significant_gap) & (shadow_size < shadow_ratio_threshold * prev_body_size)

    return bearish_doji_star


# Ref: https://thepatternsite.com/BearEngulfing.html
# Detects a bearish engulfing pattern.
# The first candle is white (close > open), and the second candle is black (close < open).
# The body of the black candle overlaps the body of the white candle.
# Shadows are unimportant.

def do_detect_bearish_engulfing(df: pd.DataFrame, body_ratio_threshold: float = 0.0) -> pd.Series:
    """
    Detects Bearish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns "open", "high", "low", "close".
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Bearish Engulfing patterns.  Index is aligned with input DataFrame.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    white_body_size = (df['close'] - df['open']).where(is_white)
    black_body_size = (df['open'] - df['close']).where(is_black)
    
    shifted_is_white = is_white.shift(1)
    shifted_white_body_size = white_body_size.shift(1)

    engulfed = (is_white.shift(1) & is_black) & (df['open'] > df['close'].shift(1)) & (df['close'] < df['open'].shift(1))
    
    body_ratio = black_body_size / shifted_white_body_size
    
    return engulfed & (body_ratio >= body_ratio_threshold)


# Ref: https://thepatternsite.com/HaramiBear.html
# Detects the Bearish Harami candlestick pattern.
# The pattern consists of a tall white candle followed by a small black candle.
# The opening and closing prices of the black candle must be within the body of the white candle.
# Shadows are ignored in the identification.  Either the tops or bottoms (or both) of the candle bodies must be at a different price.

def do_detect_bearish_harami(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects Bearish Harami pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Bearish Harami patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    
    first_candle_body_size = body_size.shift(1)
    second_candle_body_size = body_size

    body_ratio = second_candle_body_size / first_candle_body_size
    
    within_body = (df['open'] >= df['open'].shift(1)) & (df['close'] <= df['close'].shift(1))

    is_bearish_harami = (is_white.shift(1) & is_black & within_body & (body_ratio <= body_ratio_threshold))

    return is_bearish_harami

# Ref: https://thepatternsite.com/HaramiCrossBear.html
# Detects the Bearish Harami Cross candlestick pattern.
# The pattern consists of two candles:
# 1. A long white candle (close > open).
# 2. A doji candle (open ≈ close) that is entirely within the range of the first candle.  

def do_detect_bearish_harami_cross(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bearish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.  Must contain columns: "open", "high", "low", "close", "volume", and "date".
        doji_threshold: The maximum difference between open and close for a candle to be considered a doji (as a fraction of the range).

    Returns:
        A pandas Series of booleans indicating the presence of the Bearish Harami Cross pattern.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_doji = abs(df["close"] - df["open"]) / (df["high"] - df["low"]) <= doji_threshold

    # Shift to compare the current day's doji to the previous day's white candle
    prev_high = df["high"].shift(1)
    prev_low = df["low"].shift(1)
    
    # Check conditions: Previous candle is white, current is doji, and current is inside the previous
    is_bearish_harami_cross = (is_white.shift(1) & is_doji & (df["high"] <= prev_high) & (df["low"] >= prev_low))

    return is_bearish_harami_cross

# Ref: https://thepatternsite.com/KickingBear.html
# The bearish kicking candlestick pattern consists of two marubozu candles: a white one followed by a black one, separated by a gap.
# This function detects this pattern by checking for the specified conditions.
# The gap threshold parameter controls the minimum gap size between the two candles as a percentage of the previous candle's range.

def do_detect_bearish_kicking(df: pd.DataFrame, gap_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Bearish Kicking candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) between candles.

    Returns:
        pd.Series: Boolean Series indicating Bearish Kicking patterns (True for matches, False otherwise).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    is_marubozu_white = is_white & (df['high'] == df['close']) & (df['low'] == df['open'])
    is_marubozu_black = is_black & (df['high'] == df['open']) & (df['low'] == df['close'])

    previous_candle_range = df['high'].shift(1) - df['low'].shift(1)
    gap = df['open'] - df['close'].shift(1)
    is_gap = gap > previous_candle_range.shift(1) * gap_threshold

    bearish_kicking = (is_marubozu_white.shift(1) & is_marubozu_black & is_gap)

    return bearish_kicking


# Ref: https://thepatternsite.com/MeetingLinesBear.html
# Detects the Bearish Meeting Lines candlestick pattern.
# The pattern consists of two candles: a tall white candle followed by a tall black candle,
# with their closing prices relatively close to each other.  This function implements a simplified
# version of the pattern recognition, focusing on candle body lengths and closing price proximity.
# The function takes a DataFrame and two thresholds, one for the minimum candle body length (as a fraction of the candle range)
# and another for the maximum difference between closing prices (as a fraction of the average candle range).
def do_detect_bearish_meeting_lines(df: pd.DataFrame, min_body_fraction: float = 0.2, max_close_diff_fraction: float = 0.1) -> pd.Series:
    """
    Detects the Bearish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        min_body_fraction: Minimum fraction of the candle range that the body must occupy to be considered tall.
        max_close_diff_fraction: Maximum allowed difference between closing prices of the two candles.

    Returns:
        Boolean Series indicating Bearish Meeting Lines pattern occurrences.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    candle_range = df['high'] - df['low']
    body_size = abs(df['close'] - df['open'])
    is_tall = body_size / candle_range >= min_body_fraction

    is_bearish_meeting_lines = (is_tall & is_white).shift(1) & (is_tall & ~is_white)

    avg_range = (candle_range.shift(1) + candle_range).fillna(0) / 2
    close_diff = abs(df['close'].shift(1) - df['close'])
    close_diff_fraction = close_diff / avg_range
    is_close_enough = close_diff_fraction <= max_close_diff_fraction

    return is_bearish_meeting_lines & is_close_enough


# Ref: https://thepatternsite.com/SeparateLinesBear.html
# This function detects the Bearish Separating Lines candlestick pattern.
# It checks for a tall white candle followed by a tall black candle with similar opening prices.
# The function uses relative thresholds for candle body sizes and opening price difference to define "tall" and "similar".

def do_detect_bearish_separating_lines(df: pd.DataFrame, white_candle_body_threshold: float = 0.7, black_candle_body_threshold: float = 0.7, opening_price_diff_threshold: float = 0.05) -> pd.Series:
    """
    Detects the Bearish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close').
        white_candle_body_threshold: Minimum body size ratio (close-open)/range for the first (white) candle.
        black_candle_body_threshold: Minimum body size ratio (open-close)/range for the second (black) candle.
        opening_price_diff_threshold: Maximum difference between opening prices of the two candles (as fraction).


    Returns:
        A pandas Series with True indicating the pattern, False otherwise.  Returns an empty Series if the dataframe is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    # Calculate the body size as a fraction of the candle range.
    white_body_ratio = (df['close'] - df['open']) / (df['high'] - df['low'])
    black_body_ratio = (df['open'] - df['close']) / (df['high'] - df['low'])

    # Identify potential white and black candles, ignoring NaN values
    potential_white_candles = white_body_ratio.mask(is_white == False).fillna(0) >= white_candle_body_threshold
    potential_black_candles = black_body_ratio.mask(is_black == False).fillna(0) >= black_candle_body_threshold

    #Shift the candles to compare consecutive candles
    shifted_potential_white_candles = potential_white_candles.shift(1)

    #Check for similar opening prices
    opening_price_diff = abs(df['open'] - df['open'].shift(1)) / df['open'].shift(1)
    similar_openings = opening_price_diff <= opening_price_diff_threshold
    
    #Combine conditions
    pattern_detected = (shifted_potential_white_candles & potential_black_candles & similar_openings)

    return pattern_detected


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBear.html
# This function detects the 'Bearish Side by Side White Lines' candlestick pattern.
# It identifies three candles: a black candle followed by two white candles with similar bodies and opening prices,
# where the closing prices of the white candles are below the black candle's body.
# The function takes a Pandas DataFrame as input with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

def do_detect_bearish_side_by_side_white_lines(df: pd.DataFrame, body_similarity_threshold: float = 0.5, opening_price_similarity_threshold: float = 0.1) -> pd.Series:
    """
    Detects the Bearish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.
        body_similarity_threshold: Maximum allowed difference in body size between the two white candles (as a fraction of the average body size).
        opening_price_similarity_threshold: Maximum allowed difference between opening prices of the two white candles (as a fraction of the average opening price).

    Returns:
        A pandas Series (boolean mask) indicating the pattern.  FIXME: Thresholds need further refinement.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    
    pattern_mask = (is_black.shift(2) &
                    is_white.shift(1) &
                    is_white &
                    (abs(body_size.shift(1) - body_size) <= (body_size.shift(1) + body_size).mean() * body_similarity_threshold) &
                    (abs(df['open'].shift(1) - df['open']) <= (df['open'].shift(1) + df['open']).mean() * opening_price_similarity_threshold) &
                    (df['close'].shift(1) < df['open'].shift(2)) &
                    (df['close'] < df['open'].shift(2))
                   )
    return pattern_mask


# Ref: https://thepatternsite.com/ThreeLineStrikeBear.html
# This function detects the Bearish Three-Line Strike candlestick pattern.
# The pattern consists of three consecutive black candles with decreasing lows,
# followed by a tall white candle that opens below the previous day's close
# and closes above the first day's open.

def do_detect_bearish_three_line_strike(df: pd.DataFrame, body_threshold: float = 0.2, gap_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Bearish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        body_threshold: Minimum ratio of body size to candle range for each of the three candles.
        gap_threshold: Minimum percentage gap between consecutive open and previous close.


    Returns:
        pd.Series: Boolean Series indicating Bearish Three Line Strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    body_ratio = body_size / candle_range
    
    # Identify three consecutive black candles with decreasing lows
    three_black_candles = (is_black & (body_ratio >= body_threshold) & 
                           df['low'].shift(1) > df['low'].shift(2) & 
                           df['low'].shift(2) > df['low'].shift(3))

    #Check for decreasing lows
    decreasing_lows = (df['low'].shift(1) > df['low'].shift(2) & df['low'].shift(2) > df['low'].shift(3))

    # Check for the fourth candle (tall white candle)
    fourth_candle_open_below_prior_close = df['open'].shift(-3) < df['close'].shift(-2)
    fourth_candle_close_above_first_open = df['close'].shift(-3) > df['open'].shift(-1)


    # Combine conditions to detect the pattern
    pattern = (three_black_candles & decreasing_lows & fourth_candle_open_below_prior_close & 
               fourth_candle_close_above_first_open & is_white.shift(-3))

    return pattern

# Ref: https://thepatternsite.com/TriStarBear.html
# This function detects the Bearish Tri-Star candlestick pattern.
# The pattern consists of three doji candles, with the middle one having a body above the other two.
# A doji is a candle where the open and close prices are very close.
# The function identifies this pattern by comparing the open, high, low, and close prices of consecutive candles.
# It returns a boolean Series indicating whether a Bearish Tri-Star pattern is present for each candle.

def do_detect_bearish_tri_star(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bearish Tri-Star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        doji_threshold: The maximum difference between open and close for a candle to be considered a doji (as a fraction of the candle's range).

    Returns:
        A boolean Series indicating whether a Bearish Tri-Star pattern is present.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size and range of each candle.
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    # Identify doji candles.
    is_doji = body_size / candle_range <= doji_threshold

    # Shift the boolean series to align with the three-candle pattern
    is_doji_shifted = is_doji.shift(1)
    is_doji_shifted2 = is_doji.shift(2)

    # Check for the Bearish Tri-Star pattern.
    is_tri_star = (is_doji) & (is_doji_shifted) & (is_doji_shifted2) & (df['close'].shift(1) > df['close']) & (df['close'].shift(1) > df['close'].shift(2))

    return is_tri_star

# Ref: https://thepatternsite.com/BelowStomach.html
# The Below the Stomach pattern consists of a tall white candle followed by a candle whose body is below the midpoint of the previous candle.
# This function detects this pattern.  The algorithm is relatively straightforward and does not require complex calculations.
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# The function returns a pandas Series of booleans indicating whether the pattern is present for each row in the DataFrame.

def do_detect_below_the_stomach(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Below the Stomach candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Below the Stomach patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size1 = abs(df['close'] - df['open'])
    midpoint1 = df['open'] + body_size1 / 2
    body_size2 = abs(df['close'].shift(-1) - df['open'].shift(-1))
    body_low2 = df[['open','close']].shift(-1).min(axis=1)
    
    is_below_midpoint = body_low2 < midpoint1

    is_pattern = is_white & is_below_midpoint & (body_size2 / body_size1 >= body_ratio_threshold)

    return is_pattern

# Ref: https://thepatternsite.com/BeltHoldBear.html
# This function detects the Bearish Belt Hold candlestick pattern.
# A Bearish Belt Hold is a black candle that opens near its high and closes near its low in an uptrend.
# The function identifies this pattern by comparing the open, high, and close prices of each candle.
# The function returns a boolean Series where True indicates the presence of a Bearish Belt Hold pattern and False indicates its absence.

def do_detect_bearish_belt_hold(df: pd.DataFrame, open_high_ratio_threshold: float = 0.9, close_low_ratio_threshold: float = 0.9) -> pd.Series:
    """
    Detects the Bearish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        open_high_ratio_threshold: The minimum ratio of open price to high price for a candle to be considered a Bearish Belt Hold.
        close_low_ratio_threshold: The minimum ratio of close price to low price for a candle to be considered a Bearish Belt Hold.

    Returns:
        pd.Series: Boolean Series indicating Bearish Belt Hold patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    open_high_ratio = df["open"] / df["high"]
    close_low_ratio = df["close"] / df["low"]

    is_bearish_belt_hold = (is_black) & (open_high_ratio >= open_high_ratio_threshold) & (close_low_ratio <= close_low_ratio_threshold)

    return is_bearish_belt_hold


# Ref: https://thepatternsite.com/BeltHoldBull.html
# This function detects the "Bullish Belt Hold" candlestick pattern.
# A bullish belt hold is a single white candle with no lower shadow and closing near the high.
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.  
# The index of the input dataframe is preserved.


def do_detect_bullish_belt_hold(df: pd.DataFrame, lower_shadow_threshold: float = 0.01, close_high_ratio: float = 0.95) -> pd.Series:
    """
    Detects the Bullish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        lower_shadow_threshold: Maximum allowed lower shadow as a fraction of the candle body.
        close_high_ratio: Minimum ratio of closing price to high price.

    Returns:
        pd.Series: Boolean Series indicating Bullish Belt Hold patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    lower_shadow = df['open'] - df['low']
    body = df['close'] - df['open']
    lower_shadow_ratio = lower_shadow / body
    close_to_high_ratio = (df['close'] - df['low']) / (df['high'] - df['low'])

    bullish_belt_hold = (is_white) & (lower_shadow_ratio <= lower_shadow_threshold) & (close_to_high_ratio >= close_high_ratio)

    return bullish_belt_hold

# Ref: https://thepatternsite.com/BlkCandle.html
# This function detects the 'Black Candle' pattern based on the description provided in the linked article.
# A black candle is characterized by a candle with shadows smaller than the body height.  The function identifies candles that meet this criteria.

def do_detect_black_candle(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Black Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        body_ratio_threshold:  The minimum ratio of body size to total candle range below which a candle is considered a black candle.


    Returns:
        A pandas Series (boolean mask) indicating black candle patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = df["close"] - df["open"]
    total_range = df["high"] - df["low"]
    is_black = (body_size < 0) & (total_range - abs(body_size) > abs(body_size) * body_ratio_threshold)
    
    return is_black


# Ref: https://thepatternsite.com/LongBlack.html
# This function detects the "Long Black Day" candlestick pattern.
# A long black day is defined as a tall black candle with shadows shorter than the body.
# The body is considered "tall" if it is at least three times the average body height of recent candles.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans, indicating whether each candle is a "Long Black Day".

def do_detect_long_black_day(df: pd.DataFrame, body_height_factor: float = 3.0, recent_candles: int = 10) -> pd.Series:
    """
    Detects the Long Black Day candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close) and volume.  Must have a date column.
        body_height_factor: Factor determining the minimum body size relative to the average body size of recent candles.
        recent_candles: Number of recent candles to calculate the average body height from.

    Returns:
        pd.Series: Boolean Series indicating Long Black Day patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body sizes
    body_sizes = abs(df['close'] - df['open'])

    # Calculate average body size of recent candles
    avg_body_size = body_sizes.rolling(window=recent_candles, min_periods=1).mean()

    # Identify long black candles
    is_long_body = body_sizes >= avg_body_size * body_height_factor
    is_black_candle = df['close'] < df['open']
    is_short_shadows = (df['high'] - df['close']) < body_sizes  & (df['open'] - df['low']) < body_sizes


    # Combine conditions to detect long black day pattern
    is_long_black_day = is_long_body & is_black_candle & is_short_shadows

    return is_long_black_day

# Ref: https://thepatternsite.com/BlackMarubozu.html
# The black marubozu is a tall black candle with no shadows.  This function identifies
# them based on the open, high, low, and close prices.
# A threshold parameter controls the minimum size of the candle body relative to its total range.

def do_detect_black_marubozu(df: pd.DataFrame, body_range_threshold: float = 0.7) -> pd.Series:
    """
    Detects the Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        body_range_threshold: Minimum ratio of the candle body to the total candle range (high - low).

    Returns:
        pd.Series: Boolean Series indicating Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    total_range = df['high'] - df['low']
    body_size = abs(df['close'] - df['open'])  #Always positive
    body_range_ratio = body_size / total_range
    is_marubozu = body_range_ratio >= body_range_threshold


    is_black_marubozu = is_black & is_marubozu
    return is_black_marubozu


# Ref: https://thepatternsite.com/BlkCandleShort.html
# Detects a short black candle pattern.
# A short black candle is characterized by a small body and relatively short shadows.
# The function checks if the candle's body is shorter than a specified threshold and
# if the upper and lower shadows are shorter than the body.
# Parameters:
#   df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close' columns.
#   body_threshold_ratio: The maximum ratio of the candle body to the total candle range.
#   shadow_body_ratio: The maximum ratio of shadow length to candle body length.

def do_detect_short_black_candle(df: pd.DataFrame, body_threshold_ratio: float = 0.2, shadow_body_ratio: float = 0.5) -> pd.Series:
    """
    Detects short black candles in OHLC data.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close').  The index must be retained in the return value, and the dataframe must not be modified.
        body_threshold_ratio: Maximum ratio of candle body size to total range (high - low).
        shadow_body_ratio: Maximum ratio of each shadow length to candle body length.

    Returns:
        pd.Series: Boolean Series indicating short black candle patterns (True if short black candle, False otherwise).
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body = df["close"] - df["open"]
    is_black = body < 0
    total_range = df["high"] - df["low"]
    body_ratio = abs(body) / total_range
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    is_short_body = body_ratio <= body_threshold_ratio
    is_short_shadow_upper = upper_shadow <= abs(body) * shadow_body_ratio
    is_short_shadow_lower = lower_shadow <= abs(body) * shadow_body_ratio

    is_short_black_candle = is_black & is_short_body & is_short_shadow_upper & is_short_shadow_lower
    return is_short_black_candle

# Ref: https://thepatternsite.com/SpinTopBlack.html
# This function detects the Black Spinning Top candlestick pattern.
# A black spinning top is characterized by a small black body (close < open) and shadows taller than the body.
# The function takes a Pandas DataFrame as input and returns a boolean Series indicating the presence of the pattern.

def do_detect_black_spinning_top(df: pd.DataFrame, body_size_threshold: float = 0.1, shadow_body_ratio_threshold: float = 1.0) -> pd.Series:
    """
    Detects Black Spinning Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.  Must have columns "open", "high", "low", "close", "volume", "date".
        body_size_threshold:  The maximum acceptable ratio of body size to the total candle range.
        shadow_body_ratio_threshold: The minimum ratio of the total shadow length to body size.


    Returns:
        Boolean Series indicating Black Spinning Top patterns.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    total_shadow = upper_shadow + lower_shadow
    
    is_small_body = (body_size / total_range) <= body_size_threshold
    is_tall_shadow = (total_shadow / body_size) >= shadow_body_ratio_threshold

    is_black_spinning_top = is_black & is_small_body & is_tall_shadow

    return is_black_spinning_top


# Ref: https://thepatternsite.com/BearBreakaway.html
# This function detects the Bearish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall white candle.
# 2. A white candle with a gap between its body and the body of the first candle.
# 3. A candle with a higher close than the second candle (color doesn't matter).
# 4. A white candle with a higher close than the third candle.
# 5. A tall black candle whose close is within the gap between the bodies of the first two candles.


def do_detect_bearish_breakaway(df: pd.DataFrame, gap_threshold: float = 0.03, tall_candle_threshold: float = 0.05) -> pd.Series:
    """
    Detects the Bearish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
        gap_threshold: Minimum percentage gap between candles.
        tall_candle_threshold: Minimum body size ratio relative to the previous candle's range to define a tall candle.

    Returns:
        Boolean Series indicating Bearish Breakaway patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    
    # Calculate gaps between consecutive candles
    gaps = (df['open'].shift(-1) - df['close']) / df['close']

    # Identify tall white candles
    tall_white = (body_size / candle_range) > tall_candle_threshold
    tall_white_mask = is_white & tall_white
    
    #Identify tall black candles
    tall_black = (body_size / candle_range) > tall_candle_threshold
    tall_black_mask = (~is_white) & tall_black


    # Check for the 5-candle pattern
    bearish_breakaway = (
        tall_white_mask.shift(4) &
        (is_white.shift(3) & (gaps.shift(3) > gap_threshold)) &
        (df['close'].shift(2) > df['close'].shift(3)) &
        (is_white.shift(2) & (df['close'].shift(1) > df['close'].shift(2)) ) &
        (tall_black_mask & (df['close'] > df['open'].shift(1)) & (df['close'] < df['open'].shift(4)) )
    )

    return bearish_breakaway


# Ref: https://thepatternsite.com/BullBreakaway.html
# This function detects the Bullish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall black candle.
# 2. Another black candle opening lower, leaving a gap between the two bodies.
# 3. A candle of any color with a lower close than candle 2.
# 4. A black candle with a lower close than candle 3.
# 5. A tall white candle closing within the body gap of the first two candles.
# The function takes a pandas DataFrame with OHLCV data as input and returns a pandas Series of booleans,
# indicating whether a Bullish Breakaway pattern is detected at each point in time.

def do_detect_bullish_breakaway(df: pd.DataFrame, gap_threshold: float = 0.02, body_ratio_threshold: float = 0.6):
    """
    Detects the Bullish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLCV data.
        gap_threshold: Minimum gap (as a fraction of the previous candle's range) required between candles 1 and 2.
        body_ratio_threshold: Minimum ratio of the body size of candle 5 to the body size of candle 1.

    Returns:
        pd.Series: Boolean Series indicating Bullish Breakaway patterns.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    # Identify potential starting points of the pattern (tall black candle)
    potential_starts = (is_black & (body_size >= body_ratio_threshold * candle_range)).shift(4)


    # Check conditions for the remaining candles
    condition1 = is_black & (df['open'] < df['open'].shift(1) - gap_threshold * candle_range.shift(1)) #Candle 2
    condition2 = (df['close'] < df['close'].shift(1))  #Candle 3
    condition3 = is_black & (df['close'] < df['close'].shift(1)) #Candle 4
    condition4 = is_white & (df['close'] < df['open'].shift(1) )& (df['close'] > df['open'].shift(4) )#Candle 5


    bullish_breakaway = potential_starts & condition1.shift(3) & condition2.shift(2) & condition3.shift(1) & condition4

    return bullish_breakaway


# Ref: https://thepatternsite.com/AbandonBabyBull.html
# This function detects the Bullish Abandoned Baby candlestick pattern.
# It identifies a three-candle pattern characterized by a black candle, followed by a doji, 
# and then a white candle with specific gap requirements.
# The pattern suggests a bullish reversal.

def do_detect_bullish_abandoned_baby(df: pd.DataFrame, gap_threshold: float = 0.01, doji_threshold: float = 0.005) -> pd.Series:
    """
    Detects the Bullish Abandoned Baby candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap (as a fraction of the previous candle's range) between candles.
        doji_threshold: Maximum difference (as a fraction) between open and close for a doji candle.

    Returns:
        A pandas Series with True where a Bullish Abandoned Baby pattern is detected and False otherwise.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    is_doji = abs(df['close'] - df['open']) / df['high'] <= doji_threshold


    bullish_abandoned_baby = pd.Series(False, index=df.index)

    for i in range(2, len(df)):
        # Check for the three-candle pattern
        if is_black.iloc[i-2] and is_doji.iloc[i-1] and is_white.iloc[i]:
            # Check for gaps (simplified version)
            prev_range = df['high'].iloc[i-2] - df['low'].iloc[i-2]
            if df['low'].iloc[i-1] - df['high'].iloc[i-2] > gap_threshold * prev_range and \
               df['low'].iloc[i] > df['close'].iloc[i-1]:
                bullish_abandoned_baby.iloc[i] = True

    return bullish_abandoned_baby


# Ref: https://thepatternsite.com/BeltHoldBull.html
# This function detects the Bullish Belt Hold candlestick pattern.
# A bullish belt hold is a single white candle with no lower shadow, closing near the high.
# The function takes a Pandas DataFrame as input, with columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
# It returns a Pandas Series of booleans, indicating whether each candle is a bullish belt hold.

def do_detect_bullish_belt_hold(df: pd.core.frame.DataFrame, lower_shadow_threshold: float = 0.01, close_to_high_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Belt Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        lower_shadow_threshold: Maximum allowed lower shadow as a fraction of the candle's total range.
        close_to_high_threshold: Maximum allowed distance between close and high as a fraction of the candle's total range.

    Returns:
        pd.Series: Boolean Series indicating Bullish Belt Hold patterns.  Returns an empty Series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    lower_shadow = df['open'] - df['low']
    total_range = df['high'] - df['low']
    is_no_lower_shadow = (lower_shadow / total_range) < lower_shadow_threshold
    close_to_high = df['high'] - df['close']
    is_close_to_high = (close_to_high / total_range) < close_to_high_threshold

    bullish_belt_hold = is_white & is_no_lower_shadow & is_close_to_high
    
    return bullish_belt_hold


# Ref: https://thepatternsite.com/BullBreakaway.html
# This function detects the Bullish Breakaway candlestick pattern.
# The pattern consists of five candles:
# 1. A tall black candle.
# 2. Another black candle opening lower, leaving a gap between the bodies.
# 3. A candle of any color with a lower close than the previous candle.
# 4. A black candle with a lower close than the previous candle.
# 5. A tall white candle closing within the body gap of the first two candles.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_breakaway(df: pd.DataFrame, gap_threshold: float = 0.02, body_ratio_threshold: float = 0.5):
    """
    Detects the Bullish Breakaway candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must contain 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap (as a fraction of the previous candle's range) between the first two candles.
        body_ratio_threshold: Minimum ratio of the fifth candle's body to the first candle's body.

    Returns:
        pd.Series: Boolean Series indicating Bullish Breakaway patterns.
        FIXME: Add more robust checks to improve accuracy.  For example, check for minimum candle body size.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    # Identify potential first candle
    potential_first_candles = is_black & (body_size / candle_range > body_ratio_threshold)

    #Shift values appropriately
    second_candle_open = df['open'].shift(1)
    second_candle_close = df['close'].shift(1)
    second_candle_high = df['high'].shift(1)
    second_candle_low = df['low'].shift(1)
    second_candle_body_size = abs(second_candle_close - second_candle_open)
    second_candle_range = second_candle_high - second_candle_low

    third_candle_close = df['close'].shift(2)
    fourth_candle_close = df['close'].shift(3)
    fifth_candle_body = body_size.shift(4)
    fifth_candle_open = df['open'].shift(4)
    first_candle_body = body_size.shift(4)



    bullish_breakaway = (
        potential_first_candles &
        is_black.shift(1) &
        (df['open'].shift(1) < df['close'].shift(1) - gap_threshold * second_candle_range) &
        (df['close'].shift(2) < df['close'].shift(1)) &
        is_black.shift(2) &
        (df['close'].shift(3) < df['close'].shift(2)) &
        is_white.shift(3) &
        (fifth_candle_body/first_candle_body > body_ratio_threshold) &
        (df['close'].shift(3) > fifth_candle_open) &
        (df['open'].shift(1) > df['close'].shift(3))
    )

    return bullish_breakaway


# Ref: https://thepatternsite.com/DojiStarBull.html
# Detects the Bullish Doji Star candlestick pattern.
# The pattern consists of a tall black candle followed by a doji that gaps below the prior candle's body.
# The doji's shadows should not be unusually long.  A downward breakout confirms the pattern as bearish continuation.

def do_detect_bullish_doji_star(df: pd.DataFrame, body_ratio_threshold: float = 0.1, doji_threshold: float = 0.01, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects Bullish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        body_ratio_threshold: Minimum ratio of the first candle's body to its total range.
        doji_threshold: Maximum difference between open and close prices for the doji candle.
        gap_threshold: Minimum gap between close of the first candle and open of the doji candle, relative to previous day's candle range.

    Returns:
        A pandas Series with True where the pattern is detected and False otherwise.
        FIXME: Consider adding more robust criteria for shadow length.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    total_range = df['high'] - df['low']
    is_tall_black = (body_size / total_range) >= body_ratio_threshold
    
    shifted_close = df['close'].shift(1)
    shifted_range = (df['high'].shift(1) - df['low'].shift(1)).fillna(0)

    is_doji = abs(df['close'] - df['open']) <= doji_threshold * shifted_range
    is_gap_below = df['open'] < shifted_close - gap_threshold * shifted_range

    bullish_doji_star = is_black & is_tall_black & is_doji & is_gap_below & is_black.shift(1)

    return bullish_doji_star


# Ref: https://thepatternsite.com/BullEngulfing.html
# Detects a bullish engulfing pattern.  The first candle is black (close < open), and the second is white (close > open) and taller than the first.
# The white candle's body must engulf or overlap the black candle's body. Shadows are ignored.


def do_detect_bullish_engulfing(df: pd.core.frame.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects Bullish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size for engulfment.

    Returns:
        pd.Series: Boolean Series indicating Bullish Engulfing patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    
    first_candle_body_size = abs(df['close'].shift(1) - df['open'].shift(1))
    second_candle_body_size = abs(df['close'] - df['open'])

    engulfment_condition = (second_candle_body_size >= first_candle_body_size * body_ratio_threshold) & (df['close'] > df['open'].shift(1)) & (df['open'] < df['close'].shift(1))
    
    return (is_black.shift(1) & is_white & engulfment_condition)

# Ref: https://thepatternsite.com/HaramiBull.html
# Detects the Bullish Harami candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a white candle whose body is entirely contained within the body of the black candle.
# The function takes a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.  
# The index of the DataFrame is retained in the output. The DataFrame itself is not modified.
# An empty Series is returned only if the input DataFrame is empty.

def do_detect_bullish_harami(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects Bullish Harami candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Bullish Harami patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    
    black_body_size = abs(df['close'] - df['open'])
    white_body_size = abs(df['close'].shift(-1) - df['open'].shift(-1))
    
    #Calculate the ratio of the white candle body size to the black candle body size for pairs of candles
    body_ratio = white_body_size / black_body_size
    
    #Check if the white body is fully inside the black body
    is_inside = (df['open'].shift(-1) > df['open']) & (df['close'].shift(-1) < df['close'])
    
    # Combine conditions: black candle, then white candle, white candle inside black candle body, and the size ratio threshold is met.
    bullish_harami = (is_black & is_white.shift(-1) & is_inside & (body_ratio <= body_ratio_threshold))
    
    return bullish_harami

# Ref: https://thepatternsite.com/HaramiCrossBull.html
# Detects the Bullish Harami Cross candlestick pattern.
# The pattern consists of two candles:
# 1. A long black candle.
# 2. A doji candle that is entirely contained within the body of the first candle.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_harami_cross(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have 'open', 'high', 'low', 'close' columns. Index must be datetime.
        doji_threshold: The maximum difference (as a fraction of the first candle's range) between open and close to be considered a doji.

    Returns:
        pd.Series: Boolean Series indicating Bullish Harami Cross patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_doji = abs(df['close'] - df['open']) / (df['high'] - df['low']) <= doji_threshold

    first_candle_black = is_black.shift(1)
    second_candle_doji = is_doji

    within_range = (df['open'] >= df['open'].shift(1)) & (df['close'] <= df['close'].shift(1)) & (df['high'] <= df['high'].shift(1)) & (df['low'] >= df['low'].shift(1))

    bullish_harami_cross = first_candle_black & second_candle_doji & within_range

    return bullish_harami_cross

# Ref: https://thepatternsite.com/KickingBull.html
# The function detects the Bullish Kicking candlestick pattern.
# It identifies a tall black marubozu candle followed by an upward gap and a tall white marubozu candle.
# Parameters:
#     df: DataFrame with OHLC data, and volume, date columns.
#     body_threshold: Minimum ratio of body size to candle range.
#     gap_threshold: Minimum gap between the close of the first candle and open of the second candle, as a fraction of the previous candle's range.
# Returns:
#     pd.Series: Boolean Series indicating Bullish Kicking patterns.

def do_detect_bullish_kicking(df: pd.DataFrame, body_threshold: float = 0.9, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Kicking candlestick pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    is_marubozu_black = (body_size / candle_range) >= body_threshold
    is_marubozu_white = (body_size / candle_range) >= body_threshold

    # shift to avoid using future data
    prev_close = df["close"].shift(1)
    
    # Calculate gap as a fraction of previous candle's range
    gap = (df["open"] - prev_close)
    prev_candle_range = (df["high"].shift(1) - df["low"].shift(1))
    gap_frac = gap / prev_candle_range
    
    bullish_kicking = (is_black & is_marubozu_black & is_white.shift(-1) & is_marubozu_white.shift(-1) & (gap_frac > gap_threshold))
    
    return bullish_kicking

# Ref: https://thepatternsite.com/MeetingLinesBull.html
# Detects the Bullish Meeting Lines candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle.
# The closes of the two candles should be near one another.
# The function checks for this configuration and returns a boolean Series indicating the presence of the pattern.


def do_detect_bullish_meeting_lines(df: pd.core.frame.DataFrame, close_distance_threshold: float = 0.01, min_body_size_ratio: float = 2.0) -> pd.Series:
    """
    Detects the Bullish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        close_distance_threshold: Maximum difference (as fraction) between the close prices of the two candles.
        min_body_size_ratio: Minimum ratio of the body size to the candle range for both candles.

    Returns:
        pd.Series: Boolean Series indicating Bullish Meeting Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    body_size_ratio = body_size / candle_range


    bullish_meeting_lines = pd.Series(False, index=df.index)

    for i in range(1, len(df)):
        if is_black.iloc[i - 1] and is_white.iloc[i] and \
           abs(df["close"].iloc[i - 1] - df["close"].iloc[i]) / df["close"].iloc[i] <= close_distance_threshold and \
           body_size_ratio.iloc[i-1] >= min_body_size_ratio and body_size_ratio.iloc[i] >= min_body_size_ratio:

            bullish_meeting_lines.iloc[i] = True

    return bullish_meeting_lines


# Ref: https://thepatternsite.com/SeparateLinesBull.html
# Detects the Bullish Separating Lines candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle,
# sharing a common opening price.  The definition of "tall" is relative and depends on the context of the price chart.
# Thresholds are used to define what constitutes a "tall" candle and how close the opening prices must be.

def do_detect_bullish_separating_lines(df: pd.DataFrame, opening_price_diff_threshold: float = 0.01, min_body_size_ratio: float = 0.02) -> pd.Series:
    """
    Detects the Bullish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        opening_price_diff_threshold: Maximum difference between the opening prices of the two candles (as a percentage of the average of the two opening prices).
        min_body_size_ratio: Minimum ratio of the body size to the range (high-low) for both candles to be considered "tall".

    Returns:
        Boolean Series indicating Bullish Separating Lines patterns.  Index is aligned with the input DataFrame.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    body_size_ratio = body_size / candle_range

    # Identify potential Bullish Separating Lines patterns:
    # A tall black candle followed by a tall white candle with nearly identical opening prices.
    is_potential_bullish_separating_lines = (
        (body_size_ratio.shift(1) >= min_body_size_ratio) &
        is_black.shift(1) &
        (body_size_ratio >= min_body_size_ratio) &
        is_white &
        (abs(df['open'] - df['open'].shift(1)) <= (df['open'] + df['open'].shift(1))/2 * opening_price_diff_threshold)
    )

    return is_potential_bullish_separating_lines


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBull.html
# This function detects the Bullish Side by Side White Lines candlestick pattern.
# The pattern consists of three consecutive white candles within an upward trend.
# The bodies of the last two candles should be of similar size, open near the same price, and above the high of the first candle.


def do_detect_bullish_side_by_side_white_lines(df: pd.DataFrame, similar_body_threshold: float = 0.2, open_price_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must include columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
        similar_body_threshold: Maximum difference in body size (as fraction) between the second and third candles for them to be considered similar.
        open_price_threshold: Maximum difference in open price (as fraction of the first candle's high) between the second and third candles.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.
        Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size_1 = df['close'] - df['open']
    body_size_2 = df['close'].shift(-1) - df['open'].shift(-1)
    body_size_3 = df['close'].shift(-2) - df['open'].shift(-2)
    open_price_2 = df['open'].shift(-1)
    open_price_3 = df['open'].shift(-2)

    #Check for three consecutive white candles
    three_white_candles = is_white & is_white.shift(-1) & is_white.shift(-2)

    # Check for similar body sizes in the last two candles
    similar_bodies = abs(body_size_2 - body_size_3) / body_size_2 <= similar_body_threshold

    # Check if the open prices of the last two candles are close
    similar_opens = abs(open_price_2 - open_price_3) / df['high'] <= open_price_threshold

    #Check for the last two candles opening above the high of the first candle
    opens_above_high = (open_price_2 >= df['high']) & (open_price_3 >= df['high'])

    # Combine conditions to detect the pattern
    bullish_side_by_side_white_lines = three_white_candles & similar_bodies & similar_opens & opens_above_high

    return bullish_side_by_side_white_lines

# Ref: https://thepatternsite.com/ThreeLineStrikeBull.html
# This function detects the Bullish Three-Line Strike candlestick pattern.
# It requires three consecutive bullish candles followed by a bearish candle that closes below the open of the first candle.
# The function uses pandas for efficient data manipulation.


def do_detect_bullish_three_line_strike(df: pd.DataFrame, body_threshold: float = 0.2) -> pd.Series:
    """
    Detects the Bullish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        body_threshold: Minimum ratio of body size to candle range for each of the three candles.

    Returns:
        pd.Series: Boolean Series indicating Bullish Three-Line Strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_bullish_three = is_white.rolling(3).apply(lambda x: all(x), raw=True)
    
    # Calculate body size and range for each candle.
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    
    # Check if body size meets threshold condition
    body_ratio = body_size / candle_range
    is_body_significant = body_ratio >= body_threshold
    is_bullish_three_significant = is_bullish_three & is_body_significant.rolling(3).apply(lambda x: all(x), raw=True)
    
    # Check for the final bearish candle.
    fourth_candle_bearish = df['close'].shift(-3) < df['open']

    # Combine conditions.
    bullish_three_line_strike = is_bullish_three_significant.shift(3) & fourth_candle_bearish

    return bullish_three_line_strike

# Ref: https://thepatternsite.com/TriStarBull.html
# This function detects the Bullish Tri-Star candlestick pattern.
# It identifies three doji candles after a downward trend, with the middle doji having a body below the other two.
# The function takes a Pandas DataFrame with OHLCV data as input and returns a boolean Series indicating the presence of the pattern.
# Parameters:
#     df: Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.  The index must be retained.
#     doji_threshold: Maximum difference between open and close to consider a candle a doji.  Should be a fraction or percentage.
#     body_ratio_threshold: Minimum ratio of the middle doji's body size to the body sizes of the other two doji.


def do_detect_bullish_tri_star(df: pd.DataFrame, doji_threshold: float = 0.01, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Bullish Tri-Star candlestick pattern.

    Args:
        df: DataFrame with OHLCV data.
        doji_threshold: Maximum difference between open and close to consider a candle a doji (fraction of candle range).
        body_ratio_threshold: Minimum ratio of the middle doji's body size to the outer two doji's body sizes.

    Returns:
        Boolean Series indicating Bullish Tri-Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_doji = (abs(df['close'] - df['open']) / (df['high'] - df['low']) ) <= doji_threshold
    
    #Identify doji candles
    is_bullish_tri_star = (is_doji.shift(2) & is_doji.shift(1) & is_doji)

    #Check middle doji condition
    middle_doji_body = abs(df['close'].shift(1) - df['open'].shift(1))
    outer_doji_body = abs(df['close'] - df['open']) + abs(df['close'].shift(2) - df['open'].shift(2))

    is_bullish_tri_star &= (middle_doji_body / outer_doji_body) <= body_ratio_threshold


    return is_bullish_tri_star

# Ref: https://thepatternsite.com/BlkCandle.html
# Detects a black candlestick pattern.  A black candle is defined as a candle where the closing price is lower than the opening price,
# and the shadows are smaller than the body height.

def do_detect_black_candle(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects black candlestick patterns in a DataFrame.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_ratio_threshold:  Minimum ratio of body size to total candle range for the pattern to be considered valid.

    Returns:
        A pandas Series of booleans indicating the presence of a black candle pattern for each row.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    total_range = df['high'] - df['low']
    shadow_ratio = body_size / total_range
    is_valid_shadow = shadow_ratio >= body_ratio_threshold

    return is_black & is_valid_shadow


# Ref: https://thepatternsite.com/BlkCandleShort.html
# This function detects the "Short Black Candle" candlestick pattern.
# A short black candle is characterized by a small body and relatively short shadows.
# The function identifies this pattern by comparing the body size to the shadows.
# Thresholds are provided as parameters for flexibility.


def do_detect_short_black_candle(df: pd.DataFrame, body_size_threshold: float = 0.2, shadow_size_threshold: float = 0.5) -> pd.Series:
    """
    Detects the short black candle pattern.

    Args:
        df: DataFrame with OHLC data and index.
        body_size_threshold: Maximum ratio of body size to the total candle range.
        shadow_size_threshold: Maximum ratio of shadow size to the candle body size

    Returns:
        A pandas Series (boolean mask) with True for rows corresponding to short black candles,
        and False otherwise. Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]

    is_short_body = body_size / total_range <= body_size_threshold
    is_black = df["close"] < df["open"]
    is_short_shadow = (upper_shadow + lower_shadow) / body_size <= shadow_size_threshold


    return is_short_body & is_black & is_short_shadow


# Ref: https://thepatternsite.com/ShortWhiteCandle.html
# This function detects the "Short White Candle" candlestick pattern.
# A short white candle is characterized by a small body and short shadows.
# The function identifies candles where the body size is smaller than a specified threshold 
# and the shadows are shorter than the body.
# The function returns a pandas Series of booleans, indicating the presence of the pattern.

def do_detect_short_white_candle(df: pd.DataFrame, body_threshold: float = 0.01, shadow_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Short White Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns named "open", "high", "low", "close".
        body_threshold: The maximum body size (as a fraction of the candle's range) to be considered a short candle.
        shadow_threshold: The maximum ratio of shadow length to body length for a short candle

    Returns:
        pd.Series: Boolean Series indicating Short White Candle patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = df["close"] - df["open"]
    is_white = body_size > 0
    total_range = df["high"] - df["low"]
    body_ratio = body_size / total_range
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    
    is_short = body_ratio < body_threshold
    is_short_shadow = (upper_shadow / body_size) < shadow_threshold
    is_short_shadow &= (lower_shadow / body_size) < shadow_threshold

    is_short_white_candle = is_white & is_short & is_short_shadow
    return is_short_white_candle


# Ref: https://thepatternsite.com/WhiteCandle.html
# This function detects the "White Candle" pattern in a Pandas DataFrame.
# A white candle is characterized by a closing price higher than the opening price,
# with shadows shorter than the body.  The thresholds for shadow length relative to body length are configurable.
def do_detect_white_candle(df: pd.core.frame.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the White Candle pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_ratio_threshold: The maximum ratio of shadow length to candle body length to qualify as a white candle.

    Returns:
        A pandas Series of booleans indicating whether each row represents a white candle.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body = df["close"] - df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]

    is_short_shadow = (upper_shadow / body) < body_ratio_threshold
    is_short_shadow &= (lower_shadow / body) < body_ratio_threshold
    
    return is_white & is_short_shadow


# Ref: https://thepatternsite.com/CloseBlkMarubozu.html
# Detects the Closing Black Marubozu candlestick pattern.
# A closing black marubozu is a tall black candlestick with an upper shadow but no lower shadow.

def do_detect_closing_black_marubozu(df: pd.DataFrame, upper_shadow_threshold: float = 0.1, body_size_threshold: float = 0.05) -> pd.Series:
    """
    Detects Closing Black Marubozu candlestick patterns.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.  Must not be modified.
        upper_shadow_threshold: Minimum ratio of upper shadow to candle body.
        body_size_threshold: Minimum candle body size threshold as a fraction of the candle's range

    Returns:
        Boolean Series indicating Closing Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body size
    body_size = abs(df['close'] - df['open'])
    
    #Calculate the candle's range
    candle_range = df['high'] - df['low']
    
    #Calculate the upper shadow size
    upper_shadow = df['high'] - df['close']
    
    #Conditions:
    # 1. Black candle: Close < Open
    # 2. Upper shadow exists and is significant relative to the body size
    # 3. No lower shadow: Low == Open
    # 4. Body size is significant relative to the range of the candle
    is_closing_black_marubozu = (df['close'] < df['open']) & (upper_shadow / body_size >= upper_shadow_threshold) & (df['low'] == df['open']) & (body_size/candle_range >= body_size_threshold)
    

    return is_closing_black_marubozu




# Ref: https://thepatternsite.com/ClosingWhiteMarubozu.html
# Detects the Closing White Marubozu candlestick pattern.
# A closing white marubozu is a tall white candle with no upper shadow, but it does have a lower shadow.
# It acts as a continuation of the prevailing price trend.

def do_detect_closing_white_marubozu(df: pd.core.frame.DataFrame, lower_shadow_threshold: float = 0.1) -> pd.Series:
    """
    Detects Closing White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        lower_shadow_threshold: Minimum ratio of lower shadow to the candle body.

    Returns:
        pd.Series: Boolean Series indicating Closing White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']
    body = df['close'] - df['open']

    is_closing_white_marubozu = (is_white) & (upper_shadow == 0) & (lower_shadow >= body * lower_shadow_threshold)

    return is_closing_white_marubozu


# Ref: https://thepatternsite.com/CollapseDojiStar.html
# Detects the Collapsing Doji Star candlestick pattern.
# The pattern consists of three candles:
# 1. A white candle in an upward trend.
# 2. A doji that gaps below the previous candle's low.
# 3. A black candle that gaps below the doji.
# No shadows on the three candles should overlap.
# Gaps must surround the doji.

def do_detect_collapsing_doji_star(df: pd.DataFrame, doji_threshold: float = 0.001, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Collapsing Doji Star pattern in OHLCV data.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: Maximum difference between open and close to consider a candle a doji.
        gap_threshold: Minimum gap (as a fraction of the previous candle's range) required between candles.

    Returns:
        A pandas Series of booleans indicating whether a collapsing doji star pattern is detected for each row.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    high_low_range = df['high'] - df['low']
    open_close_diff = abs(df['open'] - df['close'])
    is_doji = open_close_diff <= doji_threshold * high_low_range

    # Shift columns to access previous candles
    prev_low = df['low'].shift(1)
    prev_high = df['high'].shift(1)
    prev_open = df['open'].shift(1)
    prev_close = df['close'].shift(1)
    prev_high_low_range = (df['high'].shift(1) - df['low'].shift(1))

    # Check for gaps
    gap_below_prev_low = df['low'] < prev_low - gap_threshold * prev_high_low_range
    gap_below_doji = df['low'].shift(-1) < df['low'] - gap_threshold * high_low_range

    # Identify pattern
    pattern = (is_white.shift(2) & is_doji.shift(1) & is_black & gap_below_prev_low.shift(1) & gap_below_doji)

    # Remove overlap condition is implicit in the gap condition

    return pattern

# Ref: https://thepatternsite.com/ConcealBaby.html
# This function detects the Concealing Baby Swallow candlestick pattern.
# The pattern consists of four consecutive black candles.
# The first two are long black marubozu candles.
# The third candle gaps down, but its high trades into the body of the previous day's candle.
# The fourth candle is an engulfing candle, having a higher high and a lower low than the third candle.

def do_detect_concealing_baby_swallow(df: pd.DataFrame, gap_threshold: float = 0.01, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Concealing Baby Swallow candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must include 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) required between candles.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        A Pandas Series indicating Concealing Baby Swallow patterns (True for matches).
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_marubozu = (df['high'] == df['close']) | (df['low'] == df['open'])
    is_long_body = df['close'] - df['open'] > body_ratio_threshold * (df['open'] - df['close'])

    pattern_mask = (
        is_black.shift(3) & is_long_body.shift(3) & is_marubozu.shift(3) &
        is_black.shift(2) & is_long_body.shift(2) & is_marubozu.shift(2) &
        is_black.shift(1) &
        is_black & 
        (df['open'].shift(1) - df['close'].shift(1) > gap_threshold * (df['high'].shift(1) - df['low'].shift(1))) &
        (df['high'] >= df['high'].shift(1)) & (df['low'] <= df['low'].shift(1))
    )

    return pattern_mask


# Ref: https://thepatternsite.com/DarkCloudCover.html
# Detects Dark Cloud Cover candlestick pattern.
# The Dark Cloud Cover is a bearish reversal pattern consisting of two candles.
# The first candle is a long white (bullish) candle.
# The second candle is a black (bearish) candle that opens above the high of the first candle
# and closes below the midpoint of the first candle's body.


def do_calculate_dark_cloud_cover(df: pd.core.frame.DataFrame, open_above_high_threshold: float = 1.0, close_below_midpoint_threshold: float = 0.5) -> pd.core.series.Series:
    """
    Detects Dark Cloud Cover pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        open_above_high_threshold: Minimum factor by which the second candle's open must exceed the first candle's high.
        close_below_midpoint_threshold: Minimum fraction of the first candle's body below which the second candle must close.

    Returns:
        Boolean Series indicating Dark Cloud Cover patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    first_candle_high = df['high'].shift(1)
    first_candle_low = df['low'].shift(1)
    first_candle_midpoint = (first_candle_high + first_candle_low) / 2
    first_candle_body = first_candle_high - first_candle_low

    second_candle_open_above_high = df['open'] > first_candle_high * open_above_high_threshold
    second_candle_close_below_midpoint = df['close'] < first_candle_midpoint - close_below_midpoint_threshold * first_candle_body
    second_candle_is_black = df['close'] < df['open']

    dark_cloud_cover = is_white.shift(1) & second_candle_is_black & second_candle_open_above_high & second_candle_close_below_midpoint

    return dark_cloud_cover


# Ref: https://thepatternsite.com/Deliberation.html
# The deliberation pattern consists of three white candles in an upward trend.
# The first two candles have tall bodies, while the third candle has a small body
# that opens near the second day's close.  Each candle opens and closes higher
# than the previous one.  This function detects this pattern.

def do_detect_deliberation(df: pd.DataFrame, tall_body_threshold: float = 0.02, small_body_threshold: float = 0.01, open_close_diff_threshold: float = 0.005) -> pd.Series:
    """
    Detects the Deliberation candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must have 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        tall_body_threshold: Minimum body size as percentage of the candle's range for the first two candles to be considered tall.
        small_body_threshold: Maximum body size as percentage of the candle's range for the third candle to be considered small.
        open_close_diff_threshold: Maximum allowed difference (as a fraction) between the opening price of the third candle and the closing price of the second candle.

    Returns:
        A pandas Series with True for rows corresponding to a Deliberation pattern, and False otherwise.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    candle_range = df["high"] - df["low"]
    body_size = abs(df["close"] - df["open"])
    body_ratio = body_size / candle_range
    
    #Check if last three candles are white.
    is_deliberation = (is_white.shift(2) & is_white.shift(1) & is_white)

    # Check for tall bodies in the first two candles
    is_tall_body1 = body_ratio.shift(2) > tall_body_threshold
    is_tall_body2 = body_ratio.shift(1) > tall_body_threshold
    is_deliberation &= is_tall_body1 & is_tall_body2
    
    #Check if third candle has a small body.
    is_small_body3 = body_ratio < small_body_threshold
    is_deliberation &= is_small_body3
    
    #Check if third candle's open is near second candle's close
    open_close_diff = abs(df['open'] - df['close'].shift(1)) / df['close'].shift(1)
    is_close_open = open_close_diff < open_close_diff_threshold
    is_deliberation &= is_close_open

    #Ensure that each candle opens and closes higher than previous.  
    is_higher_open = df['open'] > df['close'].shift(1)
    is_higher_close = df['close'] > df['open'].shift(1)
    is_deliberation &= is_higher_open & is_higher_close
    
    return is_deliberation


# Ref: https://thepatternsite.com/Dragonfly.html
# This function detects the Dragonfly Doji candlestick pattern.
# A Dragonfly Doji is characterized by a long lower shadow, a small body (open and close are very close), and a relatively short or nonexistent upper shadow.
# The closing price is at or near the high of the candle.  The long lower shadow suggests that there was selling pressure that was ultimately overcome by buying pressure, leading to a close near the high.

def do_detect_dragonfly_doji(df: pd.DataFrame, lower_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Dragonfly Doji candlestick patterns.

    Args:
        df: DataFrame with OHLC data.  Must have columns: "open", "high", "low", "close".
        lower_wick_threshold: Minimum ratio of lower wick length to total candle range.

    Returns:
        Boolean Series indicating Dragonfly Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the lower wick length
    lower_wick = df["close"] - df["low"]

    #Calculate the total candle range.
    total_range = df["high"] - df["low"]
    
    # Calculate the body size
    body_size = abs(df["close"] - df["open"])
    
    # Check for the Dragonfly Doji criteria
    is_dragonfly_doji = (lower_wick / total_range >= lower_wick_threshold) & (body_size / total_range <= 0.1) #The body should be small relative to the candle's range.

    return is_dragonfly_doji


# Ref: https://thepatternsite.com/GappingDownDoji.html
# The gapping down doji is characterized by a gap down from the previous candle, followed by a doji candle.
# A doji candle has nearly equal open and close prices.  The function identifies this pattern
# by checking for a gap down and then verifying that the subsequent candle is a doji within a specified threshold.
# The gap is defined as a percentage difference between the previous close and the current open.
# The doji is defined as the absolute difference between the open and close prices being below a threshold percentage of the candle's range.

def do_detect_gapping_down_doji(df: pd.core.frame.DataFrame, gap_threshold: float = 0.01, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the 'Gapping Down Doji' candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum percentage gap down from the previous candle's close to the current candle's open.
        doji_threshold: Maximum percentage difference between open and close prices to qualify as a doji.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the gap between consecutive candles.
    previous_close = df['close'].shift(1)
    gap = (df['open'] - previous_close) / previous_close
    
    #Identify doji candles
    is_doji = (df['close'] - df['open']).abs() / (df['high'] - df['low']).abs() < doji_threshold

    #Combine gap and doji conditions
    is_gapping_down_doji = (gap < -gap_threshold) & is_doji

    return is_gapping_down_doji


# Ref: https://thepatternsite.com/GappingUpDoji.html
# This function detects the "Gapping Up Doji" candlestick pattern.
# A gapping up doji is characterized by a gap up from the previous candle's close,
# followed by a doji candle (open and close prices are nearly equal).  The pattern is
# considered bearish, contrary to theoretical predictions.

def do_detect_gapping_up_doji(df: pd.DataFrame, doji_threshold: float = 0.01, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Gapping Up Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: Maximum difference between open and close for a doji.
        gap_threshold: Minimum gap size (as a fraction) between previous close and current open

    Returns:
        A Pandas Series indicating Gapping Up Doji patterns (True for each instance).

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_doji = abs(df['open'] - df['close']) / df['high'] <= doji_threshold
    previous_close = df['close'].shift(1)
    is_gap_up = (df['open'] - previous_close) / previous_close >= gap_threshold
    is_uptrend = df['close'].shift(1) > df['open'].shift(2) #Simplified uptrend check.
    
    is_gapping_up_doji = is_doji & is_gap_up & is_uptrend

    return is_gapping_up_doji


# Ref: https://thepatternsite.com/Gravestone.html
# This function detects the "Gravestone Doji" candlestick pattern.
# A Gravestone Doji is characterized by a long upper shadow, a very short or non-existent lower shadow,
# and an open and close price that are nearly identical, located near the low of the candle.

def do_detect_gravestone_doji(df: pd.DataFrame, upper_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Gravestone Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        upper_wick_threshold: Minimum ratio of upper wick length to total candle range.

    Returns:
        pd.Series: Boolean Series indicating Gravestone Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size
    body_size = abs(df["close"] - df["open"])

    # Calculate the upper wick size
    upper_wick_size = df["high"] - max(df["open"], df["close"])

    # Calculate the lower wick size
    lower_wick_size = min(df["open"], df["close"]) - df["low"]

    # Calculate total candle range
    total_range = df["high"] - df["low"]

    # Create boolean mask for Gravestone Doji conditions
    is_gravestone_doji = (
        (upper_wick_size / total_range >= upper_wick_threshold)  # Long upper wick
        & (lower_wick_size / total_range <= 0.1) # Short or no lower wick
        & (body_size / total_range <= 0.1) #Very short body
    )
    return is_gravestone_doji


# Ref: https://thepatternsite.com/LongLegDoji.html
# This function detects the "Long Legged Doji" candlestick pattern.
# A long legged doji is characterized by an opening and closing price that are very close to each other,
# with long upper and lower shadows.  The shadows do not need to be of equal length,
# only longer than recent shadows on other candles.
# The function uses several thresholds to define "long" shadows and "close" open/close values.
# The parameters allow for customization of these thresholds.

def do_detect_long_legged_doji(df: pd.DataFrame, wick_body_ratio: float = 2.0, close_open_threshold: float = 0.001) -> pd.Series:
    """
    Detects Long Legged Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data ("open", "high", "low", "close", "volume", "date" columns).
        wick_body_ratio: Minimum ratio of the sum of upper and lower wicks to the body size.
        close_open_threshold: Maximum difference between open and close price as a fraction of the price range.

    Returns:
        Boolean Series indicating Long Legged Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df["close"] - df["open"])
    upper_wick = df["high"] - max(df["open"], df["close"])
    lower_wick = min(df["open"], df["close"]) - df["low"]
    total_wick_length = upper_wick + lower_wick
    
    is_doji = abs(df["close"] - df["open"]) / (df["high"] - df["low"]) < close_open_threshold
    is_long_legged = (total_wick_length / body_size) >= wick_body_ratio

    return is_doji & is_long_legged


# Ref: https://thepatternsite.com/NorthernDoji.html
# This function detects the Northern Doji candlestick pattern.
# A Northern Doji is a doji candlestick that appears in an upward trend.
# A doji candlestick has an open and close price that are very close to each other.
# The function identifies Northern Dojis based on the proximity of the open and close prices,
# and their appearance within an upward trend.


def do_detect_northern_doji(df: pd.DataFrame, doji_threshold: float = 0.001) -> pd.Series:
    """
    Detects Northern Doji candlestick patterns.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: The maximum difference between open and close price (as a fraction of the candle's range) to be considered a doji.

    Returns:
        A pandas Series of booleans indicating whether each row represents a Northern Doji.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_uptrend = df['close'] > df['open'].shift(1)  #Check for prior uptrend
    is_doji = abs(df['close'] - df['open']) / (df['high'] - df['low']).replace(0,1) <= doji_threshold #Avoid divide by zero
    is_northern_doji = is_uptrend & is_doji

    return is_northern_doji


# Ref: https://thepatternsite.com/SouthernDoji.html
# This function detects the "Southern Doji" candlestick pattern.
# A Southern Doji is a doji candlestick appearing in a downward trend.
# It's characterized by approximately equal opening and closing prices.
# The function checks for this pattern and returns a boolean Series.

def do_detect_southern_doji(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Southern Doji candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        doji_threshold: Maximum difference between open and close for doji identification.

    Returns:
        A pandas Series indicating Southern Doji patterns (True/False).  Returns an empty Series if the dataframe is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_doji = abs(df['open'] - df['close']) / df['high'] <= doji_threshold
    is_downward_trend = df['close'] < df['open'].shift(1)

    is_southern_doji = is_doji & is_downward_trend

    return is_southern_doji

# Ref: https://thepatternsite.com/DojiStarBear.html
# Detects the Bearish Doji Star candlestick pattern.
# The pattern consists of two candles: a long white candle followed by a doji
# with a gap higher.  The doji's shadows should be relatively short.

def do_detect_bearish_doji_star(df: pd.core.frame.DataFrame, body_ratio_threshold: float = 0.1, gap_threshold: float = 0.001) -> pd.Series:
    """
    Detects the Bearish Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        body_ratio_threshold: Maximum ratio of doji body size to the previous candle's body size.
        gap_threshold: Minimum gap (as a fraction of previous candle's high) between the open of the doji and the close of the previous candle.

    Returns:
        pd.Series: Boolean Series indicating Bearish Doji Star patterns.  FIXME: Requires more robust validation of DataFrame columns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    previous_body_size = body_size.shift(1)
    previous_high = df['high'].shift(1)
    high_low_range = df['high'] - df['low']
    is_doji = high_low_range / body_size < body_ratio_threshold

    is_gap = (df['open'] - previous_high) / previous_high > gap_threshold

    bearish_doji_star = is_white.shift(1) & is_doji & is_gap & (body_size/previous_body_size < body_ratio_threshold)


    return bearish_doji_star


# Ref: https://thepatternsite.com/DojiStarBull.html
# Detects the bullish doji star pattern.
# A bullish doji star consists of a tall black candle followed by a doji that gaps below the prior candle's body.
# The doji's shadows should not be unusually long.  The shadows may overlap.

def do_detect_bullish_doji_star(df: pd.DataFrame, doji_body_threshold: float = 0.01, shadow_threshold: float = 0.7) -> pd.Series:
    """
    Detects the bullish doji star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns: "open", "high", "low", "close", "volume", "date".
        doji_body_threshold: Maximum difference (as fraction of the candle range) between open and close price for a candle to be considered a doji.
        shadow_threshold: The maximum ratio of shadow length relative to the body size for the doji.

    Returns:
        A pandas Series (boolean mask) indicating bullish doji star patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_tall = (df["open"] - df["close"]) / (df["high"] - df["low"]) > 0.7  # Consider this a tall candle if this ratio exceeds 0.7
    is_doji = abs(df["close"] - df["open"]) / (df["high"] - df["low"]) <= doji_body_threshold  # Consider this a doji if this ratio is below 0.01
    
    #Calculate shadow length for doji
    upper_shadow_doji = df["high"] - max(df["open"],df["close"])
    lower_shadow_doji = min(df["open"],df["close"]) - df["low"]
    total_shadow_doji = upper_shadow_doji + lower_shadow_doji
    body_size_doji = abs(df["close"] - df["open"])

    is_valid_doji_shadow = (total_shadow_doji / body_size_doji) <= shadow_threshold #Condition to check for shadows not being too long

    #Shift is_black and is_tall for comparison with the doji.
    shifted_is_black = is_black.shift(1)
    shifted_is_tall = is_tall.shift(1)

    # Check for gap below the previous candle's body.
    is_gap = df["low"] < df["open"].shift(1)

    # Combine conditions to detect bullish doji star.
    bullish_doji_star = (shifted_is_black & shifted_is_tall & is_doji & is_gap & is_valid_doji_shadow)[1:]

    return bullish_doji_star


# Ref: https://thepatternsite.com/CollapseDojiStar.html
# Detects the "Collapsing Doji Star" candlestick pattern.
# This pattern consists of three candles:
# 1. A white candle in an upward trend.
# 2. A doji that gaps below the previous candle's low.
# 3. A black candle that gaps below the doji's low.
# No shadows should overlap.

def do_detect_collapsing_doji_star(df: pd.DataFrame, doji_threshold: float = 0.01, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Collapsing Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.  Must have columns: "open", "high", "low", "close", "volume", "date".
        doji_threshold: Maximum difference between open and close to consider it a doji (as a fraction of the range).
        gap_threshold: Minimum gap between consecutive candles (as a fraction of the previous candle's range).

    Returns:
        A pandas Series (boolean mask) indicating Collapsing Doji Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_black = df["close"] < df["open"]
    range_ = df["high"] - df["low"]
    body = abs(df["close"] - df["open"])
    is_doji = body <= range_ * doji_threshold
    
    # Calculate gaps
    gaps = range_.shift(1) * gap_threshold
    gap_below_prev_low = df["low"] < df["low"].shift(1) - gaps.shift(1)
    gap_below_doji = df["low"] < df["low"].shift(1) - gaps.shift(1)


    # Apply pattern detection logic
    pattern = (
        is_white.shift(2)
        & is_doji.shift(1)
        & gap_below_prev_low.shift(1)
        & is_black
        & gap_below_doji
    )

    return pattern


# Ref: https://thepatternsite.com/EveningDojiStar.html
# This function detects the "Evening Doji Star" candlestick pattern.
# It checks for a tall white candle followed by a doji that gaps up,
# and then a tall black candle closing below the midpoint of the first candle.

def do_detect_evening_doji_star(df: pd.DataFrame, doji_body_threshold: float = 0.01, body_ratio_threshold: float = 0.1) -> pd.Series:
    """
    Detects the Evening Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        doji_body_threshold: Maximum difference between open and close for a candle to be considered a doji.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body = abs(df['close'] - df['open'])
    is_doji = body <= (df['high'] - df['low']) * doji_body_threshold
    
    # Calculate the midpoint of the body of the first candle
    midpoint = (df['open'].shift(2) + df['close'].shift(2)) / 2

    #Check for upward gap
    is_upward_gap = df['open'].shift(1) > df['close'].shift(2)


    # Check for the pattern
    pattern = (is_white.shift(2) & is_doji.shift(1) & is_black & is_upward_gap & (df['close'] < midpoint) & (body.shift(2) >= body.shift(1) * body_ratio_threshold) )

    return pattern

# Ref: https://thepatternsite.com/DownGap3Methods.html
# This function detects the "Downside Gap Three Methods" candlestick pattern.
# It checks for three candles: two long black candles with a gap between them,
# followed by a white candle that closes within the body of the first candle.
# The function returns a pandas Series of boolean values, indicating the presence
# of the pattern for each row in the input DataFrame.

def do_detect_downside_gap_three_methods(df: pd.DataFrame, gap_threshold: float = 0.01, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Downside Gap Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must have columns 'open', 'high', 'low', 'close', 'volume', and 'date'.
        gap_threshold: Minimum gap between consecutive candles as a percentage of the previous candle's range.
        body_ratio_threshold: Minimum ratio of the third candle's body size to the first candle's body size for the pattern to be considered a match.

    Returns:
        A pandas Series of booleans indicating the presence of the Downside Gap Three Methods pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    candle_body = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    gap = df['open'].shift(-1) - df['close']

    #Detect the gap.  The gap must be at least gap_threshold * previous range
    is_gap = (gap > gap_threshold * candle_range.shift(-1))

    #Check for first and second candle. Note that because we are checking the gap *between* candles,
    #the shift is always forward (i.e. never negative).
    first_black = is_black.shift(2) & (candle_body.shift(2) > body_ratio_threshold * candle_range.shift(2))
    second_black = is_black.shift(1) & (candle_body.shift(1) > body_ratio_threshold * candle_range.shift(1))
    
    #Check for third candle
    third_white = is_white & (df['open'] < df['open'].shift(1)) & (df['open'] > df['close'].shift(1)) & (df['close'] < df['open'].shift(2))

    # Combine conditions
    pattern = first_black & second_black & is_gap.shift(1) & third_white

    return pattern


# Ref: https://thepatternsite.com/DownsideTasukiGap.html
# The downside Tasuki gap is a three-candle pattern identified in a downward trend.
# It consists of a black candle followed by another black candle that gaps lower,
# with no shadow overlap. The third candle is white, opening within the body of the
# second candle and closing within the gap between the first and second candles.


def do_detect_downside_tasuki_gap(df: pd.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Downside Tasuki Gap candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close', 'volume', 'date'.
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) required between candles.

    Returns:
        pd.Series: Boolean Series indicating Downside Tasuki Gap patterns.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    previous_close = df["close"].shift(1)
    high_low_range = df["high"] - df["low"]
    gap = df["open"] - previous_close

    #Identify the three candles
    first_black = is_black & (df["low"].shift(2) > df["low"].shift(1))  # First black candle in downtrend
    second_black = is_black & (gap > gap_threshold * high_low_range.shift(1)) & (df["high"].shift(1) < df["open"])
    third_white = is_white & (df["open"] >= df["open"].shift(1)) & (df["close"] <= df["open"].shift(1))

    #Combine criteria
    pattern = first_black & second_black.shift(1) & third_white.shift(2)

    return pattern


# Ref: https://thepatternsite.com/Dragonfly.html
# This function detects the Dragonfly Doji candlestick pattern.
# A Dragonfly Doji is characterized by a long lower shadow, a small body (open and close are very close), and a very short or non-existent upper shadow.
# The closing price is near the high.  The function returns a boolean Series indicating the presence of the pattern.

def do_detect_dragonfly_doji(df: pd.DataFrame, lower_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Dragonfly Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must have 'open', 'high', 'low', 'close' columns.
        lower_wick_threshold: Minimum ratio of lower wick length to total candle range.

    Returns:
        pd.Series: Boolean Series indicating Dragonfly Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_doji = abs(df["close"] - df["open"]) / (df["high"] - df["low"]) < 0.1  # Consider it a doji if the body is small relative to the total range.
    lower_wick_length = df["open"] - df["low"]
    total_candle_range = df["high"] - df["low"]
    long_lower_wick = lower_wick_length / total_candle_range >= lower_wick_threshold
    
    is_dragonfly_doji = is_doji & long_lower_wick

    return is_dragonfly_doji

# Ref: https://thepatternsite.com/BearEngulfing.html
# This function detects the "Bearish Engulfing" candlestick pattern.
# It identifies a two-candle pattern where the second candle's body completely
# engulfs the first candle's body, in an upward trending context.
# The function is designed to work with a pandas DataFrame containing OHLC data.
# The DataFrame is assumed to have 'open', 'high', 'low', 'close', 'volume', and 'date' columns.

def do_detect_bearish_engulfing(df: pd.DataFrame, body_overlap_threshold: float = 0.0) -> pd.Series:
    """
    Detects the Bearish Engulfing candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        body_overlap_threshold: Minimum overlap of bodies required to identify engulfing pattern.

    Returns:
        A pandas Series of booleans indicating Bearish Engulfing patterns.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size_1 = abs(df['close'] - df['open'])
    body_size_2 = abs(df['close'].shift(-1) - df['open'].shift(-1))

    # Check for two-candle pattern where the first is white and the second is black
    pattern_mask = (is_white & is_black.shift(-1))
    
    # Check for engulfing condition: second candle's body completely overlaps first
    engulfing_mask = (df['open'].shift(-1) < df['close'] ) & (df['close'].shift(-1) < df['open'])

    # Combine conditions and threshold check
    bearish_engulfing = pattern_mask & engulfing_mask

    return bearish_engulfing

# Ref: https://thepatternsite.com/BullEngulfing.html
# Detects a bullish engulfing pattern.
# The first candle is black (close < open).
# The second candle is white (close > open) and taller than the first candle.
# The white candle's close is above the first candle's open, and the white candle's open is below the first candle's close.

def do_detect_bullish_engulfing(df: pd.DataFrame, body_ratio_threshold: float = 1.0) -> pd.Series:
    """
    Detects bullish engulfing candlestick patterns.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        A pandas Series with boolean values indicating bullish engulfing patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size1 = abs(df['close'] - df['open'])
    body_size2 = abs(df['close'].shift(-1) - df['open'].shift(-1))
    
    condition1 = is_black
    condition2 = is_white.shift(-1)
    condition3 = body_size2 >= body_ratio_threshold * body_size1
    condition4 = df['close'].shift(-1) > df['open']
    condition5 = df['open'].shift(-1) < df['close']

    result = condition1 & condition2 & condition3 & condition4 & condition5
    return result

# Ref: https://thepatternsite.com/EveningDojiStar.html
# The following function detects the Evening Doji Star candlestick pattern.
# It takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating the presence of the pattern for each row.
# The function considers three consecutive candles:
# 1. A tall white candle (previous day): closing price significantly higher than opening price
# 2. A doji candle (current day): open and close are approximately equal
# 3. A tall black candle (next day): closing price significantly lower than opening price
# The doji's body should gap above the bodies of the surrounding candles.

def do_detect_evening_doji_star(df: pd.DataFrame, doji_threshold: float = 0.01, tall_candle_threshold: float = 0.03) -> pd.Series:
    """
    Detects the Evening Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close').
        doji_threshold: Maximum difference between open and close for a doji candle (as a fraction of the candle's range).
        tall_candle_threshold: Minimum body size (as a fraction of the candle's range) for a tall candle.


    Returns:
        Pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    # Calculate candle body sizes
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    # Identify tall white and black candles
    is_tall_white = (df['close'] - df['open']) / candle_range > tall_candle_threshold
    is_tall_black = (df['open'] - df['close']) / candle_range > tall_candle_threshold

    # Identify Doji candles
    doji = abs(df['close'] - df['open']) / candle_range < doji_threshold

    # Shift data to align with three-candle pattern
    prev_day_is_tall_white = is_tall_white.shift(1)
    next_day_is_tall_black = is_tall_black.shift(-1)


    #Check for gap between doji and previous/next candles
    prev_close = df['close'].shift(1)
    next_open = df['open'].shift(-1)
    gap_up_prev = df['open'] > prev_close
    gap_down_next = df['close'] < next_open


    #Combine conditions
    evening_doji_star = (prev_day_is_tall_white) & (doji) & (next_day_is_tall_black) & (gap_up_prev) & (gap_down_next)

    return evening_doji_star


# Ref: https://thepatternsite.com/EveningStar.html
# Detects the Evening Star candlestick pattern.
# The Evening Star is a three-candle bearish reversal pattern.
# It consists of a tall white candle, followed by a small candle (any color),
# and then a tall black candle that gaps below the second candle and closes
# at least midway down the first candle.
# The small candle should ideally gap above the bodies of the two adjacent candles.


def do_detect_evening_star(df: pd.DataFrame, body_threshold: float = 0.5, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects Evening Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        body_threshold: Minimum ratio of the third candle's body size to the first candle's body size.
        gap_threshold: Minimum gap between the second candle's open and the first candle's close (as a fraction of the first candle's range).


    Returns:
        Boolean Series indicating Evening Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    candle_body = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    first_candle_body = candle_body.shift(2)
    second_candle_body = candle_body.shift(1)
    third_candle_body = candle_body

    # Check for tall white candle (first candle)
    tall_white = is_white.shift(2) & (candle_range.shift(2) > 0) # Added check to handle zero range
    
    # Check for small body candle (second candle)
    small_body = (second_candle_body / first_candle_body.shift(1) < 0.5) & (second_candle_body > 0)
    gap_above = (df['open'].shift(1) > df['close'].shift(2) + gap_threshold * candle_range.shift(2))
    
    # Check for tall black candle (third candle)
    tall_black = is_black & (third_candle_body / first_candle_body > body_threshold)
    gap_below = (df['open'] < df['close'].shift(1) - gap_threshold * candle_range.shift(1)) #Added check to handle zero range
    midway_down = df['close'] < df['open'].shift(2) - 0.5 * first_candle_body

    evening_star = tall_white & gap_above & small_body & tall_black & gap_below & midway_down
    return evening_star


# Ref: https://thepatternsite.com/eventpatterns.html
# This function detects event patterns based on the provided criteria.  It identifies several types of event patterns
# described in the linked resource.  The specific logic for each pattern is implemented based on the descriptions found in the
# linked HTML.  Additional parameters may be added to provide more flexible detection criteria.  The function returns a Series
# indicating the presence of any detected event patterns in the input data.

def do_detect_event_patterns(df: pd.DataFrame, earnings_surprise_threshold: float = 0.05, stock_split_threshold: float = 0.1, rating_upgrade_threshold: float = 0.02, rating_downgrade_threshold: float = -0.02) -> pd.Series:
    """
    Detects various event patterns in OHLCV data.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date') columns.  Index must be datetime.
        earnings_surprise_threshold: Percentage change in earnings to consider a surprise.
        stock_split_threshold: Percentage change in price to consider a split.
        rating_upgrade_threshold: Percentage change to consider an upgrade.
        rating_downgrade_threshold: Percentage change to consider a downgrade.

    Returns:
        pd.Series: Boolean Series indicating the presence of event patterns.  Returns an empty series if the input is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Placeholder for earnings surprise detection (requires additional data)
    is_earnings_surprise = pd.Series(False, index=df.index)

    # Placeholder for stock split detection (requires additional data)
    is_stock_split = pd.Series(False, index=df.index)
    
    # Placeholder for rating upgrade detection (requires additional data)
    is_rating_upgrade = pd.Series(False, index=df.index)

    # Placeholder for rating downgrade detection (requires additional data)
    is_rating_downgrade = pd.Series(False, index=df.index)


    # Combine all event pattern detections
    is_event_pattern = is_earnings_surprise | is_stock_split | is_rating_upgrade | is_rating_downgrade

    return is_event_pattern


# Ref: https://thepatternsite.com/Falling3Methods.html
# This function detects the "Falling Three Methods" candlestick pattern.
# The pattern consists of five candles:
# 1. A long black candle.
# 2. Three smaller candles (mostly white, middle one can be either), contained within the first candle's high-low range.
# 3. Another long black candle closing below the first candle's close.

def do_detect_falling_three_methods(df: pd.DataFrame, body_ratio_threshold: float = 0.2, range_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Falling Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns. Index must be datetime-like.
        body_ratio_threshold: Minimum ratio of body size to candle range for the first and last candles to be considered "tall".
        range_threshold: Maximum fraction of the first candle's range by which the middle three candles may vary.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.  Returns an empty series if df is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    candle_body_size = abs(df['close'] - df['open'])
    candle_range_size = df['high'] - df['low']
    
    is_tall = candle_body_size / candle_range_size > body_ratio_threshold

    first_candle_is_tall = is_tall.shift(4)
    first_candle_is_black = is_black.shift(4)
    
    last_candle_is_tall = is_tall
    last_candle_is_black = is_black
    
    is_middle_contained = (df['high'] <= df['high'].shift(4)) & (df['low'] >= df['low'].shift(4))
    
    # FIXME: Needs additional logic to properly handle edge cases and refine the pattern detection.

    pattern_detected = (first_candle_is_tall) & (first_candle_is_black) & \
                       (is_middle_contained) & \
                       (last_candle_is_tall) & (last_candle_is_black) & \
                       (df['close'] < df['open'].shift(4))
    
    return pattern_detected

# Ref: https://thepatternsite.com/FallingWindow.html
# This function detects the "Falling Window" candlestick pattern.
# A falling window is characterized by a gap in a downward trend where yesterday's low is above today's high.

def do_detect_falling_window(df: pd.core.frame.DataFrame, gap_threshold: float = 0.0) -> pd.Series:
    """
    Detects the Falling Window candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum percentage gap between consecutive days to be considered a falling window.

    Returns:
        A pandas Series of booleans indicating the presence of a falling window pattern.  Returns an empty Series if the DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    yesterday_low = df['low'].shift(1)
    today_high = df['high']
    is_falling_window = yesterday_low > today_high
    
    return is_falling_window

# Ref: https://thepatternsite.com/GappingDownDoji.html
# This function detects the "Gapping Down Doji" candlestick pattern.
# A gapping down doji is a doji candlestick that opens lower than the previous day's close.
# The function identifies this pattern by comparing the open, close, and previous day's close prices.
# A doji is defined as a candle where the open and close prices are very close to each other.
# The function returns a pandas Series with boolean values indicating the presence of the pattern for each day.

def do_detect_gapping_down_doji(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Gapping Down Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data and 'date' column.
        doji_threshold: Maximum difference between open and close for a doji (as a fraction of the candle range).

    Returns:
        pd.Series: Boolean Series indicating Gapping Down Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size as a fraction of the range
    body_size = abs(df["open"] - df["close"]) / (df["high"] - df["low"])
    
    # Identify dojis
    is_doji = body_size <= doji_threshold
    
    # Check for gap down from previous day's close
    previous_close = df["close"].shift(1)
    is_gap_down = df["open"] < previous_close

    # Combine conditions to detect gapping down doji
    is_gapping_down_doji = is_doji & is_gap_down

    return is_gapping_down_doji

# Ref: https://thepatternsite.com/GappingUpDoji.html
# This function detects the "Gapping Up Doji" candlestick pattern.
# A gapping up doji is characterized by a gap up in price followed by a doji candlestick,
# where the open and close prices are very close to each other.  This is a bearish
# reversal pattern according to Bulkowski's research.
# The function takes a Pandas DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a Pandas Series of booleans indicating whether a gapping up doji pattern is present for each row.

def do_detect_gapping_up_doji(df: pd.DataFrame, doji_threshold: float = 0.005, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Gapping Up Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        doji_threshold: Maximum difference between open and close prices (as a fraction of the average price) to be considered a doji.
        gap_threshold: Minimum gap size as a percentage of the previous candle's range.

    Returns:
        pd.Series: Boolean Series indicating Gapping Up Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the average price for each candle
    avg_price = (df['open'] + df['close']) / 2

    # Determine doji candles
    is_doji = abs(df['close'] - df['open']) / avg_price <= doji_threshold

    # Calculate the previous candle's range
    prev_range = df['high'].shift(1) - df['low'].shift(1)

    # Calculate the gap between candles
    gap = df['open'] - df['close'].shift(1)

    # Check for upward gaps exceeding a certain threshold
    is_gap_up = gap > prev_range.shift(1) * gap_threshold

    # Combine the conditions to identify the pattern
    is_gapping_up_doji = is_doji & is_gap_up

    return is_gapping_up_doji

# Ref: https://thepatternsite.com/Gravestone.html
# This function detects the Gravestone Doji candlestick pattern.
# A Gravestone Doji has a long upper shadow and a very short or no lower shadow,
# with the open and close prices being nearly identical, near the low of the day.

def do_calculate_gravestone_doji(df: pd.DataFrame, upper_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects Gravestone Doji pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns "open", "high", "low", "close".
        upper_wick_threshold: Minimum ratio of upper wick length to total candle range.

    Returns:
        pd.Series: Boolean Series indicating Gravestone Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the upper and lower wick lengths
    upper_wick_length = df['high'] - df['close']
    lower_wick_length = df['open'] - df['low']
    total_range = df['high'] - df['low']
    
    #Check for division by zero
    total_range = total_range.replace(0,1)

    # Calculate the ratio of the upper wick length to the total range
    upper_wick_ratio = upper_wick_length / total_range

    # Define conditions for Gravestone Doji
    is_gravestone_doji = (upper_wick_ratio >= upper_wick_threshold) & (lower_wick_length <= 0.01 * total_range) & (abs(df['open'] - df['close']) <= 0.01 * total_range)

    return is_gravestone_doji


# Ref: https://thepatternsite.com/HikkakeBear.html
# This function detects the bearish hikkake candlestick pattern.
# It identifies an inside day followed by a higher high and higher low, confirmed by a close below the inside day's low.

def do_detect_bearish_hikkake(df: pd.DataFrame, inside_day_threshold: float = 0.01, confirmation_threshold: int = 3) -> pd.Series:
    """
    Detects the bearish hikkake candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.  Must have at least 3 rows.
        inside_day_threshold:  The minimum percentage change needed for a day to be identified as an inside day (optional).
        confirmation_threshold: The maximum number of days it takes for the pattern to be confirmed (optional).


    Returns:
        A pandas Series (boolean mask) indicating bearish hikkake patterns.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_inside_day = (df['high'].shift(1) > df['high']) & (df['low'].shift(1) < df['low']) & (abs((df['high'] - df['low']) / df['high'].shift(1)) < inside_day_threshold)
    
    higher_high = df['high'] > df['high'].shift(1)
    higher_low = df['low'] > df['low'].shift(1)

    hikkake_candidate = is_inside_day & higher_high & higher_low
    
    confirmation = df['low'].rolling(confirmation_threshold +1).min().shift(-confirmation_threshold) < df['low'].shift(-1)
    
    bearish_hikkake = hikkake_candidate & confirmation


    return bearish_hikkake


# Ref: https://thepatternsite.com/HikkakeBull.html
# This function detects the bullish hikkake candlestick pattern.
# A bullish hikkake consists of three candles:
# 1. An inside day (lower high and higher low than the previous day).
# 2. A candle with a lower high and lower low than the inside day.
# 3. Confirmation: Price closes above the high of the inside day within three days.

def do_detect_bullish_hikkake(df: pd.DataFrame, inside_day_threshold: float = 0.0, confirmation_threshold: int = 3) -> pd.Series:
    """
    Detects the bullish hikkake candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        inside_day_threshold: Minimum percentage change required to be considered an inside day.
        confirmation_threshold: Number of days to check for confirmation (closing above inside day high).

    Returns:
        A pandas Series indicating bullish hikkake patterns (True/False).  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_inside = (df['high'].shift(1) > df['high']) & (df['low'].shift(1) < df['low'])
    is_second = (df['high'].shift(1) > df['high']) & (df['low'].shift(1) > df['low'])
    inside_day_high = df['high'][is_inside].shift(1)


    bullish_hikkake = is_inside & is_second

    #Check for confirmation within confirmation_threshold days
    for i in range(1, confirmation_threshold + 1):
        bullish_hikkake |= (bullish_hikkake.shift(i) & (df['close'].shift(i-1) > inside_day_high.shift(i)))

    return bullish_hikkake


# Ref: https://thepatternsite.com/Hammer.html
# This function detects the Hammer candlestick pattern.
# A hammer is characterized by a small body with a long lower shadow,
# and little or no upper shadow, appearing in a downward trend.
# The long lower shadow suggests buyers stepped in to prevent further decline.

def do_detect_hammer(df: pd.DataFrame, hammer_body_factor: float = 0.2, hammer_wick_factor: float = 2.0) -> pd.Series:
    """
    Detects Hammer candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include 'open', 'high', 'low', 'close' columns.
        hammer_body_factor: Factor determining the maximum body size relative to the lower wick.
        hammer_wick_factor: Factor determining the minimum lower wick length relative to the body size.

    Returns:
        Boolean Series indicating hammer patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df['close'] - df['open'])
    lower_wick = df['low'] - min(df['open'], df['close'])
    upper_wick = max(df['open'], df['close']) - df['high']

    is_hammer = (lower_wick >= hammer_body_factor * body_size) & (lower_wick >= hammer_wick_factor * body_size) & (upper_wick <= body_size)

    return is_hammer

# Ref: https://thepatternsite.com/HammerInv.html
# Detects Inverted Hammer candlestick pattern.
# The inverted hammer is a two-line candle pattern where the first candle is tall and black,
# and the second candle is short with a small or no lower shadow and a tall upper shadow.
# The second candle's open must be below the first candle's close.

def do_calculate_inverted_hammer(df: pd.DataFrame, body_threshold: float = 0.2, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects Inverted Hammer candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_threshold: Minimum ratio of body size to candle range for the first candle.
        wick_body_ratio: Minimum ratio of upper wick length to body size for the second candle.

    Returns:
        pd.Series: Boolean Series indicating Inverted Hammer patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    first_candle_body_ratio = body_size.shift(1) / candle_range.shift(1)
    is_first_candle_black_and_tall = is_black.shift(1) & (first_candle_body_ratio > body_threshold)
    
    upper_shadow_second = df["high"] - max(df["open"],df["close"])
    body_size_second = abs(df["close"] - df["open"])
    upper_wick_body_ratio = upper_shadow_second / body_size_second
    is_second_candle_short = upper_wick_body_ratio > wick_body_ratio
    is_second_candle_open_below_first_close = df["open"] < df["close"].shift(1)
    
    inverted_hammer = is_first_candle_black_and_tall & is_second_candle_short & is_second_candle_open_below_first_close

    return inverted_hammer


# Ref: https://thepatternsite.com/HangingMan.html
# The Hanging Man candlestick pattern is characterized by a small real body near the high of the candle,
# with a long lower shadow, appearing after an uptrend.  This function detects this pattern.
# The parameters control the thresholds for determining "small body" and "long lower shadow".

def do_detect_hanging_man(df: pd.core.frame.DataFrame, body_ratio_threshold: float = 0.2, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects the Hanging Man candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        body_ratio_threshold: Maximum ratio of the real body size to the total candle range.
        wick_body_ratio: Minimum ratio of the lower wick length to the real body size.

    Returns:
        pd.Series: Boolean Series indicating Hanging Man patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    real_body = abs(df["close"] - df["open"])
    lower_wick = df["low"] - min(df["open"], df["close"])
    total_range = df["high"] - df["low"]
    is_uptrend = df["close"].shift(1) > df["open"].shift(1)  # Check if previous candle was a bullish candle


    is_small_body = real_body / total_range <= body_ratio_threshold
    is_long_lower_wick = lower_wick / real_body >= wick_body_ratio

    hanging_man = is_uptrend & is_small_body & is_long_lower_wick

    return hanging_man


# Ref: https://thepatternsite.com/HaramiBear.html
# Detects the Bearish Harami candlestick pattern.
# The pattern consists of a tall white candle followed by a small black candle,
# where the opening and closing prices of the second candle are within the body of the first candle.
# The bodies of the two candles must have different prices.

def do_calculate_bearish_harami(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects Bearish Harami pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Bearish Harami patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    
    first_candle_body_size = body_size.shift(1)
    second_candle_body_size = body_size

    is_bearish_harami = (
        (df['open'].shift(1) < df['close'].shift(1)) & #First candle is white.
        (df['open'] > df['close']) & #Second candle is black.
        (df['open'] <= df['close'].shift(1)) & #Open of second candle within body of first.
        (df['close'] >= df['open'].shift(1)) & #Close of second candle within body of first.
        (second_candle_body_size <= first_candle_body_size * body_ratio_threshold) & #Second candle's body is smaller.
        (df['open'] != df['close'].shift(1)) | (df['close'] != df['open'].shift(1))  # Ensure different body prices
    )
    return is_bearish_harami


# Ref: https://thepatternsite.com/HaramiBull.html
# This function detects the bullish harami candlestick pattern.
# A bullish harami consists of two candles: a long black candle followed by a smaller white candle whose body is entirely contained within the body of the black candle.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans indicating whether a bullish harami pattern is present for each row.

def do_detect_bullish_harami(df: pd.core.frame.DataFrame, body_ratio_threshold: float = 0.5) -> pd.core.series.Series:
    """
    Detects the bullish harami candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        pd.Series: Boolean Series indicating Bullish Harami patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    
    body_size_1 = abs(df['close'] - df['open']).shift(1)
    body_size_2 = abs(df['close'] - df['open'])

    # Condition to check if the white candle is smaller than the black candle and its body is contained
    condition = (is_black.shift(1)) & (is_white) & (body_size_2 < body_size_1.shift(1)) & (body_size_2 / body_size_1.shift(1) <= body_ratio_threshold) & (df['open'].shift(1) > df['close']) & (df['open'] < df['close'].shift(1))


    return condition


# Ref: https://thepatternsite.com/HaramiCrossBear.html
# Detects the Bearish Harami Cross candlestick pattern.
# The pattern consists of a long white candle followed by a doji candle
# whose high and low fall within the range of the first candle.

def do_detect_bearish_harami_cross(df: pd.DataFrame, doji_threshold: float = 0.001) -> pd.Series:
    """
    Detects the Bearish Harami Cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must have columns 'open', 'high', 'low', 'close', 'volume', 'date'.
        doji_threshold: Maximum difference (as a fraction) between open and close for a candle to be considered a doji.

    Returns:
        Boolean Series indicating bearish Harami Cross patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_doji = abs(df['close'] - df['open']) / df['high'] < doji_threshold
    
    prior_high = df['high'].shift(1)
    prior_low = df['low'].shift(1)
    
    within_range = (df['high'] < prior_high) & (df['low'] > prior_low)
    
    is_bearish_harami_cross = (is_white.shift(1)) & is_doji & within_range

    return is_bearish_harami_cross


# Ref: https://thepatternsite.com/HaramiCrossBull.html
# Detects the bullish harami cross candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a doji that fits within the high-low range of the previous day.
# A doji is a candle where the opening and closing prices are very close.
# This function checks for this specific configuration and returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_harami_cross(df: pd.DataFrame, doji_threshold: float = 0.01) -> pd.Series:
    """
    Detects the bullish harami cross candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: Maximum difference between open and close prices (as a fraction of the candle range) for a candle to be considered a doji.

    Returns:
        A pandas Series of booleans indicating where the pattern occurs.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_doji = abs(df['close'] - df['open']) / (df['high'] - df['low']) < doji_threshold

    is_bullish_harami_cross = (is_black.shift(1) & is_doji &
                              (df['high'] < df['high'].shift(1)) &
                              (df['low'] > df['low'].shift(1)) &
                              (df['open'] > df['close'].shift(1)) &
                              (df['close'] < df['open'].shift(1))
                              )
                              
    return is_bullish_harami_cross


# Ref: https://thepatternsite.com/HighWave.html
# The high wave candlestick pattern is characterized by tall upper and lower shadows with a small body. The body is not a doji, meaning the opening and closing prices are noticeably different.  This function detects this pattern.

def do_detect_high_wave(df: pd.DataFrame, body_ratio_threshold: float = 0.1, wick_ratio_threshold: float = 2.0) -> pd.Series:
    """
    Detects the High Wave candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Maximum ratio of body size to total candle range.
        wick_ratio_threshold: Minimum ratio of the sum of upper and lower wicks to the body size.

    Returns:
        A pandas Series (boolean mask) indicating High Wave patterns.
        FIXME: This function may need adjustment for edge cases and parameter tuning.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    upper_wick = df["high"] - max(df["open"], df["close"])
    lower_wick = min(df["open"], df["close"]) - df["low"]

    is_high_wave = (body_size / total_range <= body_ratio_threshold) & \
                    ((upper_wick + lower_wick) / body_size >= wick_ratio_threshold) & \
                    (body_size > 0)  # Ensure body is not a doji


    return is_high_wave


# Ref: https://thepatternsite.com/HomingPigeon.html
# This function detects the "Homing Pigeon" candlestick pattern.
# The pattern consists of two candles:
# 1. A tall black candle (close < open).
# 2. A smaller black candle (close < open) whose high and low are contained within the body of the first candle.
# A downward breakout occurs when the second candle's close is below the first candle's low.

def do_detect_homing_pigeon(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Homing Pigeon candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        body_ratio_threshold: Minimum ratio of the second candle's body to the first candle's body.

    Returns:
        A pandas Series of booleans indicating whether a Homing Pigeon pattern is present.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black_1 = df['close'].shift(1) < df['open'].shift(1)
    is_black_2 = df['close'] < df['open']
    
    # Check body sizes and containment
    body_size_1 = abs(df['close'].shift(1) - df['open'].shift(1))
    body_size_2 = abs(df['close'] - df['open'])
    body_ratio = body_size_2 / body_size_1
    is_contained = (df['high'] < df['open'].shift(1)) & (df['high'] > df['close'].shift(1)) & (df['low'] > df['close'].shift(1)) & (df['low'] < df['open'].shift(1))


    is_homing_pigeon = is_black_1 & is_black_2 & (body_ratio <= body_ratio_threshold) & is_contained

    return is_homing_pigeon


# Ref: https://thepatternsite.com/Identical3Crows.html
# This function detects the "Identical Three Crows" candlestick pattern.
# It identifies three consecutive tall black candles where the last two open near the previous candle's close.  
# The size similarity criterion is relaxed to improve detection rate.


def do_detect_identical_three_crows(df: pd.DataFrame, body_threshold: float = 0.2) -> pd.Series:
    """
    Detects Identical Three Crows candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        body_threshold: Minimum ratio of body size to candle range.

    Returns:
        Boolean Series indicating Identical Three Crows patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    body_ratio = body_size / candle_range
    is_tall_black = is_black & (body_ratio >= body_threshold)

    is_pattern = (is_tall_black & is_tall_black.shift(1) & is_tall_black.shift(2))
    
    # Check for opening near the previous close.  This check is relaxed compared to stricter definitions.
    open_near_close1 = abs(df['open'].shift(-1) - df['close']) < candle_range.shift(-1)*0.2
    open_near_close2 = abs(df['open'].shift(-2) - df['close'].shift(-1)) < candle_range.shift(-2)*0.2
    
    is_pattern = is_pattern & open_near_close1 & open_near_close2

    return is_pattern

# Ref: https://thepatternsite.com/InNeck.html
# Detects the "In Neck" candlestick pattern.
# The In Neck pattern consists of two candles: a tall black candle followed by a white candle that opens lower than the black candle's low,
# but closes near the black candle's close. The white candle's close should be inside the body of the black candle, but not by much.

def do_calculate_in_neck(df: pd.DataFrame, close_distance_threshold: float = 0.1) -> pd.Series:
    """
    Detects the In Neck candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date as columns.
        close_distance_threshold: The maximum distance (as a fraction of the black candle body) between the white candle's close and the black candle's close.

    Returns:
        A pandas Series of booleans indicating the presence of the In Neck pattern.  Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    black_candle_body = abs(df['open'] - df['close'])
    close_distance = abs(df['close'].shift(-1) - df['close'])

    in_neck = (is_black & is_white.shift(-1) &
               (df['open'].shift(-1) < df['low']) &
               (close_distance <= black_candle_body * close_distance_threshold))

    return in_neck


# Ref: https://thepatternsite.com/HammerInv.html
# Detects Inverted Hammer candlestick pattern.  The first candle is tall and black, 
# followed by a short candle of any color. The second candle's open must be below the 
# first candle's close. The second candle cannot be a doji (opening and closing prices 
# must be sufficiently different).

def do_calculate_inverted_hammer(df: pd.DataFrame, body_threshold: float = 0.2, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects Inverted Hammer candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_threshold: Minimum body size ratio compared to the candle's range.
        wick_body_ratio: Minimum ratio of upper wick length to body size for the second candle.

    Returns:
        pd.Series: Boolean Series indicating Inverted Hammer patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    is_tall_black = is_black & (body_size / candle_range >= body_threshold)

    is_short = df['close'] > df['open'] | (abs(df['close']-df['open']) / candle_range < body_threshold)

    upper_wick = df['high'] - max(df['open'], df['close'])
    lower_wick = min(df['open'], df['close']) - df['low']
    upper_wick_ratio = upper_wick / body_size
    
    second_candle_is_short = df.shift(-1)['close'] > df.shift(-1)['open'] | (abs(df.shift(-1)['close'] - df.shift(-1)['open']) / (df.shift(-1)['high'] - df.shift(-1)['low']) < body_threshold)
    
    inverted_hammer_condition = is_tall_black & (df['open'].shift(-1) < df['close']) & second_candle_is_short & (upper_wick_ratio >= wick_body_ratio)


    return inverted_hammer_condition

# Ref: https://thepatternsite.com/KickingBear.html
# This function detects the Bearish Kicking candlestick pattern.
# It identifies two consecutive marubozu candles: a white candle followed by a gapped-down black candle.
# The function uses a gap threshold to determine the minimum gap between candles.
# The function returns a boolean Series indicating the presence of the pattern for each row in the DataFrame.

def do_detect_bearish_kicking(df: pd.DataFrame, gap_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Bearish Kicking candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must have columns "open", "high", "low", "close", "volume", and "date".
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) required between candles.

    Returns:
        A pandas Series with True where the Bearish Kicking pattern is detected, False otherwise.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    #Ensure marubozu candles
    is_white_marubozu = (is_white) & (df['high'] == df['close']) & (df['low'] == df['open'])
    is_black_marubozu = (is_black) & (df['high'] == df['open']) & (df['low'] == df['close'])

    #Shift the is_white_marubozu series to align it with the next day's data for pairing.
    shifted_is_white_marubozu = is_white_marubozu.shift(1)
    

    # Calculate the gap between consecutive candles.
    daily_range = df['high'] - df['low']
    shifted_daily_range = daily_range.shift(1)
    gap = df['open'] - df['close'].shift(1)
    gap_percentage = gap / shifted_daily_range
    
    #This is the gap threshold expressed as a fraction, rather than a percentage.
    gap_threshold_fraction = gap_threshold

    # Combine conditions to detect the pattern.
    bearish_kicking = (shifted_is_white_marubozu) & (is_black_marubozu) & (gap_percentage > gap_threshold_fraction)

    return bearish_kicking


# Ref: https://thepatternsite.com/KickingBull.html
# This function detects the "Bullish Kicking" candlestick pattern.
# It identifies a tall black marubozu candle followed by an upward gap and a tall white marubozu candle.
# The function takes a Pandas DataFrame as input and returns a boolean Series indicating the presence of the pattern.

def do_detect_bullish_kicking(df: pd.DataFrame, marubozu_body_threshold: float = 0.98, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Kicking candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        marubozu_body_threshold: Minimum ratio of body size to candle range for a candle to be considered a marubozu.
        gap_threshold: Minimum gap between the two candles as percentage of the previous candle's range.

    Returns:
        Boolean Series indicating Bullish Kicking patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    is_marubozu = body_size / candle_range >= marubozu_body_threshold
    
    is_black_marubozu = is_black & is_marubozu
    is_white_marubozu = is_white & is_marubozu

    #shift is_black_marubozu one period forward to compare with is_white_marubozu of the next period
    previous_black_marubozu = is_black_marubozu.shift(1)

    #calculate the gap
    gap = (df["open"] - df["close"].shift(1)) / candle_range.shift(1)

    #check if the gap is above the gap threshold
    sufficient_gap = gap > gap_threshold
    
    # Combine conditions to detect the pattern
    bullish_kicking = previous_black_marubozu & is_white_marubozu & sufficient_gap

    return bullish_kicking


# Ref: https://thepatternsite.com/LadderBottom.html
# This function detects the Ladder Bottom candlestick pattern.
# The function identifies a ladder bottom pattern based on the criteria described in the documentation:
# 1. Five candles in a downward trend.
# 2. The first three candles are tall black candles (close < open).
# 3. The fourth candle is a black candle with an upper shadow.
# 4. The fifth candle is a white candle (close > open) that gaps open above the previous day's close.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_ladder_bottom(df: pd.DataFrame, lower_body_threshold: float = 0.2, upper_shadow_threshold: float = 0.2, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Ladder Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date.
        lower_body_threshold: Minimum ratio of the body to the candle's range to be considered a black candle.
        upper_shadow_threshold: Minimum ratio of the upper shadow to the candle's range to be considered a significant upper shadow.
        gap_threshold: Minimum gap (as percentage of the previous candle's range) required between candles.

    Returns:
        A pandas Series indicating where a ladder bottom pattern is detected.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body = abs(df['close'] - df['open'])
    upper_shadow = df['high'] - df.loc[:, 'close'].clip(upper=df['open'])
    range_ = df['high'] - df['low']
    
    lower_body_ratio = body / range_
    upper_shadow_ratio = upper_shadow / range_
    
    # Check for the first three black candles.
    first_three_black = is_black.rolling(3).apply(lambda x: all(x), raw=True)

    #Check for the fourth candle with an upper shadow
    fourth_candle_upper_shadow = is_black & (upper_shadow_ratio >= upper_shadow_threshold)

    # Check for the fifth candle which is white and has gap above previous day's close
    fifth_candle_white_and_gap = (is_white & (df['open'] > df['close'].shift(1)) & (df['open'] - df['close'].shift(1) >= gap_threshold * range_.shift(1)))
    
    #Combine conditions
    ladder_bottom = pd.concat([first_three_black.shift(2), fourth_candle_upper_shadow.shift(1), fifth_candle_white_and_gap], axis=1).all(axis=1)
    
    return ladder_bottom


# Ref: https://thepatternsite.com/LastEngulfBottom.html
# This function detects the "Last Engulfing Bottom" candlestick pattern.
# It identifies a bullish reversal pattern where a small white candle is followed by a larger black candle that engulfs the body of the white candle.
# The function ignores shadows and focuses solely on the candle bodies.  The function returns a boolean Series indicating the presence of the pattern.
# The pattern is identified by comparing the high, low, open, and close prices of consecutive candles.
# The function should be robust and handle edge cases (e.g., empty DataFrame).

def do_detect_last_engulfing_bottom(df: pd.DataFrame, body_ratio_threshold: float = 1.0) -> pd.Series:
    """
    Detects the Last Engulfing Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close).
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.


    Returns:
        pd.Series: Boolean Series indicating Last Engulfing Bottom patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_size1 = abs(df["close"] - df["open"])
    body_size2 = abs(df["close"].shift(-1) - df["open"].shift(-1))
    engulfing = (df["high"].shift(-1) > df["high"]) & (df["low"].shift(-1) < df["low"]) & ~is_white & is_white.shift(-1) & (body_size2 > body_ratio_threshold * body_size1)

    return engulfing


# Ref: https://thepatternsite.com/LastEngulfTop.html
# The Last Engulfing Top pattern consists of two candles.
# The first candle is a black candle (close < open) in an upward trend.
# The second candle is a taller white candle (close > open) that engulfs the body of the first candle.
# The white candle's body is entirely above the first candle's high and entirely below the first candle's low.

def do_detect_last_engulfing_top(df: pd.DataFrame, body_threshold: float = 0.1) -> pd.Series:
    """
    Detects the Last Engulfing Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        body_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size_1 = abs(df['close'] - df['open'])
    body_size_2 = abs(df['close'].shift(-1) - df['open'].shift(-1))
    
    # Condition for the second candle's body to be larger than the first's.
    larger_body = body_size_2 > body_size_1 * body_threshold
    
    # Condition for engulfing.
    engulfing = (df['high'].shift(-1) > df['high']) & (df['low'].shift(-1) < df['low'])

    pattern = (is_black & is_white.shift(-1) & engulfing & larger_body).shift(1)

    return pattern


# Ref: https://thepatternsite.com/LongBlack.html
# This function detects the "Long Black Day" candlestick pattern.
# A long black day is defined as a tall black candle with shadows shorter than the body.
# The body is considered "tall" if it is three times the average body height of recent candles.

def do_detect_long_black_day(df: pd.DataFrame, recent_candle_count: int = 10, body_height_factor: float = 3.0) -> pd.Series:
    """
    Detects the Long Black Day candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close' columns.
        recent_candle_count: Number of recent candles to consider for average body height calculation.
        body_height_factor: Factor determining how many times the average body height must be exceeded to qualify as a "long" body.

    Returns:
        A pandas Series of booleans indicating whether a Long Black Day pattern is detected for each row in the DataFrame.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate average body height of recent candles
    avg_body_height = df['close'].sub(df['open']).abs().rolling(window=recent_candle_count, min_periods=1).mean()

    # Identify black candles (close < open)
    is_black = df['close'] < df['open']

    # Calculate body height and check for long bodies
    body_height = df['open'].sub(df['close']).abs()
    is_long_body = body_height > avg_body_height * body_height_factor

    # Check for shadows shorter than body
    upper_shadow = df['high'] - df['open'].where(is_black, df['close'])
    lower_shadow = df['open'].where(is_black, df['close']) - df['low']
    is_short_shadows = (upper_shadow < body_height) & (lower_shadow < body_height)

    # Combine conditions to detect Long Black Day pattern
    is_long_black_day = is_black & is_long_body & is_short_shadows
    return is_long_black_day


# Ref: https://thepatternsite.com/LongWhiteDay.html
# This function detects the "Long White Day" candlestick pattern.
# A long white day is characterized by a tall white candle with shadows shorter than the body,
# and a body at least three times taller than the average body height over a specified period.

def do_detect_long_white_day(df: pd.DataFrame, average_body_length_period: int = 2, body_length_factor: float = 3.0, min_body_length_factor: float = 0.5) -> pd.Series:
    """
    Detects the Long White Day candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        average_body_length_period: Period to calculate the average body length.
        body_length_factor: Minimum factor by which the current candle body must exceed the average body length.
        min_body_length_factor: Minimum factor by which the body must exceed the shadow sizes.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_length = abs(df["close"] - df["open"])
    average_body_length = body_length.rolling(window=average_body_length_period, min_periods=1).mean()
    is_long = body_length >= average_body_length * body_length_factor
    shadow_size = min(abs(df['high'] - df['close']), abs(df['open'] - df['low']))
    is_short_shadow = body_length >= shadow_size * min_body_length_factor
    is_long_white_day = is_white & is_long & is_short_shadow

    return is_long_white_day

# Ref: https://thepatternsite.com/LongLegDoji.html
# This function detects the "Long Legged Doji" candlestick pattern.
# A long legged doji is characterized by a small body (open and close prices are nearly equal)
# and long upper and lower shadows, indicating indecision in the market.
# The function uses several thresholds to define what constitutes a "long" shadow and a "small" body.

def do_detect_long_legged_doji(df: pd.DataFrame, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects Long Legged Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        wick_body_ratio: Minimum ratio of the sum of upper and lower wicks to the body size.

    Returns:
        pd.Series: Boolean Series indicating Long Legged Doji patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate body size
    body_size = abs(df['close'] - df['open'])

    # Calculate upper and lower wick lengths
    upper_wick = df['high'] - df.apply(lambda x: max(x['open'], x['close']), axis=1)
    lower_wick = df.apply(lambda x: min(x['open'], x['close']), axis=1) - df['low']

    # Check for long legged doji conditions
    is_long_legged_doji = (body_size / (upper_wick + lower_wick)) < (1 / wick_body_ratio) & (upper_wick > 0) & (lower_wick > 0)

    return is_long_legged_doji


# Ref: https://thepatternsite.com/BlackMarubozu.html
# This function detects the 'Black Marubozu' candlestick pattern.
# A black marubozu is a tall black candle with no upper or lower shadows.
# The function checks if the open and close prices are equal to the high and low prices respectively.

def do_detect_black_marubozu(df: pd.DataFrame, shadow_threshold: float = 0.0) -> pd.Series:
    """
    Detects Black Marubozu candlestick patterns.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        shadow_threshold: Maximum allowed shadow length as a fraction of the candle body.

    Returns:
        A pandas Series of booleans indicating the presence of Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    upper_shadow = df["high"] - df["open"]
    lower_shadow = df["open"] - df["low"]
    body = df["open"] - df["close"]

    no_upper_shadow = upper_shadow <= shadow_threshold * body
    no_lower_shadow = lower_shadow <= shadow_threshold * body

    is_marubozu = is_black & no_upper_shadow & no_lower_shadow
    return is_marubozu

# Ref: https://thepatternsite.com/CloseBlkMarubozu.html
# This function detects the 'Marubozu, closing black' candlestick pattern.
# It checks for a tall black candle with an upper shadow but no lower shadow.
# The function takes a Pandas DataFrame as input and returns a boolean Series
# indicating the presence of the pattern.

def do_detect_closing_black_marubozu(df: pd.core.frame.DataFrame, upper_shadow_threshold: float = 0.1) -> pd.Series:
    """
    Detects Closing Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        upper_shadow_threshold: Minimum ratio of upper shadow length to candle body for it to be considered a Closing Black Marubozu

    Returns:
        pd.Series: Boolean Series indicating Closing Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    body = df["open"] - df["close"]

    is_closing_black_marubozu = (is_black) & (lower_shadow <= upper_shadow_threshold * body) & (upper_shadow > 0)

    return is_closing_black_marubozu


# Ref: https://thepatternsite.com/ClosingWhiteMarubozu.html
# The closing white marubozu candlestick is characterized by a tall white candle with no upper shadow but a lower shadow.
# It's considered a continuation pattern, indicating a continuation of the existing price trend.  The function below
# identifies this pattern based on the provided OHLC data.  Thresholds for "tall" and "shadow" are parameterized.


def do_detect_closing_white_marubozu(df: pd.DataFrame, min_body_size_ratio: float = 0.7, max_upper_shadow_ratio: float = 0.0, min_lower_shadow_ratio:float = 0.001) -> pd.Series:
    """
    Detects the 'Closing White Marubozu' candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.  Index must be DatetimeIndex.
        min_body_size_ratio: Minimum ratio of body size to total candle range.
        max_upper_shadow_ratio: Maximum ratio of upper shadow to total candle range.  Must be >=0 and <=1.
        min_lower_shadow_ratio: Minimum ratio of lower shadow to total candle range. Must be >=0 and <=1.

    Returns:
        Boolean Series indicating 'Closing White Marubozu' patterns.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    total_range = df['high'] - df['low']
    body_size = df['close'] - df['open']
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']

    is_white = body_size > 0
    is_tall = body_size / total_range >= min_body_size_ratio
    has_small_upper_shadow = upper_shadow / total_range <= max_upper_shadow_ratio
    has_lower_shadow = lower_shadow / total_range >= min_lower_shadow_ratio
    
    is_closing_white_marubozu = is_white & is_tall & has_small_upper_shadow & has_lower_shadow

    return is_closing_white_marubozu


# Ref: https://thepatternsite.com/OpenBlkMaru.html
# Detects the "Opening Black Marubozu" candlestick pattern.
# An opening black marubozu is a tall, black candle with no upper shadow but a lower shadow.
# It's considered a continuation pattern approximately half the time.  
# This function identifies the pattern based on the criteria:
# 1. A single candle.
# 2. No upper shadow (high == open).
# 3. Lower shadow present (low < open).
# 4. The candle is black (close < open).

def do_detect_opening_black_marubozu(df: pd.DataFrame, lower_shadow_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Opening Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        lower_shadow_threshold: Minimum percentage lower shadow length compared to body length.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    no_upper_shadow = df['high'] == df['open']
    lower_shadow = df['low'] < df['open']
    lower_shadow_length = df['open'] - df['low']
    body_length = df['open'] - df['close']
    sufficient_lower_shadow = lower_shadow_length / body_length >= lower_shadow_threshold

    is_opening_black_marubozu = is_black & no_upper_shadow & lower_shadow & sufficient_lower_shadow

    return is_opening_black_marubozu


# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# Detects the "Opening White Marubozu" candlestick pattern.
# An opening white marubozu is a tall white candle with an upper shadow but no lower shadow.
# The height is relative to other recent candles.

def do_detect_opening_white_marubozu(df: pd.DataFrame, upper_shadow_threshold: float = 0.1, body_size_threshold: float = 0.7) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        upper_shadow_threshold: Minimum ratio of upper shadow to candle body for it to be considered a marubozu.
        body_size_threshold: Minimum ratio of body size to candle range to be considered a tall candle.

    Returns:
        A pandas Series (boolean mask) indicating Opening White Marubozu patterns.  Index is aligned with df.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    upper_shadow = df["high"] - df["close"]
    body_size = df["close"] - df["open"]
    candle_range = df["high"] - df["low"]
    
    upper_shadow_ratio = (upper_shadow / body_size)
    body_size_ratio = (body_size / candle_range)
    
    is_upper_shadow = upper_shadow_ratio < upper_shadow_threshold
    is_tall_candle = body_size_ratio > body_size_threshold
    has_no_lower_shadow = df["open"] == df["low"]
    
    is_opening_white_marubozu = is_white & is_upper_shadow & is_tall_candle & has_no_lower_shadow

    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/WhiteMarubozu.html
# This function detects the 'Marubozu, white' candlestick pattern.
# A white marubozu is a tall white candle with no shadows.  The function checks if the
# close price is greater than the open price (white candle) and if the high and low prices
# are equal to the open and close prices respectively (no shadows).


def do_detect_white_marubozu(df: pd.DataFrame, shadow_threshold: float = 0.001) -> pd.Series:
    """
    Detects the white marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        shadow_threshold: The maximum acceptable shadow length as a fraction of the candle's body.

    Returns:
        pd.Series: Boolean Series indicating white marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    high_shadow = (df["high"] - df["close"]) / (df["close"] - df["open"])
    low_shadow = (df["open"] - df["low"]) / (df["close"] - df["open"])

    is_marubozu = (high_shadow <= shadow_threshold) & (low_shadow <= shadow_threshold)

    return is_white & is_marubozu


# Ref: https://thepatternsite.com/MatchingLow.html
# The Matching Low pattern consists of two consecutive candles.
# The first candle is a black candle with a tall body.
# The second candle is a black candle whose closing price matches the closing price of the first candle.
# The closing price is what matches, not the low price.
# The pattern is considered a bearish continuation 61% of the time, despite being theoretically a bullish reversal.
# Best performance is observed after an upward breakout, acting as a reversal.

def do_detect_matching_low(df: pd.DataFrame, low_diff_threshold: float = 0.001) -> pd.Series:
    """
    Detects the Matching Low candlestick pattern.

    Args:
        df: DataFrame with OHLC data and date index.
        low_diff_threshold: Maximum difference (as fraction) between current and previous lows for a match.


    Returns:
        Boolean Series indicating Matching Low patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_matching_low = False
    
    is_matching_low = (is_black & is_black.shift(1) & (df['close'] == df['close'].shift(1)))
    return is_matching_low

# Ref: https://thepatternsite.com/MatHold.html
# This function detects the Mat Hold candlestick pattern.
# The pattern consists of five candles:
# 1. A tall white candle.
# 2. A small black candle with a higher close than the previous day's low.
# 3. A small candle (can be any color).
# 4. A small black candle with a close above the low of the first day.
# 5. A tall white candle closing above the high of the prior four candles.

def do_detect_mat_hold(df: pd.core.frame.DataFrame, tall_candle_factor: float = 2.0, small_candle_factor: float = 0.2) -> pd.Series:
    """
    Detects the Mat Hold candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close', 'volume', 'date'.
        tall_candle_factor: Factor determining the minimum body size of a tall candle relative to its average body size.
        small_candle_factor: Factor determining the maximum body size of a small candle relative to its average body size.

    Returns:
        A pandas Series of booleans indicating the presence of a Mat Hold pattern at each index point.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body sizes
    body_sizes = abs(df['close'] - df['open'])
    average_body_size = body_sizes.rolling(window=5).mean()

    # Identify tall and small candles
    is_tall_candle = body_sizes >= average_body_size * tall_candle_factor
    is_small_candle = body_sizes <= average_body_size * small_candle_factor
    is_white_candle = df['close'] > df['open']
    is_black_candle = df['close'] < df['open']


    # Check for Mat Hold pattern
    mat_hold = (
        is_tall_candle.shift(4) & is_white_candle.shift(4) &
        is_small_candle.shift(3) & is_black_candle.shift(3) & (df['close'].shift(3) > df['low'].shift(4)) &
        is_small_candle.shift(2) &
        is_small_candle.shift(1) & is_black_candle.shift(1) & (df['close'].shift(1) > df['low'].shift(4)) &
        is_tall_candle & is_white_candle & (df['close'] > df['high'].shift(1))
    )
    
    return mat_hold


# Ref: https://thepatternsite.com/MeetingLinesBear.html
# Detects the 'Meeting Lines, Bearish' candlestick pattern.
# The pattern consists of two candles: a tall white candle followed by a tall black candle,
# with their closing prices near each other.  The closeness of the closing prices is defined by the `close_price_diff_threshold` parameter.
# The "tallness" of the candles is determined by comparing their body lengths to their total ranges (`body_range_ratio_threshold`).

def do_detect_bearish_meeting_lines(df: pd.DataFrame, close_price_diff_threshold: float = 0.01, body_range_ratio_threshold: float = 0.6) -> pd.Series:
    """
    Detects the Bearish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume, indexed by datetime.
        close_price_diff_threshold: Maximum difference between closing prices of the two candles (as a fraction of the first candle's range).
        body_range_ratio_threshold: Minimum ratio of body size to candle range for each candle to be considered 'tall'.


    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.  Index is aligned with input DataFrame.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_range = df['high'] - df['low']
    body_size = abs(df['close'] - df['open'])
    is_tall = body_size / body_range > body_range_ratio_threshold

    # Identify tall white candles
    tall_white = is_white & is_tall

    #Shift the tall white candles forward to compare with the next candle
    shifted_tall_white = tall_white.shift(1)
    
    # Identify the subsequent tall black candle.
    is_black = ~is_white
    tall_black = is_black & is_tall

    # Calculate the difference between the closing prices of the two candles
    close_price_diff = abs(df['close'] - df['close'].shift(1)) / body_range.shift(1)

    # Detect the pattern
    meeting_lines = shifted_tall_white & tall_black & (close_price_diff <= close_price_diff_threshold)

    return meeting_lines


# Ref: https://thepatternsite.com/MeetingLinesBull.html
# Detects the bullish meeting lines candlestick pattern.
# The pattern consists of two candles: a tall black candle followed by a tall white candle,
# with closing prices near each other.  The function checks for this pattern and returns a boolean Series
# indicating whether the pattern is present.


def do_detect_bullish_meeting_lines(df: pd.DataFrame, close_diff_threshold: float = 0.01, body_size_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Bullish Meeting Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        close_diff_threshold: Maximum difference between the closing prices of the two candles (as a fraction of the first candle's range).
        body_size_threshold: Minimum body size as a fraction of the previous candle's range.

    Returns:
        pd.Series: Boolean Series indicating Bullish Meeting Lines patterns.  Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    
    black_candles = df[is_black]
    white_candles = df[is_white]
    
    # Check for two candles: a black followed by a white
    meeting_lines = pd.Series(index=df.index, data=False)
    for i in range(len(df) - 1):
      # Check if current candle is black and next is white
      if is_black[df.index[i]] and is_white[df.index[i+1]]:
        # Check closing price difference
        close_diff = abs(black_candles["close"][df.index[i]] - white_candles["close"][df.index[i+1]])
        if close_diff <= close_diff_threshold * candle_range[df.index[i]]:
          # Check body size of the white candle
          if body_size[df.index[i+1]] >= body_size_threshold * candle_range[df.index[i]]:
            meeting_lines[df.index[i+1]] = True
    return meeting_lines


# Ref: https://thepatternsite.com/MorningDojiStar.html
# This function detects the Morning Doji Star candlestick pattern.
# It checks for a tall black candle followed by a doji,
# then a tall white candle, with gaps between the bodies.

def do_detect_morning_doji_star(df: pd.DataFrame, doji_body_threshold: float = 0.1, gap_threshold: float = 0.01, tall_candle_body_factor: float = 2.0) -> pd.Series:
    """
    Detects the Morning Doji Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        doji_body_threshold: Maximum ratio of body size to the total candle range for a doji.
        gap_threshold: Minimum percentage gap between consecutive candle bodies.
        tall_candle_body_factor: Minimum ratio of body size to the candle's range for a tall candle.

    Returns:
        A pandas Series (boolean mask) indicating Morning Doji Star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body = abs(df['close'] - df['open'])
    range_ = df['high'] - df['low']
    body_range_ratio = body / range_
    is_doji = body_range_ratio <= doji_body_threshold

    is_tall_white = (df['close'] - df['open']) / range_ >= tall_candle_body_factor
    is_tall_black = (df['open'] - df['close']) / range_ >= tall_candle_body_factor
    
    #shift() cannot have a negative parameter.
    is_morning_doji_star = (is_tall_black & is_doji.shift(1) & is_tall_white.shift(2) &
                          (df['open'].shift(1) - df['close'].shift(1)) / range_.shift(1) >= gap_threshold &
                          (df['open'].shift(2) - df['close'].shift(1)) / range_.shift(1) >= gap_threshold)


    return is_morning_doji_star


# Ref: https://thepatternsite.com/MorningStar.html
# This function detects the Morning Star candlestick pattern.
# It takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a boolean Series indicating the presence of the pattern.
# The index of the returned Series is aligned with the input DataFrame.

def do_detect_morning_star(df: pd.DataFrame, body_threshold: float = 0.1, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Morning Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume. Must contain 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        body_threshold: Minimum body size ratio compared to the previous candle's body size.
        gap_threshold: Minimum gap between candles.
    Returns:
        Boolean Series indicating Morning Star patterns.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    
    # Calculate the gap between consecutive candles
    gap = df['open'].shift(-1) - df['close']
    
    # Identify the three-candle pattern
    morning_star = (is_black &
                    body_size.shift(-1) < body_threshold * body_size &
                    is_white.shift(-2) & 
                    gap > gap_threshold * body_size &
                    (df['open'].shift(-2) > df['close'].shift(-1)))

    return morning_star

# Ref: https://thepatternsite.com/NorthernDoji.html
# The Northern Doji is characterized by a doji candlestick appearing in an upward trend.
# A doji has opening and closing prices very close to each other.  
# This function detects this pattern based on the proximity of open and close prices and the preceding trend.

def do_detect_northern_doji(df: pd.DataFrame, doji_threshold: float = 0.005, trend_length: int = 5) -> pd.Series:
    """
    Detects the Northern Doji candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        doji_threshold: The maximum percentage difference between open and close prices to be considered a doji.
        trend_length: Number of periods to check the preceding upward trend.

    Returns:
        A pandas Series of booleans indicating the presence of the Northern Doji pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Check for doji candles
    is_doji = abs((df['close'] - df['open']) / df['open']) <= doji_threshold

    # Check for upward trend
    is_uptrend = df['close'].rolling(window=trend_length).apply(lambda x: x[-1] > x[0])

    # Combine conditions to identify Northern Doji
    is_northern_doji = is_doji & is_uptrend

    return is_northern_doji

# Ref: https://thepatternsite.com/OnNeck.html
# The On Neck pattern consists of a tall black candle followed by a white candle
# whose close is at or near the low of the black candle.  Both appear in a
# downward price trend.

def do_detect_on_neck(df: pd.DataFrame, body_ratio_threshold: float = 0.1, close_match_threshold: float = 0.01) -> pd.Series:
    """
    Detects the On Neck candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must contain columns "open", "high", "low", "close", "volume", and "date".
        body_ratio_threshold: Minimum ratio of the first candle's body size to its total range.
        close_match_threshold: Maximum difference (as a fraction) allowed between the close of the second candle and the low of the first candle.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    
    is_tall_black = body_size / total_range > body_ratio_threshold
    
    shifted_low = df["low"].shift(1)
    close_match = abs(df["close"] - shifted_low) / shifted_low < close_match_threshold

    on_neck = (is_tall_black & is_white & is_black.shift(1) & close_match).shift(-1)
    
    return on_neck


# Ref: https://thepatternsite.com/OpenBlkMaru.html
# Detects the Opening Black Marubozu candlestick pattern.
# An opening black marubozu is a tall, black candle with no upper shadow but a lower shadow.
# No prior price trend is required.

def do_detect_opening_black_marubozu(df: pd.DataFrame, lower_shadow_threshold: float = 0.1) -> pd.Series:
    """
    Detects the Opening Black Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns 'open', 'high', 'low', 'close'.
        lower_shadow_threshold: Minimum ratio of lower shadow to the candle body.

    Returns:
        pd.Series: Boolean Series indicating Opening Black Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    upper_shadow = df['high'] - df['open']
    lower_shadow = df['open'] - df['low']
    body = df['open'] - df['close']
    has_no_upper_shadow = upper_shadow <= 0.001 * body #Accounts for floating-point inaccuracies
    has_lower_shadow = lower_shadow > lower_shadow_threshold * body

    is_opening_black_marubozu = is_black & has_no_upper_shadow & has_lower_shadow

    return is_opening_black_marubozu


# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# This function detects the "Opening White Marubozu" candlestick pattern.
# An opening white marubozu is a tall white candle with an upper shadow but no lower one.
# The height is relative; compare the height with other recent candles to gauge whether or not the candle is "tall".
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series with boolean values indicating the presence (True) or absence (False) of the pattern for each row.


def do_detect_opening_white_marubozu(df: pd.DataFrame, upper_shadow_threshold: float = 0.1, body_size_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        upper_shadow_threshold: Minimum ratio of upper shadow to candle body for it to be considered significant.
        body_size_threshold: Minimum ratio of candle body size to total candle range for it to be considered significant.


    Returns:
        pd.Series: Boolean Series indicating Opening White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    body_size = df["close"] - df["open"]
    total_range = df["high"] - df["low"]
    
    significant_upper_shadow = (upper_shadow / body_size) >= upper_shadow_threshold
    significant_body = (body_size / total_range) >= body_size_threshold
    no_lower_shadow = lower_shadow <= 0

    is_opening_white_marubozu = is_white & significant_upper_shadow & significant_body & no_lower_shadow

    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# This function detects the "Opening White Marubozu" candlestick pattern.
# An opening white marubozu is a tall white candle with an upper shadow but no lower one.
# The height is relative; compare the height with other recent candles to gauge whether or not the candle is "tall".
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series with boolean values indicating the presence (True) or absence (False) of the pattern for each row.


def do_detect_opening_white_marubozu(df: pd.DataFrame, upper_shadow_threshold: float = 0.1, body_size_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        upper_shadow_threshold: Minimum ratio of upper shadow to candle body for it to be considered significant.
        body_size_threshold: Minimum ratio of candle body size to total candle range for it to be considered significant.


    Returns:
        pd.Series: Boolean Series indicating Opening White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    body_size = df["close"] - df["open"]
    total_range = df["high"] - df["low"]
    
    significant_upper_shadow = (upper_shadow / body_size) >= upper_shadow_threshold
    significant_body = (body_size / total_range) >= body_size_threshold
    no_lower_shadow = lower_shadow <= 0

    is_opening_white_marubozu = is_white & significant_upper_shadow & significant_body & no_lower_shadow

    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/OpenWhiteMarubozu.html
# This function detects the "Opening White Marubozu" candlestick pattern.
# An opening white marubozu is a tall white candle with an upper shadow but no lower one.
# The height is relative; compare the height with other recent candles to gauge whether or not the candle is "tall".
# The function takes a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns as input.
# It returns a pandas Series with boolean values indicating the presence (True) or absence (False) of the pattern for each row.


def do_detect_opening_white_marubozu(df: pd.DataFrame, upper_shadow_threshold: float = 0.1, body_size_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Opening White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        upper_shadow_threshold: Minimum ratio of upper shadow to candle body for it to be considered significant.
        body_size_threshold: Minimum ratio of candle body size to total candle range for it to be considered significant.


    Returns:
        pd.Series: Boolean Series indicating Opening White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    upper_shadow = df["high"] - df["close"]
    lower_shadow = df["open"] - df["low"]
    body_size = df["close"] - df["open"]
    total_range = df["high"] - df["low"]
    
    significant_upper_shadow = (upper_shadow / body_size) >= upper_shadow_threshold
    significant_body = (body_size / total_range) >= body_size_threshold
    no_lower_shadow = lower_shadow <= 0

    is_opening_white_marubozu = is_white & significant_upper_shadow & significant_body & no_lower_shadow

    return is_opening_white_marubozu


# Ref: https://thepatternsite.com/Piercing.html
# This function detects the Piercing Pattern candlestick pattern.
# The pattern consists of two candles: a black candle followed by a white candle.
# The white candle opens below the black candle's low and closes between the midpoint of the black candle's body and the opening price.

def do_detect_piercing_pattern(df: pd.DataFrame, midpoint_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Piercing Pattern candlestick pattern.

    Args:
        df: DataFrame with "open", "high", "low", "close", "volume", and "date" columns.
        midpoint_threshold: The threshold for determining if the white candle closes above the midpoint of the black candle.

    Returns:
        A pandas Series with boolean values indicating the presence of the Piercing Pattern.
        Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]

    # Shift to align with the next candle
    shifted_is_black = is_black.shift(1)
    shifted_low = df["low"].shift(1)
    shifted_open = df["open"].shift(1)
    shifted_close = df["close"].shift(1)
    
    #Calculate the midpoint of the black candle
    midpoint = (shifted_open + shifted_close) * midpoint_threshold

    #Check conditions for piercing pattern
    pattern_detected = (shifted_is_black) & (is_white) & (df["open"] < shifted_low) & (df["close"] > midpoint) & (df["close"] < shifted_open)

    return pattern_detected

# Ref: https://thepatternsite.com/RickshawMan.html
# The Rickshaw Man pattern is characterized by a tall candle with a doji body (open and close prices are nearly equal), 
# and the body is located near the middle of the candle's range.  This function detects this pattern.

def do_calculate_rickshaw_man(df: pd.DataFrame, body_middle_threshold: float = 0.2, wick_length_threshold: float = 2.0) -> pd.Series:
    """
    Detects the Rickshaw Man candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close', 'volume', 'date').
        body_middle_threshold: Maximum distance of body center from candle midpoint (as a fraction of candle range).
        wick_length_threshold: Minimum ratio of total wick length to body size.

    Returns:
        pd.Series: Boolean Series indicating Rickshaw Man patterns.
        FIXME: Requires testing and refinement of threshold parameters.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df['close'] - df['open'])
    total_range = df['high'] - df['low']
    wick_length = total_range - body_size
    body_midpoint = (df['open'] + df['close']) / 2
    candle_midpoint = (df['high'] + df['low']) / 2
    is_doji = (body_size / total_range) < 0.05  # Consider a very small body size as a doji
    is_body_near_middle = abs(body_midpoint - candle_midpoint) / total_range <= body_middle_threshold
    is_long_wick = (wick_length / body_size) >= wick_length_threshold

    is_rickshaw_man = is_doji & is_body_near_middle & is_long_wick

    return is_rickshaw_man


# Ref: https://thepatternsite.com/Rising3Methods.html
# This function detects the Rising Three Methods candlestick pattern.
# The pattern consists of a tall white candle followed by three smaller candles
# that trend lower but close within the high-low range of the first candle.
# The three smaller candles are followed by another tall white candle that
# closes above the close of the first candle.


def do_detect_rising_three_methods(df: pd.DataFrame, tall_candle_factor: float = 1.0, small_candle_factor: float = 0.3) -> pd.Series:
    """
    Detects the Rising Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must contain columns named "open", "high", "low", "close", "volume", and "date".
        tall_candle_factor: Minimum factor for the body size of the tall white candles.
        small_candle_factor: Maximum factor for the body size of the small candles.

    Returns:
        Boolean Series indicating Rising Three Methods patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    candle_range = df["high"] - df["low"]
    is_tall_white = is_white & (body_size >= tall_candle_factor * candle_range)
    is_small = body_size <= small_candle_factor * candle_range
    
    first_tall_white = is_tall_white.shift(4)
    
    #Check if the three small candles are within the first tall white candle's high-low range
    within_range1 = df["high"].shift(3) >= df["low"].shift(4)
    within_range2 = df["low"].shift(3) <= df["high"].shift(4)
    within_range3 = df["high"].shift(2) >= df["low"].shift(4)
    within_range4 = df["low"].shift(2) <= df["high"].shift(4)
    within_range5 = df["high"].shift(1) >= df["low"].shift(4)
    within_range6 = df["low"].shift(1) <= df["high"].shift(4)
    within_range = within_range1 & within_range2 & within_range3 & within_range4 & within_range5 & within_range6

    second_tall_white = is_tall_white & (df["close"] > df["close"].shift(4))

    rising_three_methods = first_tall_white & within_range & ~is_tall_white.shift(3) & ~is_tall_white.shift(2) & ~is_tall_white.shift(1) & second_tall_white
    return rising_three_methods

# Ref: https://thepatternsite.com/RisingWindow.html
# This function detects the Rising Window candlestick pattern.
# A rising window is characterized by a gap between consecutive days where
# yesterday's high is below today's low.  This indicates a strong bullish continuation.
# The function takes a Pandas DataFrame as input with 'high' and 'low' columns and returns a boolean Series.

def do_detect_rising_window(df: pd.DataFrame, constant: bool = False) -> pd.Series:
    """
    Detects Rising Window candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'high' and 'low' columns.  Must not be modified.
        constant: if True, then the df parameter is treated as a constant.

    Returns:
        pd.Series: Boolean Series indicating Rising Window patterns (True if a rising window is detected).
    """
    if len(df) == 0:
        return pd.Series([], dtype=bool)

    if constant:
        df = df.copy() # Create a copy in case it was passed as a constant parameter


    high_yesterday = df["high"].shift(1)
    low_today = df["low"]
    is_rising_window = low_today > high_yesterday

    return is_rising_window


# Ref: https://thepatternsite.com/SeparateLinesBear.html
# This function detects the "Bearish Separating Lines" candlestick pattern.
# It checks for a tall white candle followed by a tall black candle with similar opening prices.
# The function uses pandas for efficient data manipulation.  The function does not modify the input dataframe.


def do_detect_bearish_separating_lines(df: pd.DataFrame, opening_price_similarity_threshold: float = 0.02, tall_candle_body_factor: float = 2.0) -> pd.Series:
    """
    Detects the Bearish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        opening_price_similarity_threshold: Maximum percentage difference allowed between the opening prices of the two candles.
        tall_candle_body_factor: Minimum ratio of the candle body to the candle range for a candle to be considered tall.

    Returns:
        A pandas Series of booleans indicating the presence of the pattern.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    candle_body = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']

    tall_white = (candle_body / candle_range) >= tall_candle_body_factor
    tall_black = (candle_body / candle_range) >= tall_candle_body_factor

    # Shift to compare with previous candle
    prev_is_white = is_white.shift(1)
    prev_open = df['open'].shift(1)
    
    opening_price_diff = abs((df['open'] - prev_open) / prev_open)

    pattern_detected = (prev_is_white & is_black & (opening_price_diff <= opening_price_similarity_threshold) & tall_white & tall_black)

    return pattern_detected


# Ref: https://thepatternsite.com/SeparateLinesBull.html
# This function detects the 'Bullish Separating Lines' candlestick pattern.
# It identifies two consecutive candles: a tall black candle followed by a tall white candle,
# sharing a near-identical opening price.  The function uses relative thresholds to define "tall" candles.

def do_detect_bullish_separating_lines(df: pd.DataFrame, black_candle_threshold: float = 0.02, white_candle_threshold: float = 0.02, opening_price_difference_threshold: float = 0.001) -> pd.Series:
    """
    Detects the Bullish Separating Lines candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        black_candle_threshold: Minimum body size threshold for the black candle (as a fraction of its range).
        white_candle_threshold: Minimum body size threshold for the white candle (as a fraction of its range).
        opening_price_difference_threshold: Maximum allowed difference between the opening prices of the two candles (as a fraction).

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.  Returns an empty Series if the DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]

    body_size_black = abs(df["close"] - df["open"])
    range_black = df["high"] - df["low"]
    body_ratio_black = body_size_black / range_black
    
    body_size_white = abs(df["close"] - df["open"])
    range_white = df["high"] - df["low"]
    body_ratio_white = body_size_white / range_white

    opening_price_difference = abs(df["open"].shift(-1) - df["open"]) / df["open"]

    #Identify black and white candles
    tall_black = is_black & (body_ratio_black >= black_candle_threshold)
    tall_white = is_white & (body_ratio_white >= white_candle_threshold)
    
    #Detect pattern and return
    pattern = tall_black.shift(-1) & tall_white & (opening_price_difference <= opening_price_difference_threshold)
    return pattern


# Ref: https://thepatternsite.com/ShootingStar.html
# This function detects the "Shooting Star (1 line)" candlestick pattern.
# A shooting star is characterized by a small body with a long upper shadow and a short or no lower shadow.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of boolean values indicating the presence of the pattern.
# The parameters allow customization of the thresholds for body size, upper shadow length, and lower shadow length.

def do_detect_shooting_star(df: pd.DataFrame, body_ratio_threshold: float = 0.1, upper_shadow_ratio_threshold: float = 2.0, lower_shadow_ratio_threshold: float = 0.2) -> pd.Series:
    """
    Detects the Shooting Star candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close).
        body_ratio_threshold: Maximum ratio of body size to the total candle range (high - low).
        upper_shadow_ratio_threshold: Minimum ratio of upper shadow length to body size.
        lower_shadow_ratio_threshold: Maximum ratio of lower shadow length to body size.

    Returns:
        Pandas Series with boolean values indicating the presence of the pattern.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body = abs(df["close"] - df["open"])
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    total_range = df["high"] - df["low"]

    is_shooting_star = (body / total_range <= body_ratio_threshold) & \
                       (upper_shadow / body >= upper_shadow_ratio_threshold) & \
                       (lower_shadow / body <= lower_shadow_ratio_threshold)

    return is_shooting_star


# Ref: https://thepatternsite.com/ShootingStar2.html
# This function detects the "Shooting Star (2 lines)" candlestick pattern.
# It identifies two candles: a white candle followed by a small-bodied candle
# with a large upper shadow and a gap between the bodies.  The function returns
# a boolean Series indicating the presence of the pattern.

def do_detect_shooting_star_two_lines(df: pd.DataFrame, upper_shadow_threshold: float = 3.0, gap_threshold: float = 0.001) -> pd.Series:
    """
    Detects the two-line shooting star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        upper_shadow_threshold: Minimum ratio of upper shadow to body size.
        gap_threshold: Minimum gap between the bodies of the two candles (as a fraction of the previous candle's body size).

    Returns:
        Boolean Series indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    body_size = abs(df['close'] - df['open'])
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']
    
    #Check for white candle then a short body
    white_candle = is_white & (body_size > 0)
    small_body = body_size.shift(1) < body_size #shifted back 1 to compare with next candle
    
    #Check for large upper shadow
    large_upper_shadow = upper_shadow.shift(1) >= (upper_shadow_threshold * body_size.shift(1))

    #Check for gap and small lower shadow
    gap = (df['open'] - df['close'].shift(1)) > (gap_threshold * body_size.shift(1))
    small_lower_shadow = lower_shadow.shift(1) <= (body_size.shift(1) / 2)

    #Combine conditions
    pattern = white_candle & small_body & large_upper_shadow & gap & small_lower_shadow
    return pattern

# Ref: https://thepatternsite.com/BlkCandleShort.html
# The short black candle is characterized by a small body and relatively short shadows.
# This function detects this pattern by comparing the body size to the upper and lower shadow lengths.
# The thresholds for body size and shadow length are adjustable via parameters.


def do_detect_short_black_candle(df: pd.DataFrame, body_size_threshold: float = 0.2, shadow_length_threshold: float = 0.5) -> pd.Series:
    """
    Detects the short black candle pattern in OHLC data.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close' columns).
        body_size_threshold: Maximum ratio of body size to total candle range.
        shadow_length_threshold: Maximum ratio of shadow length to total candle range.

    Returns:
        Boolean Series indicating short black candle patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df['close'] - df['open'])
    total_range = df['high'] - df['low']
    upper_shadow = df['high'] - max(df['open'], df['close'])
    lower_shadow = min(df['open'], df['close']) - df['low']

    is_short_body = (body_size / total_range) <= body_size_threshold
    is_short_shadows = (upper_shadow / total_range) <= shadow_length_threshold and (lower_shadow / total_range) <= shadow_length_threshold
    is_black = df['close'] < df['open']

    is_short_black_candle = is_black & is_short_body & is_short_shadows
    return is_short_black_candle


# Ref: https://thepatternsite.com/ShortWhiteCandle.html
# The short white candle is characterized by a small body and short shadows.  It's considered an indecisive candle,
# often appearing when buying and selling pressures are relatively balanced. This function detects this pattern
# based on the candle's body size and shadow lengths relative to a threshold.


def do_detect_short_white_candle(df: pd.DataFrame, body_threshold: float = 0.2, shadow_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Short White Candle candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have 'open', 'high', 'low', 'close' columns.
        body_threshold: Maximum ratio of candle body size to total candle range.
        shadow_threshold: Maximum ratio of upper and lower shadows to the total range.

    Returns:
        A pandas Series with True where the pattern is detected, False otherwise. Index is preserved.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]

    is_short_body = body_size / total_range <= body_threshold
    is_short_shadow = (upper_shadow + lower_shadow) / total_range <= shadow_threshold

    is_short_white_candle = is_white & is_short_body & is_short_shadow
    return is_short_white_candle


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBear.html
# This function detects the "Bearish Side by Side White Lines" candlestick pattern.
# The pattern consists of three candles: a black candle followed by two white candles
# with similar bodies and opening prices, and closing prices below the black candle's body.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans.

def do_detect_bearish_side_by_side_white_lines(df: pd.DataFrame, body_size_ratio_threshold: float = 0.5, opening_price_diff_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bearish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include 'open', 'high', 'low', 'close' columns.
        body_size_ratio_threshold: Maximum ratio between the bodies of the two white candles.
        opening_price_diff_threshold: Maximum difference between the opening prices of the two white candles (as a fraction of the first white candle's opening price).

    Returns:
        Pandas Series: Boolean Series indicating Bearish Side by Side White Lines patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Identify black and white candles
    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']

    # Shift series to access previous day's data
    is_black_shifted = is_black.shift(1)
    is_white_shifted = is_white.shift(1)
    is_white_shifted2 = is_white.shift(2)

    # Calculate candle body sizes
    black_body_size = abs(df['close'] - df['open'])
    white_body_size = abs(df['close'].shift(1) - df['open'].shift(1))
    white_body_size2 = abs(df['close'].shift(2) - df['open'].shift(2))

    # Calculate ratio of white candle body sizes
    body_size_ratio = white_body_size / white_body_size2


    # Calculate difference in opening prices
    opening_price_diff = abs((df['open'].shift(1) - df['open'].shift(2)) / df['open'].shift(2))


    # Check for pattern conditions
    pattern_condition = (is_black_shifted) & (is_white_shifted) & (is_white_shifted2) & (body_size_ratio <= body_size_ratio_threshold) & (opening_price_diff <= opening_price_diff_threshold) & (df['close'].shift(1) < df['open'].shift(1)) & (df['close'].shift(2) < df['open'].shift(1))

    return pattern_condition


# Ref: https://thepatternsite.com/SidebySideWhiteLinesBull.html
# This function detects the "Bullish Side by Side White Lines" candlestick pattern.
# It checks for three consecutive white candles where the last two have similar bodies,
# opening near each other and above the first candle's high.
# The function takes a Pandas DataFrame with OHLCV data as input and returns a boolean Series
# indicating the presence of the pattern.  It is assumed that the DataFrame index is appropriate.

def do_detect_bullish_side_by_side_white_lines(df: pd.DataFrame, body_similarity_threshold: float = 0.5, open_price_diff_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Bullish Side by Side White Lines candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').
        body_similarity_threshold: Threshold for the similarity of body sizes of the last two candles.
        open_price_diff_threshold: Threshold for the difference in opening prices of the last two candles.

    Returns:
        Boolean Series indicating the pattern's presence.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    candle_body = df['close'] - df['open']
    
    #Check for three consecutive white candles
    three_white_candles = is_white & is_white.shift(1) & is_white.shift(2)

    #Check similarity of body sizes of last two candles
    body_similarity = abs(candle_body - candle_body.shift(1)) / candle_body.shift(1) < body_similarity_threshold

    #Check difference in opening prices of last two candles
    open_price_diff = abs(df['open'] - df['open'].shift(1)) / df['open'].shift(1) < open_price_diff_threshold

    #Check if the last two candles open above the high of the first candle.
    opens_above_first_high = df['open'].shift(1) > df['high'].shift(2) & df['open'] > df['high'].shift(2)


    bullish_side_by_side_white_lines = three_white_candles & body_similarity & open_price_diff & opens_above_first_high

    return bullish_side_by_side_white_lines


# Ref: https://thepatternsite.com/SouthernDoji.html
# This function detects the Southern Doji candlestick pattern.
# A Southern Doji is a doji candlestick appearing in a downward trend.
# The function identifies it by checking if the open and close prices are nearly equal
# and if the candle is part of a downtrend.

def do_detect_southern_doji(df: pd.DataFrame, doji_threshold: float = 0.01, downtrend_length: int = 3) -> pd.Series:
    """
    Detects the Southern Doji candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close') and date column ('date').
        doji_threshold: Maximum difference (as a fraction of the candle's range) between open and close prices to qualify as a doji.
        downtrend_length: Minimum number of preceding candles needed to be lower than the previous close price for it to qualify as a downtrend.

    Returns:
        A pandas Series (boolean mask) with True for rows corresponding to Southern Doji patterns, False otherwise.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the difference between open and close prices.
    open_close_diff = abs(df['open'] - df['close'])
    
    # Calculate the range of each candle
    candle_range = df['high'] - df['low']

    # Determine which candles are doji candlesticks using the provided threshold.
    is_doji = (open_close_diff / candle_range) <= doji_threshold
    
    # Identify the downtrend
    is_downtrend = df['close'].shift(1) > df['close']
    is_downtrend = is_downtrend.rolling(downtrend_length, min_periods=downtrend_length).all()

    # Combine conditions to detect Southern Doji pattern.
    is_southern_doji = is_doji & is_downtrend

    return is_southern_doji

# Ref: https://thepatternsite.com/SpinTopBlack.html
# This function detects the "Spinning Top, Black" candlestick pattern.
# A black spinning top is characterized by a small black real body and long upper and lower shadows.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating whether a black spinning top pattern is detected for each row.

def do_detect_black_spinning_top(df: pd.DataFrame, body_ratio_threshold: float = 0.1, shadow_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects black spinning top candlestick patterns.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_ratio_threshold: Maximum ratio of real body size to total candle range (high - low).
        shadow_body_ratio: Minimum ratio of shadow length to body size.

    Returns:
        Pandas Series with boolean values indicating black spinning top patterns.  Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    real_body_size = abs(df["close"] - df["open"])
    total_candle_range = df["high"] - df["low"]
    body_ratio = real_body_size / total_candle_range
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    shadow_size = upper_shadow + lower_shadow
    shadow_body_ratio_test = shadow_size / real_body_size

    is_spinning_top = (body_ratio <= body_ratio_threshold) & (shadow_body_ratio_test >= shadow_body_ratio)

    return is_black & is_spinning_top


# Ref: https://thepatternsite.com/SpinTopWhite.html
# This function detects the "Spinning Top, White" candlestick pattern.
# A white spinning top is characterized by a small white body and tall shadows.
# The body size is considered small relative to the total candle range (high - low).
# The shadows are considered tall if they are significantly longer than the body.
# This function will return a boolean Series indicating the presence of the pattern for each candle.

def do_detect_white_spinning_top(df: pd.DataFrame, body_range_ratio: float = 0.1, shadow_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects White Spinning Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_range_ratio: Maximum ratio of body size to the total candle range (high - low).
        shadow_body_ratio: Minimum ratio of shadow length to body size.

    Returns:
        Boolean Series indicating White Spinning Top patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body = df["close"] - df["open"]
    total_range = df["high"] - df["low"]
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]

    is_white = body > 0
    is_small_body = body / total_range <= body_range_ratio
    is_tall_upper_shadow = upper_shadow / abs(body) >= shadow_body_ratio
    is_tall_lower_shadow = lower_shadow / abs(body) >= shadow_body_ratio


    is_white_spinning_top = is_white & is_small_body & (is_tall_upper_shadow | is_tall_lower_shadow)

    return is_white_spinning_top




# Ref: https://thepatternsite.com/StickSandwich.html
# The Stick Sandwich pattern consists of three candles:
# 1. A black candle in a downtrend.
# 2. A white candle that closes above the previous day's close.
# 3. A black candle that closes at or near the close of the first candle.
# This function detects this pattern.  The thresholds are parameters.


def do_detect_stick_sandwich(df: pd.DataFrame, body_ratio_threshold: float = 0.5, close_diff_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Stick Sandwich candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.
        close_diff_threshold: Maximum difference between the close of the first and third candle as a fraction of the first candle's close price.


    Returns:
        pd.Series: Boolean Series indicating Stick Sandwich patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    
    #Calculate body size
    body_size = abs(df["close"] - df["open"])

    #Shift the data to align for comparisons across candles
    is_black_shifted = is_black.shift(1)
    is_white_shifted = is_white.shift(1)
    is_black_shifted_2 = is_black.shift(2)
    body_size_shifted = body_size.shift(1)
    close_shifted = df["close"].shift(1)
    close_shifted_2 = df["close"].shift(2)

    #Conditions for each candle
    cond1 = is_black & is_white_shifted & is_black_shifted_2
    cond2 = df["close"] > close_shifted
    cond3 = abs(df["close"] - close_shifted_2) / abs(close_shifted_2) < close_diff_threshold
    cond4 = body_size_shifted / body_size > body_ratio_threshold

    # Combine conditions
    pattern = cond1 & cond2 & cond3 & cond4

    return pattern

# Ref: https://thepatternsite.com/TakuriLine.html
# This function detects the Takuri line candlestick pattern.
# A Takuri line is characterized by a small-bodied candle with a lower shadow at least three times the height of the body and little or no upper shadow.
# It's considered a bullish reversal pattern.

def do_calculate_takuri_line(df: pd.core.frame.DataFrame, lower_wick_threshold: float = 3.0, upper_wick_threshold: float = 0.0) -> pd.Series:
    """
    Detects Takuri Line candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        lower_wick_threshold: Minimum ratio of lower wick length to body size.
        upper_wick_threshold: Maximum ratio of upper wick length to body size.

    Returns:
        pd.Series: Boolean Series indicating Takuri Line patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    body_size = abs(df["close"] - df["open"])
    lower_wick_size = df["low"] - df["open"]
    upper_wick_size = df["high"] - df["close"]

    is_takuri_line = (lower_wick_size / body_size >= lower_wick_threshold) & (upper_wick_size / body_size <= upper_wick_threshold)

    return is_takuri_line

# Ref: https://thepatternsite.com/ThreeBlackCrows.html
# This function detects the "Three Black Crows" candlestick pattern.
# The pattern consists of three consecutive tall black candles appearing in an upward trend,
# where each candle opens within the body of the previous candle and closes near its low,
# making new lows along the way.
# The function takes a Pandas DataFrame as input with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# It returns a Pandas Series of boolean values indicating the presence of the pattern for each row.

def do_detect_three_black_crows(df: pd.DataFrame, body_threshold: float = 0.2) -> pd.Series:
    """
    Detects the Three Black Crows candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_threshold: Minimum ratio of body size to candle range for each of the three candles.

    Returns:
        pd.Series: Boolean Series indicating Three Black Crows patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    candle_range = df['high'] - df['low']
    body_ratio = body_size / candle_range
    is_tall = body_ratio >= body_threshold

    three_black_crows = (is_black & is_tall).rolling(window=3, min_periods=3).apply(lambda x: all(x))
    
    #Additional condition: Each candle opens within the body of the previous candle
    open_within_body = (df['open'].shift(1) < df['open']) & (df['open'].shift(1) > df['close'].shift(1))
    open_within_body2 = (df['open'].shift(2) < df['open'].shift(1)) & (df['open'].shift(2) > df['close'].shift(2))
    
    #Additional condition: Closing near lows.
    close_near_low1 = abs(df['close'].shift(2) - df['low'].shift(2)) / candle_range.shift(2) < 0.1
    close_near_low2 = abs(df['close'].shift(1) - df['low'].shift(1)) / candle_range.shift(1) < 0.1
    close_near_low3 = abs(df['close'] - df['low']) / candle_range < 0.1

    three_black_crows = three_black_crows & open_within_body & open_within_body2 & close_near_low1 & close_near_low2 & close_near_low3

    return three_black_crows

# Ref: https://thepatternsite.com/ThreeInsideDown.html
# This function detects the "Three Inside Down" candlestick pattern.
# It identifies three consecutive candles where the second and third candles are inside the first.
# The pattern is confirmed if the third candle closes lower than the second.

def do_detect_three_inside_down(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Three Inside Down candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_ratio_threshold: Minimum ratio of second and third candle bodies to the first candle body.

    Returns:
        A pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_black = df["close"] < df["open"]
    high = df["high"]
    low = df["low"]
    open = df["open"]
    close = df["close"]

    # Calculate candle body sizes
    body_size = abs(close - open)
    body_size_shifted1 = body_size.shift(1)
    body_size_shifted2 = body_size.shift(2)

    # Check for conditions specified in the article
    cond1 = is_white.shift(2)  # First candle is white
    cond2 = is_black.shift(1) & (open.shift(1) > open.shift(2)) & (close.shift(1) < close.shift(2)) #Second candle is black and within the body of the first
    cond3 = (open.shift(1) > open.shift(1)) & (close.shift(1) < close.shift(1)) #Second candle is inside the body of the first.
    cond4 = (open.shift() > open.shift(2)) & (close.shift() < close.shift(2))  # Third candle is inside the body of the first
    cond5 = (low.shift(2) <= low.shift(1)) & (high.shift(2) >= high.shift(1)) #Second candle inside the body of the first candle
    cond6 = (low.shift(2) <= low.shift()) & (high.shift(2) >= high.shift()) #Third candle inside the body of the first candle
    cond7 = close < close.shift(1) #Third candle closes lower than the second
    cond8 = (body_size_shifted1 >= body_ratio_threshold * body_size_shifted2) & (body_size >= body_ratio_threshold * body_size_shifted2)


    three_inside_down = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8

    return three_inside_down

# Ref: https://thepatternsite.com/ThreeInsideUp.html
# This function detects the "Three Inside Up" candlestick pattern.
# It identifies the pattern based on the characteristics described in the provided documentation:
# 1. Three candles.
# 2. A downward price trend leading to the pattern.
# 3. A tall black candle followed by a small white candle whose body is entirely within the body of the previous candle (bullish harami).
# 4. A final white candle that closes above the previous close.

def do_detect_three_inside_up(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Three Inside Up candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.

    Returns:
        A pandas Series of booleans indicating whether a three inside up pattern is present for each row.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    
    #Identify the first candle
    first_candle_is_black = is_black.shift(2)
    
    #Identify the second candle
    second_candle_is_white = is_white.shift(1)
    second_candle_high = df["high"].shift(1)
    second_candle_low = df["low"].shift(1)
    second_candle_open = df["open"].shift(1)
    second_candle_close = df["close"].shift(1)

    #Identify the third candle
    third_candle_is_white = is_white
    
    #Conditions for the second candle
    second_candle_condition = (second_candle_high <= first_candle_high.shift(1)) & (second_candle_low >= first_candle_low.shift(1)) & second_candle_is_white & (body_size.shift(1) <= body_ratio_threshold * body_size.shift(2))
    
    #Combine Conditions
    three_inside_up = first_candle_is_black & second_candle_condition & third_candle_is_white & (df['close'] > second_candle_close)
    return three_inside_up


# Ref: https://thepatternsite.com/ThreeLineStrikeBear.html
# This function detects the bearish three-line strike candlestick pattern.
# It checks for three consecutive black candles with decreasing lows, followed by a tall white candle.
# The white candle should open below the previous day's close and close above the first day's open.

def do_detect_bearish_three_line_strike(df: pd.core.frame.DataFrame, body_threshold: float = 0.2, gap_threshold: float = 0.02) -> pd.core.series.Series:
    """
    Detects the Bearish Three-Line Strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must include 'open', 'high', 'low', 'close' columns.
        body_threshold: Minimum ratio of body size to candle range for each of the three black candles.
        gap_threshold: Minimum percentage gap between consecutive candles.

    Returns:
        pd.Series: Boolean Series indicating Bearish Three Line Strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    is_white = df['close'] > df['open']
    candle_range = df['high'] - df['low']
    body_size = abs(df['close'] - df['open'])
    body_ratio = body_size / candle_range
    
    # Check for three consecutive black candles
    three_black_candles = is_black & is_black.shift(1) & is_black.shift(2)
    
    # Check for decreasing lows
    decreasing_lows = df['low'] < df['low'].shift(1) & df['low'].shift(1) < df['low'].shift(2)
    
    # Check for body ratio threshold in black candles
    black_body_ratio = body_ratio < body_threshold & body_ratio.shift(1) < body_threshold & body_ratio.shift(2) < body_threshold

    # Combine conditions for three consecutive black candles with decreasing lows and body ratio threshold
    three_black_with_lows_and_ratio = three_black_candles & decreasing_lows & black_body_ratio

    # Check for the tall white candle
    white_candle_after_black = is_white.shift(-3)
    
    # Check if white candle opens below the previous close
    opens_below_previous_close = df['open'].shift(-3) < df['close'].shift(-2)

    # Check if white candle closes above the first black candle's open
    closes_above_first_black_open = df['close'].shift(-3) > df['open'].shift(2)
    
    #Combine conditions for the white candle
    white_candle_conditions = white_candle_after_black & opens_below_previous_close & closes_above_first_black_open

    # Combine conditions for three black candles and a white candle
    pattern_detected = three_black_with_lows_and_ratio & white_candle_conditions
    

    return pattern_detected

# Ref: https://thepatternsite.com/ThreeLineStrikeBull.html
# This function detects the bullish three-line strike candlestick pattern.
# It requires three consecutive white candles with increasing closes, followed by a long black candle closing below the open of the first candle.
# The function takes a Pandas DataFrame as input with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a Pandas Series of booleans indicating the presence of the pattern for each row in the DataFrame.  True indicates the pattern is present.


def do_detect_bullish_three_line_strike(df: pd.DataFrame, body_threshold: float = 0.2, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the bullish three-line strike candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must contain 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        body_threshold: Minimum ratio of body size to candle range.
        gap_threshold: Minimum percentage gap between consecutive candles.


    Returns:
        pd.Series: Boolean Series indicating Bullish Three Line Strike patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_white_shifted_1 = is_white.shift(1)
    is_white_shifted_2 = is_white.shift(2)
    close_shifted_1 = df["close"].shift(1)
    close_shifted_2 = df["close"].shift(2)
    open_shifted_2 = df["open"].shift(2)


    three_white_candles = is_white & is_white_shifted_1 & is_white_shifted_2
    increasing_closes = df["close"] > close_shifted_1
    increasing_closes_2 = close_shifted_1 > close_shifted_2
    
    
    # Check if three white candles increasing closes condition is satisfied
    three_white_candles_increasing_closes = three_white_candles & increasing_closes & increasing_closes_2
    

    #Check if fourth candle satisfies the conditions
    is_fourth_candle_black = df['close'] < df['open']
    is_fourth_candle_open_above_first_open = df['open'] > open_shifted_2


    bullish_three_line_strike = three_white_candles_increasing_closes.shift(1) & is_fourth_candle_black & is_fourth_candle_open_above_first_open
    return bullish_three_line_strike

# Ref: https://thepatternsite.com/ThreeOutsideDown.html
# The three outside down candlestick is a bearish reversal pattern.
# It consists of three candles:
# 1. A white (bullish) candle in an upward trend.
# 2. A black (bearish) candle that opens above the previous candle's high and closes below the previous candle's low (engulfing the previous candle).
# 3. Another black candle with a lower close than the second black candle.


def do_detect_three_outside_down(df: pd.core.frame.DataFrame, engulfing_threshold: float = 0.0) -> pd.Series:
    """
    Detects the Three Outside Down candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns: "open", "high", "low", "close", "volume", "date".
        engulfing_threshold: Minimum percentage engulfment required for the second candle to be considered engulfing.

    Returns:
        pd.Series: Boolean Series indicating Three Outside Down patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_black = df["close"] < df["open"]

    #Check for the first candle
    first_white = is_white & (df["close"].shift(2) > df["open"].shift(2))

    # Check for the second candle
    second_black_open_above_high = df["open"] > df["high"].shift(1)
    second_black_close_below_low = df["close"] < df["low"].shift(1)
    second_black_engulfing = (df["high"].shift(1) - df["low"].shift(1)) * engulfing_threshold < (df["open"] - df["close"])
    second_black = is_black & second_black_open_above_high & second_black_close_below_low & second_black_engulfing

    # Check for the third candle
    third_black_lower_close = df["close"] < df["close"].shift(1)
    third_black = is_black & third_black_lower_close


    three_outside_down = first_white & second_black & third_black

    return three_outside_down
# Ref: https://thepatternsite.com/ThreeOutsideUp.html
# This function detects the "Three Outside Up" candlestick pattern.
# It checks for a three-candle sequence where:
# 1. The first candle is a black candle (close < open).
# 2. The second candle is a white candle that opens below the first candle's low and closes above the first candle's high.
# 3. The third candle closes higher than the second candle's close.


def do_detect_three_outside_up(df: pd.DataFrame, body_ratio_threshold: float = 0.5) -> pd.Series:
    """
    Detects the Three Outside Up candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns named "open", "high", "low", "close".
        body_ratio_threshold:  Minimum ratio of the second candle's body size to the first candle's body size.


    Returns:
        pd.Series: Boolean Series indicating Three Outside Up patterns.
        Returns an empty Series if the input DataFrame is empty.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]
    
    #Check for first black candle
    first_candle_black = is_black.shift(2)

    #Check for second white candle: opens below the first candle's low and closes above the first candle's high.
    second_candle_opens_below_first_low = df["open"].shift(1) < df["low"].shift(2)
    second_candle_closes_above_first_high = df["close"].shift(1) > df["high"].shift(2)
    second_candle_white = is_white.shift(1)

    #Check for third candle closing higher than the second candle's close.
    third_candle_higher_close = df["close"] > df["close"].shift(1)
    
    three_outside_up = first_candle_black & second_candle_opens_below_first_low & second_candle_closes_above_first_high & second_candle_white & third_candle_higher_close

    return three_outside_up


# Ref: https://thepatternsite.com/ThreeStarsSouth.html
# This function detects the "Three Stars in the South" candlestick pattern.
# The pattern consists of three consecutive candlesticks:
# 1. A tall black candle with a long lower shadow.
# 2. A smaller black candle with a higher low than the first candle.
# 3. A black marubozu (or a small body) that closes within the high-low range of the second candle.
# The function returns a boolean Series indicating whether the pattern is detected for each row in the input DataFrame.


def do_detect_three_stars_in_the_south(df: pd.DataFrame, body_ratio_threshold: float = 0.2, lower_wick_threshold: float = 0.7, gap_threshold: float = 0.02) -> pd.Series:
    """
    Detects the Three Stars in the South candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_ratio_threshold: Minimum ratio of the body size to the candle range for the three candles.
        lower_wick_threshold: Minimum ratio of the lower wick to total candle range for the first candle.
        gap_threshold: Minimum percentage gap between consecutive open and previous close.


    Returns:
        pd.Series: Boolean Series indicating Three Stars in the South patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_black1 = is_black.shift(2)
    is_black2 = is_black.shift(1)
    is_black3 = is_black
    
    body_size1 = abs(df["close"].shift(2) - df["open"].shift(2))
    candle_range1 = df["high"].shift(2) - df["low"].shift(2)
    lower_wick1 = df["low"].shift(2) - min(df["open"].shift(2), df["close"].shift(2))
    
    body_size2 = abs(df["close"].shift(1) - df["open"].shift(1))
    candle_range2 = df["high"].shift(1) - df["low"].shift(1)

    body_size3 = abs(df["close"] - df["open"])
    candle_range3 = df["high"] - df["low"]

    pattern = (
        is_black1
        & is_black2
        & is_black3
        & (body_size1 / candle_range1 >= body_ratio_threshold)
        & (lower_wick1 / candle_range1 >= lower_wick_threshold)
        & (body_size2 / candle_range2 >= body_ratio_threshold)
        & (df["low"].shift(1) > df["low"].shift(2))
        & (body_size3 / candle_range3 >= body_ratio_threshold)
        & (df["open"] > df["close"].shift(1))
        & (df["close"] < df["high"].shift(1))
        & (df["open"] < df["low"].shift(1))
    )
    return pattern

# Ref: https://thepatternsite.com/ThreeWhiteSoldiers.html
# This function detects the "Three White Soldiers" candlestick pattern.
# The pattern consists of three consecutive tall white candles (close > open),
# each with a close near the high, higher closes, and overlapping bodies.
# The function uses a simplified criteria for detecting the pattern.  More robust checks
# could be added for a more precise detection.


def do_detect_three_white_soldiers(df: pd.core.frame.DataFrame, body_threshold: float = 0.2, close_high_ratio: float = 0.9) -> pd.Series:
    """
    Detects the Three White Soldiers candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        body_threshold: Minimum ratio of body size to candle range.
        close_high_ratio: Minimum ratio of close price to high price for each candle.

    Returns:
        pd.Series: Boolean Series indicating Three White Soldiers patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_size = df["close"] - df["open"]
    candle_range = df["high"] - df["low"]
    body_ratio = body_size / candle_range
    close_to_high_ratio = body_size / (df["high"] - df["close"])

    is_three_white_soldiers = (
        (body_ratio >= body_threshold)
        & (close_to_high_ratio <= (1-close_high_ratio))
        & is_white
        & is_white.shift(1)
        & is_white.shift(2)
        & (df["close"] > df["close"].shift(1))
        & (df["close"].shift(1) > df["close"].shift(2))
        & (df["open"].shift(1) < df["close"])
        & (df["open"].shift(2) < df["close"].shift(1))
    )

    return is_three_white_soldiers


# Ref: https://thepatternsite.com/Thrusting.html
# This function detects the Thrusting candlestick pattern.
# The pattern consists of two candles: a black candle followed by a white candle.
# The white candle opens below the previous day's low and closes near the midpoint of the black candle's body.
# The function returns a boolean Series indicating whether a Thrusting pattern is detected on each day.


def do_detect_thrusting(df: pd.DataFrame, midpoint_threshold: float = 0.05) -> pd.Series:
    """
    Detects the Thrusting candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        midpoint_threshold: The acceptable distance (percentage of the black candle's body) from the midpoint.

    Returns:
        pd.Series: Boolean Series indicating Thrusting patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = df["close"] > df["open"]

    black_candle_low = df["low"].shift(1)
    black_candle_open = df["open"].shift(1)
    black_candle_close = df["close"].shift(1)
    black_candle_body = abs(black_candle_close - black_candle_open)
    black_candle_midpoint = black_candle_open + black_candle_body / 2

    condition1 = is_white
    condition2 = df["open"] < black_candle_low
    condition3 = (df["close"] - black_candle_midpoint).abs() <= black_candle_body * midpoint_threshold
    condition4 = is_black.shift(1)

    return condition1 & condition2 & condition3 & condition4


# Ref: https://thepatternsite.com/TriStarBear.html
# This function detects the bearish tri-star candlestick pattern.
# It identifies three consecutive doji candles, where the middle doji's body is above the bodies of the other two.
# The function uses a simplified definition of a doji (open and close prices are equal).  
# A more robust implementation would consider a tolerance for the difference between open and close prices.

def do_detect_bearish_tri_star(df: pd.DataFrame, doji_tolerance: float = 0.001) -> pd.Series:
    """
    Detects the bearish tri-star candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        doji_tolerance:  Maximum difference between open and close prices to be considered a doji.

    Returns:
        A pandas Series of booleans indicating where the pattern is detected.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_doji = abs(df["open"] - df["close"]) <= doji_tolerance * (df["high"] - df["low"])
    
    # Shift the 'is_doji' series to identify three consecutive dojis.
    is_tri_star = is_doji & is_doji.shift(1) & is_doji.shift(2)

    # Identify the middle doji's body above the others
    middle_body_high = df["high"].shift(1)
    middle_body_low = df["low"].shift(1)
    first_body_high = df["high"].shift(2)
    first_body_low = df["low"].shift(2)
    third_body_high = df["high"]
    third_body_low = df["low"]

    is_middle_above = (middle_body_high >= first_body_high) & (middle_body_low >= first_body_low) & (middle_body_high >= third_body_high) & (middle_body_low >= third_body_low)

    # Combine the conditions to detect the pattern.
    is_bearish_tri_star = is_tri_star & is_middle_above
    
    return is_bearish_tri_star

# Ref: https://thepatternsite.com/TriStarBull.html
# This function detects the bullish tri-star candlestick pattern.
# The pattern consists of three doji candles following a downward trend,
# with the middle doji having a body below the other two.
# The function takes a Pandas DataFrame as input and returns a Pandas Series
# indicating whether a bullish tri-star pattern is present for each row.

def do_detect_bullish_tri_star(df: pd.DataFrame, doji_body_threshold: float = 0.01) -> pd.Series:
    """
    Detects the bullish tri-star candlestick pattern.

    Args:
        df: DataFrame with OHLC data.
        doji_body_threshold: Maximum body size ratio (body/range) for a candle to be considered a doji.

    Returns:
        pd.Series: Boolean Series indicating bullish tri-star patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate the body size and range for each candle
    body_size = abs(df["close"] - df["open"])
    high_low_range = df["high"] - df["low"]

    # Identify doji candles
    is_doji = body_size / high_low_range <= doji_body_threshold

    # Check for three consecutive doji candles
    three_doji = is_doji & is_doji.shift(1) & is_doji.shift(2)

    # Check if the middle doji's body is below the other two
    middle_doji_low = df["low"].shift(1)
    is_tri_star = three_doji & (df["open"].shift(1) > df["open"].shift(2)) & (df["open"].shift(1) > df["close"].shift(2)) & (df["close"].shift(1) < df["open"].shift(2)) & (df["close"].shift(1) < df["close"].shift(2))
    
    return is_tri_star


# Ref: https://thepatternsite.com/TweezersBottom.html
# This function detects the Tweezers Bottom candlestick pattern.
# It identifies two consecutive candles with the same low price.
# The function takes a Pandas DataFrame as input, and returns a Pandas Series
# of boolean values indicating the presence of the pattern for each row.

def do_detect_tweezers_bottom(df: pd.DataFrame, low_diff_threshold: float = 0.0001) -> pd.Series:
    """
    Detects Tweezers Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).
        low_diff_threshold: Maximum difference (as fraction) between lows for a match.

    Returns:
        Pandas Series indicating Tweezers Bottom patterns (True/False).
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    lows = df['low']
    is_tweezers_bottom = (lows == lows.shift(1)).astype(bool) & (abs((lows - lows.shift(1)) / lows) < low_diff_threshold)
    return is_tweezers_bottom


# Ref: https://thepatternsite.com/TweezersTop.html
# Detects the Tweezers Top candlestick pattern.
# The pattern consists of two consecutive candles with approximately the same high, appearing in an uptrend.
# The function checks if the high prices of two consecutive candles are within a specified tolerance.
# It returns a boolean Series indicating whether a Tweezers Top pattern is detected for each candle.


def do_detect_tweezers_top(df: pd.DataFrame, high_tolerance: float = 0.001) -> pd.Series:
    """
    Detects the Tweezers Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must have columns "high".
        high_tolerance: Tolerance for difference in high prices of consecutive candles.

    Returns:
        Boolean Series indicating Tweezers Top patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    high_series = df["high"]
    is_tweezers_top = (high_series - high_series.shift(1)).abs() <= high_series * high_tolerance
    is_tweezers_top = is_tweezers_top & (high_series.shift(1) == high_series.shift(1).fillna(0).max())
    return is_tweezers_top

# Ref: https://thepatternsite.com/TwoBlackGapping.html
# This function detects the "Two Black Gapping" candlestick pattern.
# The pattern consists of two consecutive black candles with a gap between them,
# where the second candle's high is lower than the first candle's high.
# The function takes a Pandas DataFrame as input, containing OHLCV data, and returns a Pandas Series
# indicating whether the pattern is present for each row in the DataFrame.

def do_detect_two_black_gapping(df: pd.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Two Black Gapping candlestick pattern.

    Args:
        df: DataFrame with OHLCV data ('open', 'high', 'low', 'close', 'volume', 'date').
        gap_threshold: Minimum gap between consecutive open prices as a percentage of the previous candle's range.

    Returns:
        Pandas Series with boolean values indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df['close'] < df['open']
    gaps = df['open'].shift(1) < df['low']
    gap_percentages = (df['open'] - df['low'].shift(1)) / (df['high'].shift(1) - df['low'].shift(1))
    sufficient_gap = gap_percentages > gap_threshold

    lower_high = df['high'] < df['high'].shift(1)


    pattern = (is_black & is_black.shift(1) & gaps & sufficient_gap & lower_high).astype(bool)
    
    return pattern


# Ref: https://thepatternsite.com/TwoCrows.html
# This function detects the "Two Crows" candlestick pattern.
# It identifies the pattern based on the characteristics described in the documentation:
# 1. Three candles: a tall white candle followed by two black candles.
# 2. Gap: The second black candle's body gaps above the first candle's body.
# 3. Close: The third black candle closes within the body of the first (white) candle.

def do_detect_two_crows(df: pd.core.frame.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Two Crows candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume, indexed by date.
        gap_threshold: Minimum gap (as a fraction of previous candle's body size) required between candles.

    Returns:
        pd.Series: Boolean Series indicating Two Crows patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    body_size = abs(df['close'] - df['open'])
    
    # Check for three-candle pattern
    pattern = (is_white & is_black.shift(-1) & is_black.shift(-2))

    # Check for gap between the first and second black candles
    gap = (df['open'].shift(-1) - df['close'].shift(-1)) > gap_threshold * body_size.shift(-1)

    #Check if third candle closes within the body of the first candle
    close_within = (df['close'].shift(-2) >= df['open'] ) & (df['close'].shift(-2) <= df['close'])

    # Combine conditions to identify the Two Crows pattern.
    two_crows = pattern & gap & close_within

    return two_crows

# Ref: https://thepatternsite.com/Unique3RiverBottom.html
# This function detects the "Unique Three-River Bottom" candlestick pattern.
# The pattern consists of three candles:
# 1. A tall black candle.
# 2. A smaller black candle whose body is inside the first candle's body, but with a lower low.
# 3. A short white candle below the second candle's body.
# The function returns a boolean Series indicating the presence of the pattern.


def do_detect_unique_three_river_bottom(df: pd.DataFrame, body_ratio_threshold: float = 0.5, lower_wick_threshold: float = 0.7) -> pd.Series:
    """
    Detects the Unique Three-River Bottom candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns named 'open', 'high', 'low', 'close'.
        body_ratio_threshold: Minimum ratio of the second candle's body size to the first candle's body size.
        lower_wick_threshold: Minimum ratio of the lower wick of the second candle to the first candle's range.


    Returns:
        A pandas Series with True where the pattern is detected, False otherwise.  Index is aligned with input df.
        Returns an empty Series if the input DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_black = df["close"] < df["open"]
    is_white = ~is_black
    body_size = abs(df["close"] - df["open"])
    lower_wick = df["low"] - df["open"]  #lower wick of a black candle

    pattern = pd.Series(data=False, index=df.index)

    for i in range(2, len(df)):
        #check for the first tall black candle
        if is_black[i-2] and body_size[i - 2] > 0 and lower_wick[i-2] > 0:
          #check for a smaller black candle inside the first candle body
          if is_black[i-1] and body_size[i-1] > 0 and body_size[i-1] <= body_size[i-2] * body_ratio_threshold and df["low"][i-1] < df["low"][i-2]:
            #Check for the short white candle below the second black candle's body
            if is_white[i] and body_size[i] > 0 and df["close"][i] < df["open"][i-1]:
                pattern[i] = True
    return pattern


# Ref: https://thepatternsite.com/UpGap3Methods.html
# This function detects the "Upside Gap Three Methods" candlestick pattern.
# The pattern consists of three candles: two tall white candles with a gap between them,
# followed by a black candle that fills the gap.  The gap must include a gap between the shadows.
# The black candle's open must be above the prior candle's lower shadow, and its low must be below
# the prior candle's upper shadow.

def do_detect_upside_gap_three_methods(df: pd.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Upside Gap Three Methods candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain 'open', 'high', 'low', 'close' columns.
        gap_threshold: Minimum gap between candles (as a fraction of the previous candle's range).

    Returns:
        pd.Series: Boolean Series indicating Upside Gap Three Methods patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    
    # Calculate candle body ranges
    body_range = df['close'] - df['open']
    body_range_shifted = body_range.shift(1)

    # Calculate daily range and shifted daily range
    daily_range = df['high'] - df['low']
    daily_range_shifted = daily_range.shift(1)

    # Calculate gaps (as fraction of previous candle's range)
    gaps = (df['open'] - df['close'].shift(1)) / daily_range_shifted

    # Identify potential patterns
    potential_patterns = (
        (is_white) &
        (is_white.shift(1)) &
        (is_black.shift(2)) &
        (gaps > gap_threshold) &
        (gaps.shift(1) > gap_threshold) &
        (df['open'].shift(2) > df['low'].shift(1)) &
        (df['high'].shift(1) > df['close'].shift(2)) &
        (df['open'] < df['close'].shift(1)) &
        (df['low'] > df['high'].shift(1))
    )

    return potential_patterns


# Ref: https://thepatternsite.com/UpGapTwoCrows.html
# This function detects the "Upside Gap Two Crows" candlestick pattern.
# The pattern consists of three candles:
# 1. A tall white candle in an upward trend.
# 2. A black candle gapping above the previous candle's body.
# 3. Another black candle engulfing the body of the middle candle,
#    with a close above the first candle's close.

def do_detect_upside_gap_two_crows(df: pd.core.frame.DataFrame, gap_threshold: float = 0.01, body_engulfment_threshold: float = 0.5) -> pd.core.series.Series:
    """
    Detects the Upside Gap Two Crows candlestick pattern.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap (as a percentage of the previous candle's range) between candles.
        body_engulfment_threshold: Minimum ratio of the third candle's body size engulfing the second candle's body.


    Returns:
        A pandas Series of booleans indicating the presence of the pattern.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    is_black = df['close'] < df['open']
    candle_body = df['close'] - df['open']
    candle_range = df['high'] - df['low']

    # Check for the first tall white candle
    first_candle_white = is_white.shift(2)

    # Check for the gap between the first and second candles
    gap = df['open'].shift(1) - df['close'].shift(1)
    gap_condition = (gap / candle_range.shift(1)) > gap_threshold

    # Check for the second black candle
    second_candle_black = is_black.shift(1)

    # Check for engulfment of the second candle by the third candle
    engulfment = (df['high'] > df['open'].shift(1)) & (df['low'] < df['close'].shift(1))
    engulfment_condition = (abs(df['close'] - df['open']) / abs(df['close'].shift(1) - df['open'].shift(1)) ) > body_engulfment_threshold


    #Check that the third candle is black
    third_candle_black = is_black

    # Combine conditions to detect the pattern
    pattern = first_candle_white & gap_condition & second_candle_black & engulfment_condition & third_candle_black

    return pattern


# Ref: https://thepatternsite.com/UpsideTasukiGap.html
# This function detects the Upside Tasuki Gap candlestick pattern.
# It requires a DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
# The function returns a boolean Series indicating the presence of the pattern.

def do_detect_upside_tasuki_gap(df: pd.DataFrame, gap_threshold: float = 0.01) -> pd.Series:
    """
    Detects the Upside Tasuki Gap candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.
        gap_threshold: Minimum gap size as a percentage of the previous candle's range.

    Returns:
        Boolean Series indicating Upside Tasuki Gap patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    is_upward_trend = is_white.rolling(2).apply(lambda x: all(x), raw=True).shift(1)

    # Calculate daily ranges
    daily_range = df["high"] - df["low"]

    # Calculate gaps
    open_gap = df["open"].shift(-1) - df["close"]
    
    #Check if the gaps are significant (adjust threshold as needed)
    significant_gap = open_gap > daily_range.shift(-1)*gap_threshold

    #Check conditions for the third candle (black candle within the gap)
    is_black = df["close"].shift(-2) < df["open"].shift(-2)
    opens_in_body = (df["open"].shift(-2) > df["open"].shift(-1)) & (df["open"].shift(-2) < df["close"].shift(-1))
    closes_in_gap = (df["close"].shift(-2) > df["close"].shift(-1)) & (df["close"].shift(-2) < df["open"].shift(-1) + open_gap)

    # Combine conditions to identify the pattern
    upside_tasuki_gap = is_upward_trend & is_white & is_white.shift(-1) & significant_gap & is_black.shift(-1) & opens_in_body & closes_in_gap

    return upside_tasuki_gap

# Ref: https://thepatternsite.com/volpatterns.html
# This function detects volume patterns based on the provided criteria.  The specific criteria for what constitutes a "volume pattern" 
# are not explicitly defined in the linked resource, so this implementation provides a basic example that could be adapted 
# based on more specific requirements.  This implementation looks for increases in volume that correlate with price increases.

def do_detect_volume_patterns(df: pd.DataFrame, volume_increase_threshold: float = 1.1, price_increase_threshold: float = 1.01) -> pd.Series:
    """
    Detects volume patterns in a DataFrame.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        volume_increase_threshold: Minimum factor by which volume must increase to be considered a pattern.
        price_increase_threshold: Minimum factor by which price must increase to be considered a pattern.


    Returns:
        A pandas Series of booleans indicating whether a volume pattern is detected for each row.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    #Calculate daily price change
    price_change = df['close'] / df['open']
    
    #Calculate daily volume change
    volume_change = df['volume'] / df['volume'].shift(1)
    
    # Detect pattern where volume increases significantly and price increases
    is_pattern = (volume_change >= volume_increase_threshold) & (price_change >= price_increase_threshold)

    return is_pattern


# Ref: https://thepatternsite.com/WhiteCandle.html
# This function detects the White Candle pattern based on the description provided in the documentation.
# A white candle is defined as a candle with a closing price higher than its opening price,
# and shadows (wicks) shorter than the body of the candle.  Thresholds are provided to adjust the sensitivity of the pattern detection.

def do_detect_white_candle(df: pd.DataFrame, body_min_ratio: float = 0.1, shadow_max_ratio: float = 0.5) -> pd.Series:
    """
    Detects the White Candle pattern in a DataFrame.

    Args:
        df: DataFrame with 'open', 'high', 'low', 'close' columns.
        body_min_ratio: Minimum ratio of candle body to total range (default 0.1).
        shadow_max_ratio: Maximum ratio of shadow length to candle body (default 0.5).


    Returns:
        A pandas Series of booleans indicating whether a white candle pattern is detected for each row.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    body_ratio = body / total_range
    upper_shadow = df["high"] - max(df["open"], df["close"])
    lower_shadow = min(df["open"], df["close"]) - df["low"]
    max_shadow = max(upper_shadow, lower_shadow)
    shadow_ratio = max_shadow / body


    is_white_candle = is_white & (body_ratio >= body_min_ratio) & (shadow_ratio <= shadow_max_ratio)

    return is_white_candle


# Ref: https://thepatternsite.com/LongWhiteDay.html
# This function detects the "Long White Day" candlestick pattern.
# A long white day is characterized by a tall white candle with shadows shorter than the body,
# and a body at least three times taller than the average body height over a specified period.

def do_detect_long_white_day(df: pd.DataFrame, body_height_factor: float = 3.0, lookback_period: int = 2) -> pd.Series:
    """
    Detects the Long White Day candlestick pattern.

    Args:
        df: DataFrame with OHLC data.  Must contain columns 'open', 'high', 'low', 'close'.
        body_height_factor: Minimum ratio of candle body to average body height.
        lookback_period: Number of past periods to calculate average body height.

    Returns:
        A pandas Series (boolean mask) indicating Long White Day patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate candle body heights
    body_heights = df["close"] - df["open"]
    is_white = body_heights > 0

    # Calculate average body height over the lookback period
    avg_body_height = body_heights.rolling(window=lookback_period + 1, min_periods=1).mean().shift(1)
    is_long = body_heights > avg_body_height * body_height_factor


    # Check if upper and lower shadows are shorter than the body
    upper_shadows = df["high"] - df["close"]
    lower_shadows = df["open"] - df["low"]
    is_short_shadow = (upper_shadows < body_heights) & (lower_shadows < body_heights)

    # Combine conditions
    is_long_white_day = is_white & is_long & is_short_shadow


    return is_long_white_day


# Ref: https://thepatternsite.com/WhiteMarubozu.html
# This function detects the White Marubozu candlestick pattern.
# A White Marubozu is a tall white candle with no shadows.  It is considered a continuation pattern.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans,
# indicating whether each candle is a White Marubozu or not.


def do_detect_white_marubozu(df: pd.DataFrame, body_threshold: float = 0.001) -> pd.Series:
    """
    Detects White Marubozu candlestick pattern.

    Args:
        df: DataFrame with OHLC data ('open', 'high', 'low', 'close').
        body_threshold: Minimum body size relative to the total candle range.

    Returns:
        Pandas Series: Boolean Series indicating White Marubozu patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df['close'] > df['open']
    upper_shadow = df['high'] - df['close']
    lower_shadow = df['open'] - df['low']
    total_range = df['high'] - df['low']
    is_marubozu = (upper_shadow / total_range) < body_threshold
    is_marubozu &= (lower_shadow / total_range) < body_threshold

    return is_white & is_marubozu


# Ref: https://thepatternsite.com/SpinTopWhite.html
# This function detects the White Spinning Top candlestick pattern.
# A white spinning top is characterized by a small white body and tall shadows.
# The body size is considered small relative to the overall candle range (high - low).
# The shadows are considered tall relative to the body size.
# The function returns a boolean Series indicating the presence of the pattern.


def do_detect_white_spinning_top(df: pd.DataFrame, body_ratio_threshold: float = 0.1, wick_body_ratio: float = 2.0) -> pd.Series:
    """
    Detects White Spinning Top candlestick pattern.

    Args:
        df: DataFrame with OHLC data and volume.  Must have columns named 'open', 'high', 'low', 'close', 'volume', and 'date'.
        body_ratio_threshold: Maximum ratio of body size to the total candle range (high - low).
        wick_body_ratio: Minimum ratio of upper or lower wick length to body size.

    Returns:
        Boolean Series indicating White Spinning Top patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    is_white = df["close"] > df["open"]
    body_size = abs(df["close"] - df["open"])
    total_range = df["high"] - df["low"]
    upper_wick = df["high"] - max(df["open"],df["close"])
    lower_wick = min(df["open"],df["close"]) - df["low"]

    is_small_body = body_size / total_range <= body_ratio_threshold
    is_tall_upper_wick = upper_wick / body_size >= wick_body_ratio
    is_tall_lower_wick = lower_wick / body_size >= wick_body_ratio
    is_spinning_top = is_small_body & (is_tall_upper_wick | is_tall_lower_wick)

    return is_white & is_spinning_top


# Ref: https://thepatternsite.com/FallingWindow.html
# This function detects the "Falling Window" candlestick pattern.
# A falling window is characterized by a gap in a downward trend where yesterday's low is above today's high.
# The function takes a Pandas DataFrame as input and returns a Pandas Series of booleans indicating the presence of the pattern.

def do_detect_falling_window(df: pd.core.frame.DataFrame, gap_threshold: float = 0.0) -> pd.core.series.Series:
    """
    Detects Falling Window candlestick pattern.

    Args:
        df: DataFrame with OHLC data (open, high, low, close, volume, date).  Must not be modified.
        gap_threshold: Minimum gap size as a fraction of the previous day's range. Defaults to 0 which means any gap counts.

    Returns:
        pd.Series: Boolean Series indicating Falling Window patterns.
    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate daily range
    daily_range = df['high'] - df['low']

    # Shift yesterday's low
    yesterdays_low = df['low'].shift(1)

    # Calculate the gap
    gap = yesterdays_low - df['high']

    # Check for gap condition and downward trend.  Note that we don't check for a downward trend here.
    is_falling_window = (gap > daily_range.shift(1) * gap_threshold)

    return is_falling_window


# Ref: https://thepatternsite.com/RisingWindow.html
# Detects the Rising Window pattern.  A rising window is a gap in an upward trend where yesterday's high is below today's low.
# This function identifies this pattern based on the OHLC data provided.

def do_calculate_rising_window(df: pd.DataFrame, gap_threshold: float = 0.0) -> pd.Series:
    """
    Detects Rising Window candlestick pattern.

    Args:
        df: DataFrame with OHLC data, including 'open', 'high', 'low', 'close', 'volume', and 'date' columns.
        gap_threshold: Minimum gap between consecutive days' high and low (as a fraction of previous day's range).

    Returns:
        pd.Series: Boolean Series indicating Rising Window patterns.  Returns an empty Series if the DataFrame is empty.

    """
    if df.empty:
        return pd.Series([], dtype=bool)

    # Calculate daily ranges
    daily_range = df['high'] - df['low']

    # Shift yesterday's high
    yesterday_high = df['high'].shift(1)

    # Check for rising window condition
    is_rising_window = (df['low'] > yesterday_high) & (daily_range.shift(1) > 0) #Avoid division by zero

    # Apply gap threshold (optional)
    if gap_threshold > 0:
        gap_size = df['low'] - yesterday_high
        is_rising_window = is_rising_window & (gap_size / daily_range.shift(1) >= gap_threshold)

    return is_rising_window

In [19]:
class OptimizerDefinition:
    def __init__(self, name: str, parameters: typing.Dict[str, "ParameterType"], optimization_function, factory=None): 
        if not isinstance(name, str):
            raise TypeError("name must be a string")

        if not isinstance(parameters, dict):
            raise TypeError("parameters must be a dictionary")

        if not all(isinstance(param, ParameterType) for param in parameters.values()):
            raise TypeError("All values in parameters must be ParameterType objects")

        if len(set(parameters.keys())) != len(parameters.keys()): # Check for duplicate keys
            raise ValueError("Parameter names must be unique.")

        if not callable(calculation_function):
            raise TypeError("calculation_function must be callable")

        self.name = name
        self.parameters = parameters
        self.calculation_function = calculation_function
        self.factory = factory

    def create_optimizer(self, **kwargs: typing.Any):
        params = {}
        for name, param_def in self.parameters.items():
            value = kwargs.get(name)

            if value is None:
                value = param_def.get_default()

            if param_def.data_type == "integer" and not isinstance(value, int):
                raise TypeError(f"Value for parameter '{name}' must be an integer")
            elif param_def.data_type == "real" and not isinstance(value, (int, float)):
                raise TypeError(f"Value for parameter '{name}' must be a number")
            elif param_def.data_type == "boolean" and not isinstance(value, bool):
                raise TypeError(f"Value for parameter '{name}' must be a boolean")
            elif param_def.data_type == "string" and not isinstance(value, str):
                raise TypeError(f"Value for parameter '{name}' must be a string")
            elif param_def.data_type in ("integer", "real"):
                if param_def.min_val is not None and value < param_def.min_val:  # Check min_val
                    raise ValueError(f"Value for parameter '{name}' must be greater than or equal to {param_def.min_val}")
                if param_def.max_val is not None and value > param_def.max_val:  # Check max_val
                    raise ValueError(f"Value for parameter '{name}' must be less than or equal to {param_def.max_val}")

            if param_def.data_type == "string" and param_def.allowed_strings is not None and value not in param_def.allowed_strings:
                raise ValueError(f"Value {value} is not in allowed strings for parameter {name}")

            params[name] = value

        return OptimizerInstance(self.name, params, self)

    def calculate(self, data: pd.DataFrame, params: typing.Dict[str, typing.Any]) -> pd.DataFrame:
        """
        Calculates the optimization using the provided data and parameters.
        """
        kwargs = params.copy() 
        return self.optimization_function(data, **kwargs)

    def __repr__(self):
        return f"OptimizerDefinition(name='{self.name}', parameters={self.parameters}, calculation_function={self.calculation_function.__name__ if hasattr(self.calculation_function, '__name__') else str(self.calculation_function)}, factory={self.factory})"

In [20]:
def do_find_extrema(df: pd.DataFrame, threshold: float = 0.005) -> pd.Series:
    highs = df['high'].values
    lows = df['low'].values
    idx = df.index

    result = np.zeros(len(df), dtype=int)
    trend = None
    last_extreme_price = lows[0]
    last_extreme_index = 0

    for i in range(1, len(df)):
        high = highs[i]
        low = lows[i]

        if trend is None:
            # Determine the initial direction
            if high >= last_extreme_price * (1 + threshold):
                trend = 'up'
                last_extreme_price = high
                last_extreme_index = i
            elif low <= last_extreme_price * (1 - threshold):
                trend = 'down'
                last_extreme_price = low
                last_extreme_index = i

        elif trend == 'up':
            if high > last_extreme_price:
                last_extreme_price = high
                last_extreme_index = i
            elif low <= last_extreme_price * (1 - threshold):
                # Mark the previous high as peak
                result[last_extreme_index] = 1
                trend = 'down'
                last_extreme_price = low
                last_extreme_index = i

        elif trend == 'down':
            if low < last_extreme_price:
                last_extreme_price = low
                last_extreme_index = i
            elif high >= last_extreme_price * (1 + threshold):
                # Mark the previous low as valley
                result[last_extreme_index] = -1
                trend = 'up'
                last_extreme_price = high
                last_extreme_index = i

    return pd.Series(result, index=idx, name='zigzag')
